The idea of this script is to re-formulate the claims into a coherent format (depending on the claim type (text, image, AI-image, altered image)) and add necessary context information to the claim that has not been covered when the original claim was shared on social media, e.g., a date or place of an event. The reformulation will either be done by prompting a LLM or manually. The decision depends on which of the two approaches is more efficient. 

1) I will start with the reformulation of claims with AI-generated and altered images, because these claims need a re-formulation due to the re-labeling in the label aggregation (see the "claims_postprocessing_label_aggregation.ipynb" script). 

2) I will proceed with the reformulation of claims with normal images. Some of these claims explicitly refer to an image in their claim, while most do not. All of the claims should be re-formulated in a standardized format which explicilty mentions the image. For the formatting, I will look into the claim formulation of the VERITE dataset.

3) Lastly, I will proceed with the reformulation of text-only claims. This is done lastly, because some of these claims might not need a re-formulation. The idea here is as well to have a coherent format, that all claims cover the same key information that is necessary for verification, e.g., time, place, context of an event. 

## 1) Gaza-Israel Dataset

In [2]:
### Loading relevant libraries and the gaza-israel dataset

import pandas as pd

df_gaza_israel = pd.read_csv("../gaza_israel/Combined_dataset/gaza_israel_dataset_combined_010724_300425_after_methodology_check_after_label_aggregation.csv", index_col=0)
print(len(df_gaza_israel))
print(df_gaza_israel.dtypes)
df_gaza_israel.head()

100
id                            int64
Website                      object
Article_URL                  object
Headline                     object
Claim_Date                   object
Review_Date                  object
Query/Keyword                object
Label_Website                object
Image_URL                    object
Original_Claim_Website       object
Original_Claim_Only          object
Context/Label_Explanation    object
Text_Only_Claim                bool
Normal_Image                   bool
AI_Generated_Image             bool
Altered_Image                  bool
Data_Collection_Type         object
Label                        object
dtype: object


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in ...,2025-04-06 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://pbs.twimg.com/media/Gn4wjgBXoAALax1?fo...,"""BREAKING: Reports confirm the death of Genera...","""BREAKING: Reports confirm the death of Genera...",A reverse image search reveals the photograph ...,False,True,False,False,Manual,False
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.39432WD,"Pictures show Gaza in ruins, not Myanmar quake...",2025-03-29 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://web.archive.org/web/20250402070342im_/...,"""A 7.9-magnitude earthquake in Myanmar, devast...","""A 7.9-magnitude earthquake in Myanmar, devast...","However, a reverse image search for the first ...",False,True,False,False,Manual,False
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.37W26WD,Image of 'Gazan mother with son's bones' is ma...,2025-03-24 00:00:00,2025-03-27 00:00:00,Israeli-Palestinian conflict,false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.308...,"""A mother holds the bones of her son in Gaza. ...","""A mother holds the bones of her son in Gaza. ...",A keyword search found a wider-angle version o...,False,False,True,False,Manual,False
3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36XY2FV,Old image misrepresented as 'Israeli settlers'...,2025-02-16 00:00:00,2025-02-19 00:00:00,Israeli-Palestinian conflict,false,https://pbs.twimg.com/media/Gj5tbXiaQAAoG1z?fo...,"""Israeli media reports there are Jews who have...","""Israeli media reports there are Jews who have...",A reverse image search via Google traced the v...,False,True,False,False,Manual,False
4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36RF6BZ,"Picture shows Israel's 2018 Gaza attack, not a...",2024-12-22 00:00:00,2024-12-26 00:00:00,Israeli-Palestinian conflict,false,https://media.gettyimages.com/id/1060718622/de...,"""Americans bombarded Yemen in reprisal after Y...","""Americans bombarded Yemen in reprisal after Y...",But a reverse image search on Google found the...,False,True,False,False,Manual,False


### 1.1) Claims with AI-/altered images

- Open questions: 
    - Use a date or not? Since, the AI-images don't have a real date. The altered images have a real date in the sense that the original, unaltered, real image should have a date of origin. 
    - Testing different claim reformulations or only one? (e.g., *This is an authentic image of...*, *This is a real image of ....*, *This image authentically shows.....*)

In [3]:
## Have a look at all claims with AI-images

pd.set_option("max_colwidth", None)

print(len(df_gaza_israel[df_gaza_israel["AI_Generated_Image"]==True]))

df_gaza_israel[df_gaza_israel["AI_Generated_Image"]==True]

9


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.37W26WD,Image of 'Gazan mother with son's bones' is made with AI,2025-03-24 00:00:00,2025-03-27 00:00:00,Israeli-Palestinian conflict,false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/486290815_1290888865331200_3774810229142652666_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=127cfc&_nc_ohc=dJB5KMYY-K8Q7kNvwG36ECt&_nc_oc=Adm1o8yPbYbSNkcgvalWokrqaqaaHITQDp4TW4yh1603BysZ0yTW6m3OzdfFRAcVyc0&_nc_zt=23&_nc_ht=scontent-hou1-1.xx&_nc_gid=7AoU9LU1o_p7jCObsgUWlQ&oh=00_AfH7GWH4PBqeBawSJ9_R87ZfrG1kYAoZjldaIHBiuECusg&oe=681E8A2A,"""A mother holds the bones of her son in Gaza. After more than a year and a half of searching for him under the rubble of their bombed-out house,"" reads a Malay-language Facebook post shared on March 24.","""A mother holds the bones of her son in Gaza. After more than a year and a half of searching for him under the rubble of their bombed-out house.""","A keyword search found a wider-angle version of the image first shared March 21 on Instagram alongside the disclaimer: ""This visual representation is an AI-generated image created for illustrative purposes"" (archived link). Watermarks that say ""in.visualart"" on the upper left side and ""designed by in.visualart"" on the lower left part have been cropped out in the circulating posts. The primary Instagram account sharing the image -- called ""in.visualart"" -- describes itself as a ""designer"" and labels its content as ""AI tools"" and ""AI-generated image"". Shu Hu, head of Purdue University's Machine Learning and Media Forensics Lab in the United States, told AFP the image was AI-generated (archived link). Hu pointed out the woman's blurry teeth in the image and said that there was ""no obvious segmentation"", one of the hallmarks of AI-generated imagery. While there is no foolproof method to spot AI-generated media, identifying watermarks and visual inconsistencies can help, as errors still occur despite the meteoric progress in generative AI.",False,False,True,False,Manual,False
6,6,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36MV6KT,AI-generated image used to falsely claim Netanyahu hospitalized,2024-11-17 00:00:00,2024-11-20 00:00:00,Israeli-Palestinian conflict,false,https://scontent-dfw5-2.xx.fbcdn.net/v/t39.30808-6/467574066_10159703629306377_2271087948666621641_n.jpg?_nc_cat=106&ccb=1-7&_nc_sid=833d8c&_nc_ohc=sLkWj06JNSIQ7kNvwF8V9Ks&_nc_oc=AdmX4eGfRA00g0oakzPCxbU37qVHOnj-5s0mPTUOMhTj-uJgcy4XGkbqrL2QVkvRdLk&_nc_zt=23&_nc_ht=scontent-dfw5-2.xx&_nc_gid=GxsCLpbhYjt7pSBgkFmzkQ&oh=00_AfHBauCQ4-oFrtT0kySyUl9ET2-BwofVsasFfT3vNsPXdg&oe=681E8BB2,"""BREAKING NETANYAHU HOSPITALISED He suffered serious health issue in Kiryah and has been shifted to a hospital in Tel Aviv,"" says a November 17 X post from Sulaiman Ahmed, a creator who has monetized misinformation about Israel's war with Hamas.","""BREAKING NETANYAHU HOSPITALISED He suffered serious health issue in Kiryah and has been shifted to a hospital in Tel Aviv.""","However, November 2024 social media posts claiming he was hospitalized with a ""serious health issue"" are false; the Israeli leader gave a speech to lawmakers when he was supposedly admitted, and experts told AFP the image shared online is likely AI-generated. The photo being shared of Netanyahu was likely generated by artificial intelligence, experts said. Hany Farid, a digital forensics expert at the University of California-Berkeley (archived here), said he analyzed the image using three models trained to identify pictures created using artificial intelligence, all of which classified it as ""likely"" being AI-generated. He told AFP on November 19 that he compared the image with actual photographs of Netanyahu and found the left ear was

**Deciding on using dates or not** (key question: Does the shared AI-image relates to a specific event or suggest to relate to a specific event?):

- claim ("id": 2): -> USE DATE
    - The image relates to the current war/genocide in Gaza 
    - Since there have been several wars in Gaza in the past, the claim should use a date to specify that it relates to the current one 
    - The image was shared on March, 24, 2025. According to the label explanation of AFP Factcheck, the AI image was generated in March, 21, 2025. Thus, the monthly date will be used.
    - Claim could be reformulated in: 
        "This is an authentic image of a mother holding the bones of her son in Gaza in March 2025. After more than a year and a half of searching for him under the rubble of their bombed-out house."


- claim ("id": 6): -> USE DATE
    - Image was shared in November 2024 (when Netanyahu was not hospitalized)
    - Netanyahu was hospitalized a month later in December 2024 due to prostate surgergy (https://apnews.com/article/netanyahu-israel-health-prostate-war-ffa0430d621307d34a673a0428ece945)
    - if date is used it should use November 2024 and not conflate with December 2024 
    - The claim should use a date, since Netanyahu has been hospitalized several times in the past. So the claim and image relates to one specific hospitalization (as it is suggested in the claim shared on social media)
    - Could be reformulated in:
        "This is an authentic image of Benjamin Netanyahu being hospitalized in Tel Aviv in November 2024 due to a serious health issue."


- claim ("id": 10): -> USE DATE
    - Image was shared in October 2024, also a spokeperson of the airline answered in October:
        - 'A spokesperson for Middle East Airlines told AFP in an October 21 email that the images appear to be AI-generated and are "far from reality."'
    - Since the image suggests it would be related to the current, since Oct. 2023 escalating wars in Gaza and Lebanon, a date should be used -> use the month 
    - Could be reformulated in: 
        - "This is an authentic image of Israel bombing Beirut's International Airport in October 2024." 
    

- claim ("id": 19): -> USE DATE
    - Social media posts shared the image as an authentic scene during Netanyahu’s February 2025 official visit to the U.S.
    - Thus, the reformulated claim should **use a date** 
    - Could be reformulated in: 
        "This is an authentic image of Donald Trump, Benjamin Netanyahu and Elon Musk during Netanyahu's official visit to the U.S. in February 2025."

- claim ("id": 22): -> USE DATE
    - Image was shared on social media ahead/in the context of Mike Tyson's fight against Jake Paul
    - Social media posts, featured a photo of the 58-year-old former heavyweight champion, purportedly in the ring for his Nov. 15 fight with Jake Paul, wearing the Palestinian flag draped on his shoulders and captioned: "Mike Tyson poses with the Palestinian flag before his historic fight."
    - Thus, the image suggests Mike Tyson wore the flag in the ring right before the fight started.
    - Thus, the reformualted claim should **use a date**
    - Could be reformulated in:
        "This is an authentic image of Mike Tyson posing with the Palestinian flag in the ring before his fight against Jake Paul on November, 15, 2024."

- claim ("id": 31): -> DON'T USE DATE
    - Image was shared in December 2024, but does not relate to a specific event 
    - Thus, no date needs to be used 

- claim ("id": 79): -> USE DATE
    - Times of India shared the image in a report from Sep, 24, 2024 of an alleged second attack Israeli Air Force's Ramat David airbase (https://www.youtube.com/watch?v=C8uxToCz1jA)
    - Hezbollah did indeed attack the Ramat David airbase on Sep, 22, 2024 (https://www.politico.eu/article/hezbollah-launches-revenge-rocket-attack-against-israeli-base/, https://www.jpost.com/breaking-news/article-821184, https://www.msn.com/en-in/news/world/israel-s-ramat-david-airbase-hit-first-attack-on-key-idf-facility-since-october/vi-AA1qYDgw)
    - Thus, the reformulated claim should **use a date** -> the exact day or month?
    - Could be reformulated in:
        "This is an authentic image of Israeli Air Force's Ramat David airbase that was attacked by Hezbollah on September, 22, 2024." 

- claim ("id": 98): -> DON'T USE DATE
    - The claim and image does not relate to a specific event. 
    - Thus, no date will be used. 

- claim ("id": 99): -> USE DATE
    - The claim and image was shared in July 2024
    - The image relates to the real event of Israel using army dogs unleadhing on eldery palestinian woman (https://www.youtube.com/watch?v=reF-iLVWWhs). The date of the video is from June 26, 2024. 
        - In the video it is stated that the attack happened "a few weeks ago."
    - There also news report (June, 26, 2024) on this incident (https://www.palestinechronicle.com/israelis-set-a-dog-on-me-video-shows-army-unleashing-dog-on-elderly-palestinian-woman/) which show the video shared on X on June, 25, 2024. 
    - Since the specific day of the attack cannot be obtained, but the attack very likely happened in June 2024, this month will be used.
    - Could be reformulated in: 
        "This is an authentic image of an elderly Palestinian women being attacked by an Israeli army dog in June 2024."



- For all claims either use: "image of" or "image that shows"
- Since some of the re-formulated claims use a date and some do not and the sample size of all claims with AI-generated images is small (n=9), the reformulation will be done manually and not with a LLM (creating a good prompt that results in high-quality results would take too long)


In [4]:
## Have a look at all claims with altered images

print(len(df_gaza_israel[df_gaza_israel["Altered_Image"] == True]))

df_gaza_israel[df_gaza_israel["Altered_Image"] == True]

7


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
26,26,Reuters,https://www.reuters.com/fact-check/jd-vances-x-profile-did-not-display-pro-israel-message-2024-07-24/,Fact Check: J.D. Vance’s X profile did not display pro-Israel message,2024-07-17 00:00:00,2024-07-24 00:00:00,"""Gaza Fact Check""",altered,https://pbs.twimg.com/media/GSp95vsXMAAo3WB?format=jpg&name=large,"A Facebook post, sharing the fabricated screenshot added the comment, “ANOTHER AMERICA LAST Politician” and quoted the original X post’s remark: “They don't even hide it. Vance has already updated his profile to align with his objectives.”",“They don't even hide it. Vance has already updated his profile to align with his objectives.”,"That remark and the fake screenshot were originally shared by a satirical account called “Decentralise Party,” whose account bio, states: “Satirical political party aiming to develop a parallel economy by rejecting the social contract. Some satirical articles and posts about conspiracy theories.”",False,False,False,True,Manual,False
30,30,Reuters,https://www.reuters.com/fact-check/headline-hamas-protecting-bibas-family-is-fake-not-bbc-2025-02-28/,"Fact Check: Headline on Hamas 'protecting' Bibas family is fake, not BBC",2025-02-23 00:00:00,2025-02-28 00:00:00,"""Israel Fact Check""",false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/493524900_1584488289133073_3353003646308446607_n.jpg?stp=dst-jpg_s1080x2048_tt6&_nc_cat=105&ccb=1-7&_nc_sid=127cfc&_nc_ohc=pI7JzIvkw5gQ7kNvwE3fQQ5&_nc_oc=Adkok-_0cwov-cfWLbsTrs22mg0Z7Kv50FO3Vjb8gsD_wNjP_dl_iw0TvsUdMLS0UPQ&_nc_zt=23&_nc_ht=scontent-hou1-1.xx&_nc_gid=uWJddw6l-YTVa0dTdff00A&oh=00_AfFFuCZFCXy9wnBg4BXQlW0k0vdO_S3DwWMhx7klCq3Q-Q&oe=681E89B2,"In mid-February, an image, using BBC News branding and mimicking a screenshot of a headline on the mobile version of the BBC website was shared on social media. The headline reads: “Hamas just wanted to protect the Bibas family but Israel killed them - Hamas said.”",“Hamas just wanted to protect the Bibas family but Israel killed them - Hamas said.”,A BBC spokesperson said in an email that the screenshot does not show a genuine headline published by the outlet.No such headline can be found on the https://www.bbc.com/.\r\n,False,False,False,True,Manual,False
45,45,Snopes,https://www.snopes.com/fact-check/israel-greta-thunberg-sign/,Greta Thunberg Was Photographed Holding 'F*** Israel' Sign?,2024-11-11 00:00:00,2024-11-14 00:00:00,"""Gaza""",fake,https://mediaproxy.snopes.com/width/1200/https://media.snopes.com/2024/11/greta_isreal_sign_featured.png,"A photograph shared to social media in November 2024 authentically showed Greta Thunberg holding a sign reading, ""F*** Israel.”","""A photograph shared to social media in November 2024 authentically showed Greta Thunberg holding a sign reading, 'F*** Israel.'""","However, others pointed out discrepancies in the image and suggested it was digitally edited or created with the use of artificial intelligence software. They were correct. The image was a cropped and manipulated version of a real photo of Thunberg holding a sign reading, ""Stand With Gaza."" We have labeled the in-question image fake. We reached out to the author of the X post above to ask if they digitally created the image — or, if not, if they could share where they found it. We will update this report if we receive a response. Thunberg shared the original, unaltered photo on Oct. 20, 2023. The post's caption expressed solidarity with the Palestinian territories and called for an immediate ceasefire.",False,False,False,True,Manual,False
72,72,usatoday.com,https://www.usatoday.com/story/news/factcheck/2024/12/13/new-york-times-headline-luigi-mangione-anti-israeli-fact-check/76965562007/,New York Times didn't publish 'anti-Israeli' headline abo

**Deciding on using dates or not** (key question: Does the shared AI-image relates to a specific event or suggest to relate to a specific event?):

- claim ("id": 26) -> DON'T USE DATE
    - The claim does not refer to a specific event
    - The altered screenshot does not contain a date. It just shows the supposedly X profile of J.D. Vance
    - Thus, a date is not necessary.
    - Claim could be reformulated to:
        "This image is an authentic screenshot of J.D. Vance's X profile that says 'I stand with Israel'."

- claim ("id": 30) -> USE DATE
    - The altered screenshot of a supposedly BBC headline does not show a date
    - The claim and image started circulating on social media in February 2025 
    - Since the claim refers to an article a date should be used. 
    - Claim could be reformulated to:
        - "This image is an authentic screenshot of a BBC headline from February 2025 saying that 'Hamas just wanted to protect the Bibas family but Israel killed them - Hamas said.'"
        - "The image shows a screenshot of an authentic BBC headline from February 2025 saying 'Hamas just wanted to protect the Bibas family but Israel killed them - Hamas said.'"

- claim ("id": 45) -> USE DATE
    - The original claim already specifies a date: November 2024 
    - The unedited, real image is from October 2023 
    - Keep the November 2024 date, since the altered image was shared and circulated in November 2024 
    - Using the Image_URL of the uncensored image (https://pbs.twimg.com/media/GcHZB3-WEAAecSV?format=jpg&name=large) instead of the censored image (https://mediaproxy.snopes.com/width/1200/https://media.snopes.com/2024/11/greta_isreal_sign_featured.png)
    - Claim could be reformulated to:
        "This is an authentic image of Greta Thunberg from November 2024 holding a sign reading, 'Fuck Israel'". 
    

- claim ("id": 72) -> USE DATE
    - Claim date is from December, 13, 2024
    - The doctored screenshot also has an article date in it: December, 12, 2024
    - The original article (whose title has been changed from "What the Glorification of Luigi Mangione Reveals About America" to "Why the Glorification of Luigi Mangione Is Inherently Anti-Israeli") is also from December, 12, 2024
    - Thus, this date will be used 
    - Claim could be reformulated to:
        - "This image is an authentic screenshot of a New York Times headline from December, 12, 2024 with the title "Why the Glorification of Luigi Mangione Is Inherently Anti-Israeli."
        - "The image shows a screenshot of an authentic New York Times headline...."


- claim ("id": 76) -> DON'T USE DATE
    - The altered screenshot does not have a date
    - The origianl full text of the screenshot is: "soon the end and they know it, that's why they want to destroy everything in the way." 
    - The screenshot also has a flag in the color of Israel that shows an hourglass
    - Claim could be reformulated to:
        - "The image shows an authentic screenshot of a social media post from Mel Gibson saying that Israel will end soon and therefore destroy everything in its way together with a flag in the color of Israel that shows an hourglass."
        - "The image shows a screenshot of an authentic social media post from Mel Gibson saying that Israel will end soon and therefore destroy everything in its way together with a flag in the color of Israel that shows an hourglass."
    
    
- claim ("id": 85) -> DON'T USE DATE
    - The altered screenshot itself does not show any date
    - The claim circulated on social media in August 2024 following the circulation of a leaked video showing Israeli soldiers sexually assaulting a detainee from Gaza at the Sde Teiman detention camp,
    - The poll was originally conducted by Channel 12 in May 2022
    - Do not use a date. The model should figure out that the image is doctored and the original poll from May 2022.
    - Claim could be reformulated to:
        - "The image shows a screenshot of an authentic question from Israel's Channel 12 public opinion poll: 'Dou you agree with the claim that a solder is allowed to rape a prisoner with his hands tied?'"
        - "The image shows an authentic screenshot of a question from Israel's Channel 12 public opinion poll: 'Dou you agree with the claim that a solder is allowed to rape a prisoner with his hands tied?'"


- claim ("id": 88) -> DON'T USE DATE
    - The claim with the altered image was shared in August 2024 on social media
    - The original image is from February 2024
    - Since the image does not relate to a specific article or event, no date will be used
    - The model should figure out itself that the image is altered and find the original image
    - Claim could be reformulated to:
        - "This is an authentic iamge of Esmail Qaani, the commander of the Iranian Revolutionary Guards, disguised in a hijab in fear of Israeli bombing."




- Claim formulation regarding screenshots, either:
    - "authentic screenshot of...."     or:
    - "screenshot of authentic headline/question/title ...."


- Making sure the format of the claim reformulation is coherent for all claims with altered images and also similar to format of claims with AI-images.





In [5]:
### Using a dictionary to store the manually reformulated claims for AI- and altered images to later add to the dataframe
#### using the indices of the claim for mapping


""" 

General rules/format for manual reformulation of claims with AI-/altered images:

1) If the claim relates to an image (and not a screenshot of something), it will be reformulated into:
    - 'This is an authentic image of .....'
    - Since all of the claims with AI-/altered images have the label "False", this claim reformulation fits to the label
      and it tests the ability of the model to detect the fabricated nature of the image 

2) If the claim relates to a screenshot of a digitally altered content, e.g., newspaper articles, it will be reformulated into:
    - 'This image shows a screenshot of an authentic article/post/question......'
    - The claim reformulation thereby tests the model's ability to detect the altered nature of the content
    - An alternative formulation could have been: 
        - 'This image shows an authentic screenshot of .....'
        - Yet, it is not the screenshot that has been altered, but the content. 

3) If the claim reformulation contains a date, the American date format is used: Month Day, Year.

"""

gaza_israel_dictionary_reformulated_claims_ai_and_altered_images = {


2: {
    "Claim": "This is an authentic image of a mother holding the bones of her son in Gaza in March 2025. After more than a year and a half of searching for him under the rubble of their bombed-out house."
},

6: {
    "Claim": "This is an authentic image of Benjamin Netanyahu being hospitalized in Tel Aviv in November 2024 due to a serious health issue."
},


10: {
    "Claim": "This is an authentic image of Israel bombing Beirut's International Airport in October 2024." 
},

19: {
    "Claim": "This is an authentic image of Donald Trump, Benjamin Netanyahu and Elon Musk during Netanyahu's official visit to the U.S. in February 2025."
},

22: {
    "Claim": "This is an authentic image of Mike Tyson posing with the Palestinian flag in the ring before his fight against Jake Paul on November 15, 2024."
},

31: {
    "Claim": "This is an authentic image of Bono and Bob Geldof holding the Israeli flag outside the Israeli embassy in Dublin." 
},


79: {
    "Claim": "This is an authentic image of Israeli Air Force's Ramat David airbase that was attacked by Hezbollah on September 22, 2024."
},

98: {
    "Claim": "This is an authentic image of Dwayne Johnson wearing the Israeli flag in support of Israel." 
},

99: {
    "Claim": "This is an authentic image of an elderly Palestinian woman being attacked by an Israeli army dog in June 2024."
},



26: {
    "Claim": "This image is an authentic screenshot of J.D. Vance's X profile that says 'I stand with Israel'." 
},


30: {
    "Claim": "This image shows a screenshot of an authentic BBC article from February 2025 with the title 'Hamas just wanted to protect the Bibas family but Israel killed them - Hamas said.'"
},

45: {
    "Claim": "This is an authentic image of Greta Thunberg from November 2024 holding a sign reading, 'Fuck Israel'."
},

72: {
    "Claim": "This image shows a screenshot of an authentic New York Times article from December 12, 2024 with the title 'Why the Glorification of Luigi Mangione Is Inherently Anti-Israeli.'"
},

76: {
    "Claim": "This image shows a screenshot of an authentic social media post from Mel Gibson saying that Israel will end soon and therefore destroy everything in its way together with a flag in the color of Israel that shows an hourglass."
},

85: {
    "Claim": "This image shows a screenshot of an authentic question from Israel's Channel 12 public opinion poll: 'Do you agree with the claim that a soldier is allowed to rape a prisoner with his hands tied?'"
},

88: {
    "Claim": "This is an authentic image of Esmail Qaani, the commander of the Iranian Revolutionary Guards, disguised in a hijab in fear of Israeli bombing."
}

}

In [6]:
## check that the dictionary is complete 

print(len(df_gaza_israel[df_gaza_israel["AI_Generated_Image"] == True]))
print(len(df_gaza_israel[df_gaza_israel["Altered_Image"] == True]))

print(len(gaza_israel_dictionary_reformulated_claims_ai_and_altered_images))

9
7
16


### 1.2) Claims with normal images

In [7]:
print(len(df_gaza_israel[df_gaza_israel["Normal_Image"] == True]))

df_gaza_israel[df_gaza_israel["Normal_Image"] == True][:10] # first 10 samples

49


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in Gaza spread online,2025-04-06 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://pbs.twimg.com/media/Gn4wjgBXoAALax1?format=jpg&name=medium,"""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip,"" says an April 6, 2025 X post with more than 30,000 likes.","""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip.""","A reverse image search reveals the photograph was uploaded to Wikimedia Commons, a website that collects free-use images, with a description that says that it was taken in 2007 in Iraq's Babil province (archived here). Wikimedia Commons credits a blog post from the US National Guard as the source of the image. While the site has since been deleted, an archived version available via the Internet Archive says that the men in the image are two sergeants named Robbie Doman and Christopher W. Sullenberger. Doman, standing in the middle of the photograph, received media coverage in 2017 when he appeared on the NBC television show ""American Ninja Warrior"" after being injured in an explosion in 2008 (archived here, and here). Several US media outlets published images of Doman from his service in Iraq, which are similar to those used in the social media posts (archived here).",False,True,False,False,Manual,False
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.39432WD,"Pictures show Gaza in ruins, not Myanmar quake damage",2025-03-29 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://web.archive.org/web/20250402070342im_/https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/487419008_122233817522067190_5197398173696178365_n.jpg?stp=dst-jpg_s600x600_tt6&_nc_cat=108&ccb=1-7&_nc_sid=833d8c&_nc_ohc=Vrh6myPuQ6QQ7kNvgEk5XXK&_nc_oc=AdlUD89_DZpLh7YT1h3kFtbXsa5hjVVf72r_jD7lK9JlOF-ZbIpsMP742_7qAbGCwXM&_nc_zt=23&_nc_ht=scontent-sjc3-1.xx&_nc_gid=fSR43b_Iy30vsY8iyeCKXQ&oh=00_AYHnGD8C7-NR7_yARBjobfW6AGmcjuJ4rs3dDhokDkqldw&oe=67F29A7A,"""A 7.9-magnitude earthquake in Myanmar, devastation everywhere, everything in ruins,"" read part of a simplified Chinese caption of a Facebook post published March 29.","""A 7.9-magnitude earthquake in Myanmar, devastation everywhere, everything in ruins.""","However, a reverse image search for the first photo in the post led to a TikTok video shared February 20 by a user based in Gaza. The clip shows the same destroyed building (archived link). A photo from the Turkish Anadolu news agency shows the same scene. ""A view of the destruction in Jabalia camp after Israeli forces withdrew from the area following the ceasefire in Gaza on February 05, 2025,"" the caption says.",False,True,False,False,Manual,False
3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36XY2FV,Old image misrepresented as 'Israeli settlers' in northern Thailand after Gaza ceasefire,2025-02-16 00:00:00,2025-02-19 00:00:00,Israeli-Palestinian conflict,false,https://pbs.twimg.com/media/Gj5tbXiaQAAoG1z?format=jpg&name=small,"""Israeli media reports there are Jews who have stopped fighting the wars in Gaza, Palestine, and Lebanon. They moved to become 'settlers' in Pai District, Mae Hong Son Province, Thailand,"" reads part of an X post published February 16, 2025.","""Israeli media reports there are Jews who have stopped fighting the wars in Gaza, Palestine, and Lebanon. They moved to become 'settlers' in Pai District, Mae Hong Son Province, Thailand.""","A reverse image search via Google traced the viral photo to a report published October 2, 2024 by the Mae Hong Son 

Methodology: 
- Access fact-checking article again, if the context (date, location) or the alleged or real event can not be inferred from one of the columns of the data frame (e.g., Headline, Claim_Date, Original_Claim_Website, Original_Claim_Only, Label_Explanation/Context)


- claim with "id" 0 -> USE MONTH OF CLAIM DATE 
    - Claim date: 06.04.2025
    - Use the month as a date
    - Claim could be reformulated in:
        "This image shows General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, which has died in the Gaza Strip in April 2025."

- claim with "id" 1 -> USE CONTEXT DATE (1 day before CLAIM DATE)
    - Context: Myanmar earthquake date: 28.03.2025 (https://en.wikipedia.org/wiki/2025_Myanmar_earthquake)
    - Claim date: 29.03.2025
    - Could be reformulated in:
        "This image shows the ruins after a 7.9-magnited earthquake in Myanmar on March 28, 2025."
        

- claim with "id" 3 -> USE MONTH OF CLAIM DATE
    - Context: Original image is from October 2024
    - Claim date: 16.02.2025
    - Use monthly date of claim
    - Claim could be refomulated in:
        "This image shows Israelis who have stopped fighting the wars in Gaza and Lebanon and moved to become 'settlers' in Pai District, Mae Hong Son Province, Thailand, in February 2025."



- claim with "id" 4: -> USE CLAIM DATE (EXACT DAY)
    - Context: US strikes Huthi targets in Yemen, hours after rebels hit Israel, 22.12.2024 (https://jordantimes.com/news/region/us-strikes-huthi-targets-yemen-hours-after-rebels-hit-israel)
    - Claim date: 22.12.2024
    - Claim could be reformulated in:
        "This image shows the U.S. bombarding Yemen on December 22, 2024 in reprisal after Yemen launched an attack on Israel's capital."


- claim with "id" 5: ->  USE CONTEXT DATE (1 day before CLAIM DATE) 
    - Context: Israel-Hezbollah Clashes in Southern Lebanon 23.11.2024 (https://www.barrons.com/news/fierce-israel-hezbollah-clashes-at-flashpoint-town-lebanon-state-media-8bbe7a90)
    - Claim date: 24.11.2024
    - Could be reformulated in: 
        "This image shows an Israeli helicopter shot down by Hezbollah in Southern Lebanon on November 23, 2024."


- clam with "id" 8: -> USE CLAIM DATE (EXACT DAY)
    - Context: After Israel launched airstrikes against military targets in Iran’s capital Tehran in October 2024 (copied from AFP article)
    - Claim date: 26.11.2024
    - Claim could be refomulated in:
        "This image shows Israel launching several waves of revengeful attacks on Iranian military targets on October 26, 2024."



- claim with "id" 9: -> USE CLAIM DATE (EXACT DAY)
    - Refers to the same context/Israeli attack on Iran as previous claim (id:8) -> Use the same date
    - Claim date: 26.11.2024
    - Coud be reformulated in:
        "This image shows an Israeli missile attack on Iran in the early hours of October 26, 2024."


- claim with "id" 13 -> USE CLAIM DATE (EXACT DAY)
    - Context (from fact-checking article): "A facility housing US personnel in the Iraqi capital of Baghdad came under fire September 30, 2024 as the conflict between Israel and the Iran-backed militant group Hezbollah escalated."
    - Claim date: 30.09.2024
    - Since the claim date is the same as the context date, it will be used 
    - Could be reformulated in:
        "This image shows a US factility come under attack in Baghdad on September 30, 2024 as the conflict between Israel and the Iran-backed militant group Hezbollah escalated."


- claim with "id" 14 -> USE MONTH DATE OF CLAIM
    - Context (from fact-checking article): "A photo of an exploded toilet in a Hong Kong public restroom ricocheted across social media following deadly attacks involving hundreds of hand-held devices used by Hezbollah which detonated across Lebanon."
    - Claim date: 19.09.2024
    - Date of pager explosions in Lebanon: 17 + 18.09.2024 (https://en.wikipedia.org/wiki/2024_Lebanon_electronic_device_attacks)
    - Since pager explosions took place on two days, monthly date will be used
    - Claim could be reformulated in:
        "This image shows the aftermath of an explosion of a Hezbollah pager in Lebanon in September 2024."


- claim with "id" 15 -> USE MONTH DATE OF CLAIM
    - Context (from fact-checking article): "Traffic signs in Montreal displayed pro-Palestinian phrases in July 2024, but social media posts claiming Mayor Valérie Plante authorized them are baseless."
    - The image used shows a traffic sign saying "Free Palestine. Escalate now"
    - Claim date: 24.07.2024
    - Since the traffic signs could also have been shown on another day in July 2024, the month will be used as a date
    - Claim could be reformulated in:
        "This image shows a traffic sign in Montreal in July 2024 which displays a 'Free Palestine. Escalate now' message and was authorized by Montreal's Mayor Valérie Plante."


In [8]:
### Using a dictionary to store the manually reformulated claims for normal images to later add to the dataframe
#### using the indices of the claim for mapping

""" 

General rules/format for manual reformulation of claims normal images:

1) 

"""



gaza_israel_dictionary_reformulated_claims_normal_images_part_1 = {

    0: {
        "Claim": "This image shows General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, which has died in the Gaza Strip in April 2025."
    },

    1: {
        "Claim": "This image shows the ruins after a 7.9-magnitude earthquake in Myanmar on March 28, 2025."
    },

    3: {
        "Claim": "This image shows Israelis who have stopped fighting the wars in Gaza and Lebanon and moved to become 'settlers' in Pai District, Mae Hong Son Province, Thailand, in February 2025."
    },

    4: {
        "Claim": "This image shows the U.S. bombarding Yemen on December 22, 2024 in reprisal after Yemen launched an attack on Israel's capital."
    },

    5: {
        "Claim": "This image shows an Israeli helicopter shot down by Hezbollah amid clashes in Southern Lebanon on November 23, 2024."
    },

    8: {
        "Claim": "This image shows Israel launching several waves of revengeful attacks on Iranian military targets on October 26, 2024."
    },

    9: {
        "Claim": "This image shows an Israeli missile attack on Iran in the early hours of October 26, 2024."
    },

    13: {
        "Claim": "This image shows a US factility come under attack in Baghdad on September 30, 2024 as the conflict between Israel and the Iran-backed militant group Hezbollah escalated."
    },

    14: {
        "Claim": "This image shows the aftermath of an explosion of a Hezbollah pager in Lebanon in September 2024."
    },

    15: {
        "Claim": "This image shows a traffic sign in Montreal in July 2024 which displays a 'Free Palestine. Escalate now' message and was authorized by Montreal's Mayor Valérie Plante."
    }


}

## verify the length
print(len(gaza_israel_dictionary_reformulated_claims_normal_images_part_1))


10


In [9]:
df_gaza_israel[df_gaza_israel["Normal_Image"] == True][10:20]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
16,16,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.368W46C,Israeli swimmers' tribute to Gaza hostages predates Olympics\r\n\r\n,2024-07-27 00:00:00,2024-08-05 00:00:00,Israeli-Palestinian conflict,misleading,"https://ynet-pic1.yit.co.il/cdn-cgi/image/f=auto,w=740,q=75/picserver5/crop_images/2023/11/19/Sk4M5SDET/Sk4M5SDET_18_11_1483_1933_0_x-large.jpg","""The Israeli Olympic team was not allowed to wear their yellow 'Bring Them Home' pins at the Olympic games so they decided to spell it out for the entire world to see it!"" says Rob Shuter, a UK gossip columnist, in a July 27, 2024 Facebook post with thousands of interactions.","""The Israeli Olympic team was not allowed to wear their yellow 'Bring Them Home' pins at the Olympic games so they decided to spell it out for the entire world to see it!""","While the picture shared online is real, it predates the Paris Olympics by eight months. A reverse image search reveals Israel's artistic swimming team posted the photo November 19, 2023 on Instagram (archived here). The caption says the picture was taken at the Wingate Institute, Israel's national sports training center outside Tel Aviv, and that it features 10 artistic swimmers. Among the athletes are Shelly Bobritsky and Ariel Nassee, who qualified for the 2024 Paris Olympics as part of the duet category (archived here and here). Israeli news outlets including the Arabic-language service of i24News reported on the team's photo in November 2023 (archived here and here). Israeli drone photographer Adam Spiegel posted the picture on Instagram the same day as the Israeli Artistic Swimming Team (archived here). Spiegel confirmed in a July 30 email that he took the original image, compiling several aerial shots of the swimmers into a composite collage. He said he was ""disappointed"" that social media users used his photo without credit and ""invented a fake news story.""",False,True,False,False,Manual,Misleading
18,18,Reuters,https://www.reuters.com/fact-check/photo-shows-trumps-hollywood-star-damaged-2018-not-gadots-2025-2025-03-28/,"Fact Check: Photo shows Trump’s Hollywood star damaged in 2018, not Gadot’s in 2025",2025-03-20 00:00:00,2025-03-28 00:00:00,"""Gaza Fact Check""",miscaptioned,https://www.vice.com/wp-content/uploads/sites/2/2018/08/1534367591174-pickax.jpeg,"A post on Threads, opens new tab on March 20 shared a photo of a destroyed Hollywood star and said: “Gal Gadot’s star on the Hollywood Walk of Fame was destroyed just hours after being placed!”",“Gal Gadot’s star on the Hollywood Walk of Fame was destroyed just hours after being placed!”,"The image, however, was featured in multiple news reports about Trump’s star being vandalized in July 2018.",False,True,False,False,Manual,Misleading
24,24,Reuters,https://www.reuters.com/fact-check/unrwa-added-black-ribbon-its-logo-2023-not-after-sinwars-death-2024-10-24/,"Fact Check: UNRWA added black ribbon to its logo in 2023, not after Sinwar’s death",2024-10-18 00:00:00,2024-10-24 00:00:00,"""Gaza Fact Check""",misleading,https://pbs.twimg.com/media/GaIg1KHXUAA4Pl9?format=jpg&name=medium,"Social media accounts shared, a photo of the United Nations Relief and Works Agency for Palestinian Refugees (UNRWA) logo with the black ribbon, on Oct. 18, 2024. One user said, “UNRWA has placed a black band over its profile photo in mourning for the death of the ‘Butcher of Khan Younis’ Yahya Sinwar. They’re not even trying to hide their affiliation with Hamas anymore.”","""UNRWA has placed a black band over its profile photo in mourning for the death of the 'Butcher of Khan Younis' Yahya Sinwar.""",Misleading. The black ribbon on the UNRWA logo predates Sinwar's death. It was added to the profile picture in October 2023 in solidarit

- claim with "id" 16 -> Use 2024 Olympic games as date reference
    - Claim date: 27.07.2024
    - Use 2024 Olympic games as date reference
    - Claim could be reformulated in:
        "This image shows the Israeli Olympic team spelling out 'Bring Them Home' at the 2024 Olympic games because they were not allowed to wear their yellow 'Bring Them Home' pins."



- claim with "id" 18 -> USE MONTH OF CLAIM DATE
    - Claim date: 20.03.2025
    - Use month of the claim date. The actual incident occured in 2018 and was Trump's star. Thus, as a fictional reference not the exact day of the claim will be used, but the month of the claim.
    - Could be reformulated in:
        "This image shows Gal Gadot’s star on the Hollywood Walk of Fame being destroyed in March 2025 just hours after being placed."
             

- claim with "id" 24 -> USE MONTH OF CLAIM DATE
    - Context (from the fact-checking article): "Sinwar, mastermind of the Oct. 7, 2023, attack that triggered the Gaza war, was killed by Israeli forces in the Palestinian enclave on Oct. 16, 2024."
    - Claim date: 18.10.2024
    - Use monthly date of claim
    - Claim could be refomulated in:
        "This image shows a screenshot of the X profile of UNRWA which has placed a black band over its profile photo in mourning for the death of Yahya Sinwar, Hamas leader in the Gaza Strip, in October 2024."


- claim with "id" 27: -> Use 2024 French parliamentary elections as date reference
    - Context: Le Pen's party won the first round of parliamentary elections one June 30, 2024 (https://www.france24.com/en/france/20240630-le-pen-s-far-right-party-wins-first-round-of-french-elections-as-macron-s-gamble-backfires) 
    - Claim date: 01.07.2024
    - Use the first round of French parliamentary elections in 2024 as date reference
    - Claim could be reformulated in:
        "This image shows the 'French' Left who is rioting against Le Pen's victory in the first round of the French parliamentary elections in 2024."


- claim with "id" 34: ->  USE CLAIM DATE (EXACT DAY)
    - Context (from fact-checking article): "Israeli airstrikes on Tehran on Oct. 26, which Israeli Prime Minister Benjamin Netanyahu said had “hit hard”, killed four soldiers and caused damage."
    - Claim date: 26.10.2024
    - The claim has been mentioned before (id 8, 9), but it relates to a different (also out-of-context) image this time
    - Could be reformulated in: 
        "This image shows Israel bombing Tehran on October 26, 2024."



- clam with "id" 35: -> USE CLAIM DATE (EXACT DAY)
    - Context (from fact-checking article): "Amid escalating Israeli attacks, Nasrallah was killed in a Sept. 28, 2024, Israeli airstrike in Beirut."
    - Claim date: 28.09.2024
    - Context and claim date are the same. Thus, claim date will be used.
    - Claim could be refomulated in:
        "This image shows the remains of Nasrallah's ring after being killed in an Israeli airstrike in Beirut on September 28, 2024."



- claim with "id" 37: -> USE CLAIM DATE (EXACT DAY)
    - Context (from fact-checking article): "Israel accused Iran-backed Hezbollah of carrying out the July 27 airstrike on a football field which killed 12 children and teenagers in Majdal Shams in the Israel-occupied Golan Heights."
    - Claim date: 27.07.2024
    - Context and claim date are the same. Thus, claim date will be used.
    - Claim could be reformulated in:
        "This image shows the youngest victim of the Hezbollah rocket that murdered 12 children playing soccer in Israel on July 27, 2024."




- claim with "id" 38: -> USE CLAIM DATE (EXACT DAY)
    - Context: Hamas leader Ismail Haniyeh was assassinated in the Iranian capital Tehran early on Wednesday morning, an attack that drew threats of revenge on Israel and fuelled further concern that the conflict in Gaza was turning into a wider Middle East war. (https://www.reuters.com/world/middle-east/hamas-chief-ismail-haniyeh-killed-iran-hamas-says-statement-2024-07-31/)
    - Claim date: 31.07.2024 
    - Context and claim date are the same. Thus, claim date will be used.
    - Coud be reformulated in:
        "This image shows the assassination of Hamas leader Ismail Haniyeh on July 31st, 2024. This is how a precision strike looks like: a single missile through the window, and only Haniyeh and his bodyguard are dead. Nobody else in the building is even injured."



- claim with "id" 39 -> Use 2024 Olympic games as date reference
    - Context: Paris Olympics 2024
    - Claim date: 20.07.2024
    - Use 2024 Olympic games as date reference
    - Could be reformulated in:
        "This image shows an athlete of China which has chosen a design of the Palestinian flag for the Paris 2024 Olympics."




- claim with "id" 44 -> USE MONTH OF CLAIM DATE (end of month)
    - Context (from fact-checking article): The photograph was taken Nov. 29, 2024, when numerous photos of Biden were taken during the Thanksgiving holiday weekend.
    - Claim date: 30.11.2024
    - The image went viral in late November 2024
    - Use late November 2024 as date
    - Claim could be reformulated in:
        "This image authentically shows U.S. President Joe Boden in late November 2024 exiting a Nantucket bookstore holding a copy of writer Rashid Khalidi's book, 'The Hundred Years' War on Palestine: A History of Settler Colonialism and Resistance, 1917–2017.'"


In [10]:
gaza_israel_dictionary_reformulated_claims_normal_images_part_2 = {

    16: {
        "Claim": "This image shows the Israeli Olympic team spelling out 'Bring Them Home' at the 2024 Olympic games because they were not allowed to wear their yellow 'Bring Them Home' pins."
    },

    18: {
        "Claim": "This image shows Gal Gadot’s star on the Hollywood Walk of Fame being destroyed in March 2025 just hours after being placed."
    },

    24: {
        "Claim": "This image shows a screenshot of the X profile of UNRWA which has placed a black band over its profile photo in mourning for the death of Yahya Sinwar, Hamas leader in the Gaza Strip, in October 2024."
    },

    27: {
        "Claim": "This image shows the 'French' Left who is rioting against Le Pen's victory in the first round of the French parliamentary elections in 2024."
    },

    34: {
        "Claim": "This image shows Israel bombing Tehran on October 26, 2024."
    },

    35: {
        "Claim": "This image shows the remains of Nasrallah's ring after being killed in an Israeli airstrike in Beirut on September 28, 2024."
    },

    37: {
        "Claim": "This image shows the youngest victim of the Hezbollah rocket that murdered 12 children playing soccer in Israel on July 27, 2024."
    },

    38: {
        "Claim": "This image shows the assassination of Hamas leader Ismail Haniyeh on July 31st, 2024. This is how a precision strike looks like: a single missile through the window, and only Haniyeh and his bodyguard are dead. Nobody else in the building is even injured."
    },

    39: {
        "Claim": "This image shows an athlete of China which has chosen a design of the Palestinian flag for the Paris 2024 Olympics."
    },

    44: {
        "Claim": "This image authentically shows U.S. President Joe Boden in late November 2024 exiting a Nantucket bookstore holding a copy of writer Rashid Khalidi's book, 'The Hundred Years' War on Palestine: A History of Settler Colonialism and Resistance, 1917–2017.'"
    }

}

## verify the length
print(len(gaza_israel_dictionary_reformulated_claims_normal_images_part_2))

10


In [11]:
df_gaza_israel[df_gaza_israel["Normal_Image"] == True][20:30]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
47,47,Snopes,https://www.snopes.com/fact-check/jetblue-map-israel-palestine/,JetBlue's In-Flight Map Showed 'Palestinian Territories' Label Over Israel?,2024-09-05 00:00:00,2024-09-13 00:00:00,"""Gaza""",true,https://mediaproxy.snopes.com/width/1200/https://media.snopes.com/2024/09/jetblue_inflight_map_1.png,"""On JetBlue Airlines' in-flight mapping system, Israel's borders were inaccurate and the area was shown under the label 'Palestinian Territories.'”","""On JetBlue Airlines' in-flight mapping system, Israel's borders were inaccurate and the area was shown under the label 'Palestinian Territories.'”","The reports of the mislabeled in-flight map were true, and as a result of the viral nature of the claim, JetBlue issued a public apology.",False,True,False,False,Manual,True
56,56,Politifact,https://www.politifact.com/factchecks/2024/sep/20/tweets/no-this-image-doesnt-show-an-iphone-that-exploded/,“iPhones exploding in Lebanon blowing up random people.”,2024-09-18 00:00:00,2024-09-20 00:00:00,"""Israel""",false,https://pbs.twimg.com/media/GXxe63hXsAAlRh4?format=jpg&name=large,"""iPhones exploding in Lebanon blowing up random people,""","""iPhones exploding in Lebanon blowing up random people.""","The image of an exploded iPhone being shared as if it was from recent device attacks in Lebanon was published in a 2021 article about an exploding iPhone in a suburb of Cairo, Egypt. Credible news reports of the attacks in Lebanon did not say that iPhones were devices targeted in the attacks.",False,True,False,False,Manual,False
57,57,misbar.com,https://www.misbar.com/en/factcheck/2025/04/13/contrary-trolls-baby-sham-lost-her-arm-and-died-israeli-airstrike,"Contrary to Trolls, Baby Sham Lost Her Arm and Died in Israeli Airstrike",2025-04-13 00:00:00+00:00,2025-04-13 00:00:00+00:00,Gaza,fake,https://pbs.twimg.com/media/GoQLcf6WUAAkeXV?format=jpg&name=medium,NaN,"Baby Sham did not lose her arm in an Israeli airstrike on the Shejaeya neighborhood, east of Gaza City.","Misbar’s investigative team spoke with Nahed Hajjaj, the original photographer of the widely shared photo showing Sham after her amputation. Hajjaj told Misbar he took the photo two days earlier at Al-Ahli Arab Baptist Hospital, where Sham was treated after an Israeli attack targeted her family’s house in the Shuja’iyya neighborhood. He confirmed that Sham lost her right arm and died the following day due to the severity of her injuries. Nahed also posted footage of baby Sham on his Instagram account two days ago, providing details about the Israeli airstrike that targeted her family.",False,True,False,False,API,False
58,58,misbar.com,https://www.misbar.com/en/factcheck/2024/12/30/this-footage-does-not-show-and-aerial-footage-of-gaza-but-zamalka-in-syria,This Footage Does Not Show Aerial Footage of Gaza But Zamalka ...,2024-12-30 00:00:00+00:00,2024-12-30 00:00:00+00:00,Gaza,misleading,https://assets.misbar.com/styles/large_870x490/public/218e6fd5e84f5220e65b83e8bc2470036f4a9b9b.jpg.webp?itok=jePQTGS7,NaN,"This is Gaza, completely destroyed. It looks like hell on earth.","Aerial Footage Shows the Destruction in Zamalka City, in Syria. The video includes an Arabic phrase, which upon translation, indicates that the footage does not show Gaza. After selecting the text, using the translation option in Google reverse image search, Google Translate showed that the Arabic phrase means, “Aerial shots of the town of Zamalka in the Damascus countryside,” not Gaza as claimed. The video ends with the logo of the Syria TV channel, casting further doubt on the claim's validity. Upon searching for the keywords “Zamalka” and “Syria TV” in Arabic, Misbar’s team found the same video published on the Syria TV YouTube channel on Dec. 16, 2024, t

- claim with "id" 47 -> USE MONTH OF CLAIM DATE
    - Claim date: 05.09.2024    
    - Social media post were shared in early September 2024 (not all on the 5th of September)
    - Thus, the month of the claim will be used
    - Claim could be reformulated in:
        "This image shows Jet Blue Airlines' in-flight map where Israel's borders were inaccurate and the area was shown under the label 'Palestinian Territories'. The indicent occured in September 2024."





- claim with "id" 56 -> USE MONTH OF CLAIM DATE
    - Claim date: 18.09.2024
    - Context: Pager attacks of Hezbollah in Lebanon on September, 18 & 19 (see previous claims)
    - To be consistent with the other claims about the pager explosion, here the month of the claim will also be used as the date.
    - Use month of the claim date. The actual incident occured in 2018 and was Trump's star. Thus, as a fictional reference not the exact day of the claim will be used, but the month of the claim.
    - Could be reformulated in:
        "This image shows an Iphone that exploded during the pager attacks on Hezbollah devices in Lebanon in September 2024, blowing up random people.”
             




- claim with "id" 57 -> USE MONTH OF CLAIM DATE
    - Context: The Israeli airstrike on the Shejaeya neighborhood, east of Gaza city, took place on April 10, 2025. (https://deepnewz.com/middle-east/palestinian-infant-sham-imad-muhareb-dies-after-losing-arm-israeli-airstrike-6-3f8d3b14)
    - Claim date: 13.04.2025    
    - Use monthly date of claim, since the claim date and the date of the context/event are different, but both from April 2025
    - Claim could be refomulated in:
        "This image shows Palestinian baby Sham Imad Muhareb which did not lose her arm in an Israeli airstrike on the Shejaeya neighborhood, east of Gaza City, in April 2025."





- claim with "id" 58: -> USE MONTH OF CLAIM DATE
    - Context: Image from Gaza genocide
    - Claim date: 30.12.2024     
    - Since the original image is from Syria and the exact date of the claim is not relevant, the month of the claim will be used
    - Claim could be reformulated in:
        "This image shows Gaza, completely destroyed, in December 2024."




- claim with "id" 59: -> USE MONTH OF CLAIM DATE
    - Claim date: 25.10.2024
    - Since original image is from different context, the month of the claim will be used
    - Could be reformulated in: 
        "This image shows an Israeli tank unit near Jabalia camp in northern Gaza that was successfully ambushed by Hamas fighters, destroying multiple vehicles and injuring or killing their crews in October 2024."





- clam with "id" 60: -> USE CLAIM DATE (EXACT DAY)
    - Context: Israeli airstrikes on Beirut on October 21, 2024 (https://www.bbc.com/news/live/cdje1vm2x9xt)
    - Claim date: 21.10.2024
    - Context and claim date are the same. Thus, claim date will be used.
    - Claim could be refomulated in:
        "This image shows an Israeli attack on Lebanon on October 21, 2024."



- claim with "id" 61: -> USE MONTH OF CLAIM DATE
    - Claim date: 15.07.2024
    - Since original image is from different context and the exact claim date is not relevant for the claim, the month of the claim will be used
    - Original claim: "Mercava Pride Zionist Industry Laundry clothesline in Gaza"
    - Since the original claim is not really informative and the fact-checking article used a claim that better describes the image ("Social media users widely circulated an image claiming to show a Palestinian woman using an Israeli tank to dry clothes in Gaza."), the claim will be reformulated more substantially.
    - Claim could be reformulated in:
        "This image shows a Palestinian woman using an Israeli tank to dry clothes in Gaza in July 2024."




- claim with "id" 62: -> USE CLAIM DATE (EXACT DAY)
    - Claim date: 15.04.2025
    - Claim was shared in different social media posts on different dates in April 2025.
    - Thus, the month of the claim will be used.
    - Coud be reformulated in:
        "This image shows thousands gathering at Sohrawardi Park in Kolkata, India, in April 2025 to show solidarity with the Palestinian people amid the ongoing Israeli assault on Gaza."




- claim with "id" 63 -> USE MONTH OF CLAIM DATE
    - Context (from fact-checking article): Social media users widely shared a photo purporting to show two Palestinian children found under the rubble of their house in Gaza two months after an Israeli airstrike destroyed it.
    - Claim date: 14.04.2025
    - Since original image is from different context and the exact claim date is not relevant for the claim, the month of the claim will be used
    - Original claim: "Two children were found two months later under the rubble of their home. No God bless Israel and its supporters"
    - Since the context from the fact-checking article (falsely) suggests an Israeli airstrike as the cause, this will be added as context to the claim. The second sentence will be removed since it is not relevant for the verificaton of the claim.
    - Could be reformulated in:
        "This image shows two children in Gaza in April 2025 found dead under the rubble of their home two months after an Israeli airstrike destroyed it."



- claim with "id" 65 -> USE MONTH OF CLAIM DATE
    - Claim date: 19.03.2025
    - Since original image is from different context and exact date of claim is not relevant, the month of the claim will be used
    - 
    - Claim could be reformulated in:
        "This image shows heavy Israeli bombardment on Deir al-Balah in the southern Gaza Strip in March 2025 which killed at least four Palestinians."


In [12]:
gaza_israel_dictionary_reformulated_claims_normal_images_part_3 = {

    47: {
        "Claim": "This image shows Jet Blue Airlines' in-flight map where Israel's borders were inaccurate and the area was shown under the label 'Palestinian Territories'. The indicent occured in September 2024."
    },

    56: {
        "Claim": "This image shows an Iphone that exploded during the pager attacks on Hezbollah devices in Lebanon in September 2024, blowing up random people."
    },

    57: {
        "Claim": "This image shows Palestinian baby Sham Imad Muhareb which did not lose her arm in an Israeli airstrike on the Shejaeya neighborhood, east of Gaza City, in April 2025."
    },

    58: {
        "Claim": "This image shows Gaza, completely destroyed, in December 2024."
    },

    59: {
        "Claim": "This image shows an Israeli tank unit near Jabalia camp in northern Gaza that was successfully ambushed by Hamas fighters, destroying multiple vehicles and injuring or killing their crews in October 2024."
    },

    60: {
        "Claim": "This image shows an Israeli attack on Lebanon on October 21, 2024."
    },

    61: {
        "Claim": "This image shows a Palestinian woman using an Israeli tank to dry clothes in Gaza in July 2024."
    },

    62: {
        "Claim": "This image shows thousands gathering at Sohrawardi Park in Kolkata, India, in April 2025 to show solidarity with the Palestinian people amid the ongoing Israeli assault on Gaza."
    },

    63: {
        "Claim": "This image shows two children in Gaza in April 2025 found dead under the rubble of their home two months after an Israeli airstrike destroyed it."
    },

    65: {
        "Claim": "This image shows heavy Israeli bombardment on Deir al-Balah in the southern Gaza Strip in March 2025 which killed at least four Palestinians."
    }

}

## verify the length
print(len(gaza_israel_dictionary_reformulated_claims_normal_images_part_3))

10


In [13]:
df_gaza_israel[df_gaza_israel["Normal_Image"] == True][39:40]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
83,83,misbar.com,https://www.misbar.com/en/factcheck/2024/09/19/photo-of-barcelona-president-with-israeli-representatives-predates-gaza-war,Photo of Barcelona President with Israeli Representatives Predates ...,2024-09-19 00:00:00+00:00,2024-09-19 00:00:00+00:00,Israel,misleading,https://pbs.twimg.com/media/FpHcIuMWAAICbmv?format=jpg&name=medium,NaN,FC Barcelona stands with Jewish terrorists and murder of civilians.,"Outdated Photo of Barcelona President with Israel Embassy Representatives. Contrary to the claim circulating on social media, the photo is not recent. Misbar's investigative team traced the original image back to February 16, 2023, following FC Barcelona's announcement to sever ties with Israel. The original photo was posted by i24 News on X that same day, with the caption, “Barcelona football club invites Israeli embassy staff in Spain to match against Manchester United, in show of solidarity after Barcelona mayor's decision to cut ties with Tel Aviv over 'Israeli apartheid'”",False,True,False,False,API,Misleading


- claim with "id" 66 -> USE MONTH OF CLAIM DATE
    - Context: A reverse image search led to a YouTube video published by AFP News Agency in September 2024. The video, which included frames similar to the viral image, showed Zelenskyy visiting an ammunition factory in Scranton, Pennsylvania.
    - Original image is from September 2024 in USA. 
    - Claim date: 05.03.2025    
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Claim could be reformulated in:
        "This image shows Ukrainian President Volodymyr Zelenskyy in March 2025 signing Israeli bombs intended for use against Palestinians in Gaza."




- claim with "id" 68 -> USE MONTH OF CLAIM DATE
    - Context: IDF striking southern Syria in February and March 2025 (https://www.timesofisrael.com/idf-strikes-military-targets-across-syria-amid-warnings-to-damascus-to-protect-druze/,https://www.jpost.com/breaking-news/article-843758)
    - Claim date: 01.03.2025
    - Since the social meda posts shown in the fact-checking article are from February, this month will be used. 
    - Could be reformulated in:
        "This image shows Israel bombing the surroundings of Southern Damascus in February 2025, destroying its last remaining army depots.”
             




- claim with "id" 69 -> USE MONTH OF CLAIM DATE
    - Context: The same context as in the previsous claim: IDF striking southern Syria in February and March 2025 (https://www.timesofisrael.com/idf-strikes-military-targets-across-syria-amid-warnings-to-damascus-to-protect-druze/,https://www.jpost.com/breaking-news/article-843758)
    - Claim date: 27.02.2025    
    - Also use February month claim (same context).
    - The second sentence of the claim will not be used, since it does not relate to the shown image ("While in the Muhajireen area in Homs, Al-Jolani elements arrest dozens of young men.")
    - Claim could be refomulated in:
        "This image shows Israel military forces entering the town of Tasil in the countryside of Daraa on foot in February 2025."





- claim with "id" 71: -> USE MONTH OF CLAIM DATE
    - Claim date: 27.01.2025     
    - Since the original image is from Syria and the exact date of the claim is not relevant, the month of the claim will be used
    - Claim could be reformulated in:
        "This image shows a vegetable stand in Gaza in January 2025 with a Merkava tank, the pride of the Israeli army, as its decor."




- claim with "id" 73: -> USE MONTH OF CLAIM DATE
    - Context: IDF invading/striking southern Syria in December 2024 (https://en.wikipedia.org/wiki/Israeli_invasion_of_Syria_(2024–present)). The same context aslo claim with "id" 68 & 69, but already from December 2024.
    - Claim date: 11.12.2024
    - Since the original image is from different context and the exact date of the claim is not relevant, but the month, the month of the claim will be used
    - Could be reformulated in: 
        "This image shows the Israeli army reaching Damascus in Syria in December 2024."





- clam with "id" 75: -> USE CONTEXT DATE (EXACT DAY)
    - Context: Iran striking Israel with multiple rockets on October, 1 2024 (https://en.wikipedia.org/wiki/October_2024_Iranian_strikes_against_Israel)
    - Claim date: 15.10.2024
    - The image is from a different context. But since the Iranian attack occured on a specific day (October 1), this day will be used as a month.
    - Claim could be refomulated in:
        "This image shows tents engulfed in flames after the Iranian strikes against Israel on October 1, 2024."






- claim with "id" 80: -> USE MONTH OF CLAIM DATE
    - Context: The original image is from a protest in Tel Aviv in September 2024.
    - Claim date: 17.09.2024
    - Since the context event and the claim date are in the same month, the month of the claim will be used.
    - Original claim: "BREAKING: 22 dead and 74 injured among Israels Unit 8200 in Hezbollahs 25 August retaliation, European security sources tell Al-Mayadeen. Iron dome failed by Hezbollah missiles."
    - Since the original claim could be more explicitly related to the image, the claim will be reformulated substantially.
    - Claim could be reformulated in:
        "This image shows coffins of dead Israeli soldiers in September, 2024."






- claim with "id" 81: -> DON'T USE DATE
    - Claim date: 11.09.2024
    - Since the 9/11 memorial in Israel has been existing way before September, 2024, it is not necessary to use a date.
    - Coud be reformulated in:
        "This image shows Israel's 9/11 memorial which is the only one outside the US."






- claim with "id" 82 -> USE CONTEXT DATE (EXACT DAY)
    - Context: Hezbollah attack on Israel on August 25, 2024 (https://www.jpost.com/israel-hamas-war/article-816204)
    - Claim date: 26.08.2024
    - Since the claim and image relate to the specific attack on August 25, this date will be used.
    - Could be reformulated in:
        "This image shows Israeli Finance Minister Smotrich, who previously threatened to invade Lebanon, taking shelter in an Israeli bomb shelter during Hezbollah's attack on Israel on August 25, 2024."





- claim with "id" 83 -> USE MONTH OF CLAIM DATE
    - Claim date: 19.09.2024
    - Since original image is from different context and exact date of claim is not relevant, the month of the claim will be used. 
    - Original claim: "FC Barcelona stands with Jewish terrorists and murder of civilians."
    - Since the original claim does not describe the image very well and contains strong normative judgements, it will be reformulated substantially. Since the context of the Gaza war is important to the claim, it will be added beyond the claim month.
    - Claim could be reformulated in:
        "This image shows the President of FC Barcelona with Israel Embassy Representatives in September 2024 during the ongoing Gaza war."



In [14]:
gaza_israel_dictionary_reformulated_claims_normal_images_part_4 = {

    66: {
        "Claim": "This image shows Ukrainian President Volodymyr Zelenskyy in March 2025 signing Israeli bombs intended for use against Palestinians in Gaza."
    },

    68: {
        "Claim": "This image shows Israel bombing the surroundings of Southern Damascus in February 2025, destroying its last remaining army depots."
    },

    69: {
        "Claim": "This image shows Israel military forces entering the town of Tasil in the countryside of Daraa on foot in February 2025."
    },

    71: {
        "Claim": "This image shows a vegetable stand in Gaza in January 2025 with a Merkava tank, the pride of the Israeli army, as its decor."
    },

    73: {
        "Claim": "This image shows the Israeli army reaching Damascus in Syria in December 2024."
    },

    75: {
        "Claim": "This image shows tents engulfed in flames after the Iranian strikes against Israel on October 1, 2024."
    },

    80: {
        "Claim": "This image shows coffins of dead Israeli soldiers in September, 2024."
    },

    81: {
        "Claim": "This image shows Israel's 9/11 memorial which is the only one outside the US."
    },

    82: {
        "Claim": "This image shows Israeli Finance Minister Smotrich, who previously threatened to invade Lebanon, taking shelter in an Israeli bomb shelter during Hezbollah's attack on Israel on August 25, 2024."
    },

    83: {
        "Claim": "This image shows the President of FC Barcelona with Israel Embassy Representatives in September 2024 during the ongoing Gaza war."
    }

}

## verify the length
print(len(gaza_israel_dictionary_reformulated_claims_normal_images_part_4))

10


In [15]:
len(df_gaza_israel[df_gaza_israel["Normal_Image"] == True])

49

In [16]:
print(len(df_gaza_israel[df_gaza_israel["Normal_Image"] == True][40:50]))
df_gaza_israel[df_gaza_israel["Normal_Image"] == True][40:42]

9


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
84,84,logicallyfacts.com,https://www.logicallyfacts.com/en/fact-check/false-no-this-photo-of-netanyahu-s-home-was-not-taken-by-a-hezbollah-drone,"No, this photo of Netanyahu's home was not taken by a Hezbollah drone",2024-08-18 00:00:00+00:00,2024-08-21 00:00:00+00:00,Israel,false,https://static-cdn.toi-media.com/www/uploads/2019/11/imgid15736_A-640x400.jpeg,NaN,Social media users sharing a photo of Israeli PM Benjamin Netanyahu's house claiming that the photo was taken by Hezbollah,"On conducting a reverse image search, we found that several Israeli news outlets had published similar images in reports from 2024. A news report by The Times of Israel published on January 30, 2024, headlined, ""State approves repairs to pool in Netanyahu's Caesarea home – report,"" carried the same photo of the property. The image captioned, ""Prime Minister Benjamin Netanyahu's private residence in Caesarea,"" was credited to Israeli public broadcaster KAN. The Times of Israel also used the photo for another article on Netanyahu's house published on July 19, 2024. Another report by Radio J, a French radio station that broadcasts information from the Jewish community, carried the same image in an article published on January 30, 2024, crediting the photo to KAN. Further, upon relevant Google searches in Hebrew, we found a video uploaded by KAN (archived here) on its YouTube channel on July 1, 2019, with the same visuals at the 0:07 timestamp. The video was originally titled in Hebrew and, when translated using Google, reads, ""The villa in Caesarea: The Netanyahu family insists that the state finance the sealing works.""",False,True,False,False,API,False
86,86,logicallyfacts.com,https://www.logicallyfacts.com/en/fact-check/false-unrelated-image-shared-as-u-s-warships-heading-to-lebanon-to-support-idf,Unrelated image shared as U.S. warships heading to Lebanon to 'support IDF',2024-08-02 00:00:00+00:00,2024-08-07 00:00:00+00:00,Israel,false,https://pbs.twimg.com/media/GPt7m0BXgAAc3oZ?format=jpg&name=4096x4096,NaN,U.S. warships are en route to the Lebanon coast to support Israel Defense Forces (IDF),"A reverse image search led us to a report dated June 24, 2024, published on the official website of the U.S. Navy, titled ""BALTOPS 24 Concludes In Kiel, Germany.” The report also included the now-viral image. The report stated that 20 NATO allies participated in the 53rd iteration of the Baltic Operations 2024 (BALTOPS) held in Kiel, Germany, on June 20, 2024. The report explains that allies gathered at the port in Kiel to reflect on the 21-day BALTOPS exercise that involved 50 ships, 85 aircraft, and nearly 9,000 personnel. The BALTOPS exercise started on June 8, 2024, in Lithuania and involved exercises by warships of NATO countries in the Baltic Sea. The same image was used in a report on the website of the Defense Visual Information Distribution Service (DVIDS), a project of the U.S. Department of Defense (DoD). The photo, dated July 6, 2024, with the description “Ships participating in exercise Baltic Operations 2024 (BALTOPS 24) steam in formation through the Baltic Sea, June 8, 2024,” was credited to Mario Coto, a U.S. Navy employee. NATO also posted this image on their official X (formerly Twitter) account (archived here) on June 10, 2024, with the caption, “19 Allies, 50+ ships, 85+ aircraft, 9,000 personnel are training together during the annual maritime exercise #BALTOPS24, where Allies will demonstrate NATO’s ability to preserve freedom of navigation and security in the Baltic Sea.""",False,True,False,False,API,False


- claim with "id" 84 -> USE MONTH OF CLAIM DATE
    - Original image is from January and July 2024. 
    - Claim date: 18.08.2024    
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Claim could be reformulated in:
        "This image shows Israeli PM Benjamin Netanyahu's house taken by a Hezbollah drone in August 2024."


- claim with "id" 86 -> USE MONTH OF CLAIM DATE 
    - Claim date: 02.08.2024    
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Claim could be reformulated in:
        "This image shows U.S. warships being en route to the Lebanon coast in August 2024 to support the Israel Defence Forces (IDF)."


- claim with "id" 87 -> USE MONTH OF CLAIM DATE
    - Although the claim reads very similary to the previsous one ("id": 84), it shows a different picture
    - Claim date: 06.08.2024
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Could be reformulated in:
        "This image shows Israeli PM Benjamin Netanyahu inside his office and was captured and published by Hezbollah in August 2024.”
             




- claim with "id" 91 -> USE MONTH OF CLAIM DATE
    - Context: Yemen-Israeli attacks in July (20) 2024 (https://en.wikipedia.org/wiki/20_July_2024_Israeli_attack_on_Yemen)
    - Claim date: 24.07.2024
    - Since the claim and context date are both from July, the month will be used as a date.
    - The second sentence of the claim will not be used, since it does not relate to the shown image ("While in the Muhajireen area in Homs, Al-Jolani elements arrest dozens of young men.")
    - Claim could be refomulated in:
        "This image shows the aftermath of a Houthi attack on Tel Aviv, Israel, in July 2024."





- claim with "id" 92: -> USE MONTH OF CLAIM DATE
    - Claim date: 23.07.2024     
    - Since the original image is from Syria in February 2023 and the exact date of the claim is not relevant, the month of the claim will be used
    - Claim could be reformulated in:
        "This image shows a palestinian child stuck under the rubble in Gaza after an Israeli airstrike in July 2024."






- claim with "id" 93: -> USE CONTEXT DATE (EXACT DAY)
    - Context: Houthi drone attack on Israel on July, 19, 2024 (https://en.wikipedia.org/wiki/2024_Houthi_drone_attack_on_Israel)
    - Claim date: 23.07.2024
    - Since the claim (falsely) relates the image to the specific Houti drone attack on Israel, the exact context date will be used
    - Could be reformulated in: 
        "This image shows Israelis fleeing the airport after the Houthis attack on July 19, 2024."







- clam with "id" 94: -> USE MONTH OF CLAIM DATE
    - Claim date: 20.07.2024
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Original claim: "This photo was taken in Gaza. She was holding her sister's oxygen mask inside the ambulance. She had already passed away, yet her hand remained on the mask firmly. Look what did Israel America Taxpayer with them."
    - The last sentence of the claim is not relevant for the image and will thus be removed.
    - Claim could be refomulated in:
        "This image shows a Palestinian girl in the Gaza war in July 2024. She was holding her sister's oxygen mask inside the ambulance. She had already passed away, yet her hand remained on the mask firmly."







- claim with "id" 95: -> USE MONTH OF CLAIM DATE
    - Claim date: 14.07.2024
    - Since the original image is from a different context and the exact date of the claim is not relevant, the month of the claim will be used as date
    - Claim could be reformulated in:
        "This image shows a man from Gaza holding the hand of his son killed in an Israeli bombing in July, 2024."






- claim with "id" 96: -> USE CONTEXT DATE (EXACT DAY)
    - Context: Israel's attack on UNRWA's Al-Jaouni school in al-Nuseirat camp in central Gaza on July, 6 2024 (https://www.cnn.com/2024/07/06/middleeast/israeli-attack-unwra-school-gaza-intl-latam/index.html).
    - Claim date: 15.07.2024
    - Since the claim relates to the specific Israeli attack on July 6, 2024, the exact context date will be used.
    - Coud be reformulated in:
        "This image shows Mayar Hamida (9 years old) and Bilal Hamida (6 years old) who were killed by Israel along with their mother and grandmother at Al-Jaoni UN school in Al-Nuseirat refugee camp on July, 6 2024."






In [17]:
gaza_israel_dictionary_reformulated_claims_normal_images_part_5 = {

    84: {
        "Claim": "This image shows Israeli PM Benjamin Netanyahu's house taken by a Hezbollah drone in August 2024."
    },

    86: {
        "Claim": "This image shows U.S. warships being en route to the Lebanon coast in August 2024 to support the Israel Defence Forces (IDF)."
    },

    87: {
        "Claim": "This image shows Israeli PM Benjamin Netanyahu inside his office and was captured and published by Hezbollah in August 2024."
    },

    91: {
        "Claim": "This image shows the aftermath of a Houthi attack on Tel Aviv, Israel, in July 2024."
    },

    92: {
        "Claim": "This image shows a palestinian child stuck under the rubble in Gaza after an Israeli airstrike in July 2024."
    },

    93: {
        "Claim": "This image shows Israelis fleeing the airport after the Houthis attack on July 19, 2024."
    },

    94: {
        "Claim": "This image shows a Palestinian girl in the Gaza war in July 2024. She was holding her sister's oxygen mask inside the ambulance. She had already passed away, yet her hand remained on the mask firmly."
    },

    95: {
        "Claim": "This image shows a man from Gaza holding the hand of his son killed in an Israeli bombing in July, 2024."
    },

    96: {
        "Claim": "This image shows Mayar Hamida (9 years old) and Bilal Hamida (6 years old) who were killed by Israel along with their mother and grandmother at Al-Jaoni UN school in Al-Nuseirat refugee camp on July, 6 2024."
    },

}

## verify the length
print(len(gaza_israel_dictionary_reformulated_claims_normal_images_part_5))

9


In [18]:
## combine all dictionaries with normal images into one and check the length

gaza_israel_dictionary_reformulated_claims_normal_images_combined = gaza_israel_dictionary_reformulated_claims_normal_images_part_1  | gaza_israel_dictionary_reformulated_claims_normal_images_part_2 | gaza_israel_dictionary_reformulated_claims_normal_images_part_3 | gaza_israel_dictionary_reformulated_claims_normal_images_part_4 | gaza_israel_dictionary_reformulated_claims_normal_images_part_5
len(gaza_israel_dictionary_reformulated_claims_normal_images_combined)

49

In [19]:
print(len(df_gaza_israel[df_gaza_israel["Normal_Image"] == True]))
print(len(df_gaza_israel[df_gaza_israel["Text_Only_Claim"] == True]))

49
35


### 1.3) Text-Only Claims

In [20]:
## Deciding which text claims need a reformulation and which do not 
## Remove unnecessary phrases like "BREAKING", "Breaking News", "WILD"
## If necessary: add date and context to the claim



### 1) Start with Claims from Snopes, because they need no or little reformulation


df_gaza_israel[(df_gaza_israel["Text_Only_Claim"] == True) & (df_gaza_israel["Website"] == "Snopes")]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
40,40,Snopes,https://www.snopes.com/fact-check/columbia-university-expelled-student-anti-nazi-demonstration/,"Yes, Columbia University expelled student for leading anti-Nazi demonstration in 1936",2025-03-15 00:00:00,2025-03-16 00:00:00,"""Gaza""",true,NaN,"In June 1936, Columbia University expelled student activist Robert Burke for leading an anti-Nazi demonstration.","""In June 1936, Columbia University expelled student activist Robert Burke for leading an anti-Nazi demonstration.""","In 1936, Columbia University expelled Robert Burke, a student who led anti-Nazi demonstrations on campus that year. His appeal to be reinstated was denied. We found numerous articles and pamphlets from that time detailing his expulsion. As such, we rate this claim as true.",True,False,False,False,Manual,True
41,41,Snopes,https://www.snopes.com/fact-check/ice-arrested-deported-mahmoud-khalil/,"Yes, ICE arrested Palestinian activist Mahmoud Khalil, a lawful permanent resident of US",2025-03-09 00:00:00,2025-03-11 00:00:00,"""Gaza""",true,NaN,"In March 2025, U.S. Immigration and Customs Enforcement (ICE) agents arrested and detained Palestinian activist Mahmoud Khalil, a lawful permanent resident of the U.S.","""In March 2025, U.S. Immigration and Customs Enforcement (ICE) agents arrested and detained Palestinian activist Mahmoud Khalil, a lawful permanent resident of the U.S.""","Snopes readers contacted us asking if the story was true — if ICE agents had arrested and detained Khalil. They had. On March 10, Trump posted to his social media platform Truth Social announcing that ICE had arrested and detained Khalil, whom he called a ""Radical Foreign Pro-Hamas Student."" Trump's post said Khalil's arrest was the ""first of many to come"" and that he expected ""every one of America's Colleges and Universities to comply"" with ICE's actions. Secretary of State Marco Rubio said in an X post the day after Khalid was arrested that the department would revoke visas and green cards for ""Hamas supporters,"" and a separate X post from the official Department of Homeland Security account claimed that ""Khalil led activities aligned to Hamas, a designated terrorist organization."" On March 9, Khalil's lawyers filed a petition alleging that his detainment was illegal on the grounds that lawful permanent residents are protected by the First Amendment right to free speech and the Fifth Amendment right to due process.",True,False,False,False,Manual,True
42,42,Snopes,https://www.snopes.com/fact-check/trump-hell-israel-hostages/,Trump Said 'All Hell Will Break Out' If Israeli Hostages Aren't Returned By Inauguration,2025-01-14 00:00:00,2025-01-15 00:00:00,"""Gaza""",correct attribution,NaN,"U.S. President-elect Donald Trump said that if Israeli hostages held by Hamas ""are not back by the time I get into office, all hell will break out in the Middle East.”","""U.S. President-elect Donald Trump said that if Israeli hostages held by Hamas 'are not back by the time I get into office, all hell will break out in the Middle East'.”","Trump did indeed say the above words, speaking at a news conference at Mar-a-Lago in Florida on Jan. 7, 2025. As such, we rate this claim a correct attribution. Trump made the comment in response to a journalist asking about the state of negotiations between Israel and Hamas. He was joined by Steve Witkoff, his pick to serve as the special envoy to the Middle East, who said negotiators were ""making a lot of progress"" and ""hopefully it will all work out and we'll save some lives."" Witkoff also credited Trump's ""stature"" as driving the negotiation. A journalist then asked Trump and Witkoff to clarify past remarks about how ""all hell must be paid"" if Hamas did not release the hostages, to

- claim with "id" 40:
    - No reformulation necessary

- claim with "id" 41:
    - No reformulation necessary

- claim with "id" 42:
    - Date needs to be added 
    - Could be reformulated in:
        "U.S. President-elect Donald Trump said on January 7, 2025 that if Israeli hostages held by Hamas 'are not back by the time I get into office, all hell will break out in the Middle East'.”

- claim with "id" 43:
    - No reformulation necessary

- claim with "id" 46:
    - No reformulation necessary

- claim with "id" 48:
    - Date needs to be added
    - Context (from fact-checking article): Trump & Netanyahu had a phone call on August 14, 2024
    - Could be reformulated in:
        "In a phone call with Benjamin Netanyahu on August 14, 2024, Donald Trump told the Israeli prime minister to stop a Gaza ceasefire deal because it would help Vice President Kamala Harris' 2024 presidential campaign."

- cliam with "id" 49:
    - No reformulation necessary

- claim with "id" 50:
    - No reformulation necessary

- claim with "id" 51:
    - No reformulation necessary



In [21]:
gaza_israel_dictionary_reformulated_claims_text_only_snopes = {

    40: {
        "Claim": "In June 1936, Columbia University expelled student activist Robert Burke for leading an anti-Nazi demonstration."
    },

    41: {
        "Claim": "In March 2025, U.S. Immigration and Customs Enforcement (ICE) agents arrested and detained Palestinian activist Mahmoud Khalil, a lawful permanent resident of the U.S."
    },

    42: {
        "Claim": "U.S. President-elect Donald Trump said on January 7, 2025 that if Israeli hostages held by Hamas 'are not back by the time I get into office, all hell will break out in the Middle East'."
    },

    43: {
        "Claim": "Anthony Bourdain once said, 'Today, nearly everything is made in China. Except for courage. Courage is made in Palestine.'"
    },

    46: {
        "Claim": "In November 2024, rock band U2 and singer Bob Geldof announced a multi-stadium tour to aid the Israeli army."
    },

    48: {
        "Claim": "In a phone call with Benjamin Netanyahu on August 14, 2024, Donald Trump told the Israeli prime minister to stop a Gaza ceasefire deal because it would help Vice President Kamala Harris' 2024 presidential campaign."
    },

    49: {
        "Claim": "On July 22, 2024, Vice President Kamala Harris said, 'Our support for Israel will continue' amid Israeli Prime Minister Benjamin Netanyahu’s visit to the United States."
    },

    50: {
        "Claim": "It is illegal to do a DNA test in Israel."
    },

    51: {
        "Claim": "American writer James Baldwin once said: 'The state of Israel was not created for the salvation of the Jews; it was created for the salvation of the Western interests.'"
    }
}

len(gaza_israel_dictionary_reformulated_claims_text_only_snopes)

9

In [22]:

### 2) Rest of the text-only claims

df_gaza_israel[(df_gaza_israel["Text_Only_Claim"] == True) & (df_gaza_israel["Website"] != "Snopes")][5:10]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
21,21,Reuters,https://www.reuters.com/fact-check/us-unrwa-funding-already-halted-2024-not-by-trump-2025-order-2025-01-28/,"Fact Check: US UNRWA funding already halted in 2024, not by Trump 2025 order",2025-01-20 00:00:00,2025-01-28 00:00:00,"""Gaza Fact Check""",misleading,NaN,Trump halts all funding to UNRWA,"""Trump halts all funding to UNRWA.""","But U.S. funding to UNRWA was already frozen, having been so since 2024. Former U.S. President Joe Biden, along with more than a dozen countries, stopped funding the agency in January 2024 after Israel accused 12 of its 13,000 employees of participating in the deadly Hamas attack on Israel on Oct. 7, 2023, which triggered the Gaza war.",True,False,False,False,Manual,Misleading
23,23,Reuters,https://www.reuters.com/fact-check/guardian-headline-gaza-becoming-city-rivalling-bradford-is-fake-2024-11-27/,Fact Check: Guardian headline on Gaza becoming a city rivalling Bradford is fake \r\n,2024-11-23 00:00:00,2024-11-27 00:00:00,"""Gaza Fact Check""",false,NaN,"Shared on Facebook, the screenshot mimics the layout and style of an opinion column in The Guardian newspaper. The headline reads: “Gaza will rise from the ashes to become a city of the future to rival Bradford.”",“Gaza will rise from the ashes to become a city of the future to rival Bradford.”,"A spokesperson for The Guardian said in an email to Reuters that neither the article nor the headline was ever published by the newspaper.\r\nBegum’s spokesperson said in a separate email that the screenshot is not authentic “but a false slur”.\r\nNo such headline could be found on The Guardian’s website, opens new tab and a list of contributors, opens new tab on The Guardian’s website does not feature Begum’s name.",True,False,False,False,Manual,False
25,25,Reuters,https://www.reuters.com/fact-check/fabricated-report-says-starmer-announced-uk-israel-boycott-2024-10-14/,Fact Check: Fabricated report says Starmer announced UK Israel boycott,2024-10-06 00:00:00,2024-10-14 00:00:00,"""Gaza Fact Check""",false,NaN,"Social media accounts published identical text, on Oct. 6: “BREAKING NEWS UK PM… keir Starmer says that UK will boycott all diplomatic relations with Israel due to its long lasting war in Gaza.”",“BREAKING NEWS UK PM… keir Starmer says that UK will boycott all diplomatic relations with Israel due to its long lasting war in Gaza.”,There were no credible reports of Starmer saying the UK would cease relations with Israel by Oct. 14.,True,False,False,False,Manual,False
28,28,Reuters,https://www.reuters.com/fact-check/israeli-ambassador-ethiopia-remains-post-says-israel-foreign-ministry-2025-04-22/,"Fact Check: Israeli ambassador to Ethiopia remains in post, says Israel foreign ministry",2025-04-07 00:00:00,2025-04-22 00:00:00,"""Israel Fact Check""",misleading,NaN,"“Ethiopia has just expelled Israel's ambassador. Well done Ethiopia,” said an April 7 X post, viewed more than 300,000 times and shared to Facebook.","“Ethiopia has just expelled Israel's ambassador.""","But Israel’s ambassador to Ethiopia, Avraham Neguise, remains in his role as of the time of writing, an Israeli foreign ministry spokesperson told Reuters.",True,False,False,False,Manual,Misleading
29,29,Reuters,https://www.reuters.com/fact-check/world-jewish-congress-did-not-amend-antisemitism-definition-2025-03-26/,Fact Check: World Jewish Congress did not amend antisemitism definition,2025-03-21 00:00:00,2025-03-26 00:00:00,"""Israel Fact Check""",false,NaN,The World Jewish Congress (WJC) has amended the International Holocaust Remembrance Alliance (IHRA) working definition of antisemitism to include; “Conflating the assassination of JFK to jews/israel.” It is now antisemitic to say Israel shot JFK.\r\n\r\n“Amendment to the I

- claim with "id" 7:
    - Reformulate the claim to take the context of Trump winning the election into account
    - Could be reformulated in:
        - "The Huthi rebels announced the final cessation of their military operations in international waters after Trump won the 2024 US presidential election."

- claim with "id" 11:
    - Context: Hezbollah drone attack on Israeli army base on October, 13 2024 (https://www.cnn.com/world/live-news/israel-war-iran-palestine-lebanon-10-13-24/)
    - Add the date of the event
    - Could be reformulated in:
        - "The chief of the Israeli Defense Forces (IDF), Herzi Halevi, was assassinated in a Hezbollah drone attack on an Israeli military base on October 13, 2024."

- claim with "id" 12:
    - Context: The advisory opinion from the ICJ was issued on July 19, 2024
    - Add the month of the context date
    - Could be reformulated in:
        - "Israel has been declared an illegal state by the International Court of Justice in July 2024."

- claim with "id" 17:
    - Claim date: 09.07.2024
    - Since the incident did not occur and the specific date of the claim is not relevant, the month will be used
    - Could be reformulated in:
        - "A US-flagged vessel has been attacked in the Guld of Aden in July 2024. A Spokesperson at Denmark's shipping company Maersk said: 'There was an attack targeted on one of the company's US-flagged vessels at the northernmost part of the Gulf of Aden.'"

- claim with "id" 20:
    - Date does not need to be added
    - Could be reformulated in:
        - "$50 Million in US taxpayer money almost went to fund condoms in Gaza before DOGE cut it."

- claim with "id" 21:
    - Context: Executive Order's signed by Trump on January 20, 2025 (The day he was sworn in as president)
    - Add this context to claim
    - Could be reformulated in:
        - "Trump signed an executive order on January 20, 2025, to halt all funding to UNRWA."

- claim with "id" 23:
    - Adding that the supposed headline is from a Guardian article
    - Could be reformulated in:
        - "The Guardian published an article with the headline: 'Gaza will rise from the ashes to become a city of the future to rival Bradford.'"

- claim with "id" 25:
    - The claims were shared in October 2024
    - Since the claims are false and the exact date of the claim is not relevant, the month will be used
    - Could be reformulated in:
        - "UK Prime Minister Keir Starmer said in October 2024 that the UK will boycott all diplomatic relations with Israel due to its long lasting war in Gaza."


- claim with "id" 28:
    - The claims were shared in April 2025
    - Since the claims are false and the exact date of the claim is not relevant, the month will be used
    - Add the monthly date to the claim
    - Could be reformulated in:
        - “Ethiopia has expelled Israel's ambassador in April 2025."

- claim with "id" 29:
    - The claims were shared in March 2025
    - Since the claims are false and the exact date of the claim is not relevant, the month will be used
    - Add the monthly date to the claim
    - Could be reformulated in:
        - "The World Jewish Congress (WJC) has amended the International Holocaust Remembrance Alliance (IHRA) working definition of antisemitism in March 2025 to include: 'Conflating the assassination of JFK to jews/israel'.”

In [23]:
gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_1 = {

    7: {
        "Claim": "The Huthi rebels announced the final cessation of their military operations in international waters after Trump won the 2024 US presidential election."
    },

    11: {
        "Claim": "The chief of the Israeli Defense Forces (IDF), Herzi Halevi, was assassinated in a Hezbollah drone attack on an Israeli military base on October 13, 2024."
    },

    12: {
        "Claim": "Israel has been declared an illegal state by the International Court of Justice in July 2024."
    },

    17: {
        "Claim": "A US-flagged vessel has been attacked in the Guld of Aden in July 2024. A Spokesperson at Denmark's shipping company Maersk said: 'There was an attack targeted on one of the company's US-flagged vessels at the northernmost part of the Gulf of Aden.'"
    },

    20: {
        "Claim": "$50 Million in US taxpayer money almost went to fund condoms in Gaza before DOGE cut it."
    },

    21: {
        "Claim": "Trump signed an executive order on January 20, 2025, to halt all funding to UNRWA."
    },

    23: {
        "Claim": "The Guardian published an article with the headline: 'Gaza will rise from the ashes to become a city of the future to rival Bradford.'"
    },

    25: {
        "Claim": "UK Prime Minister Keir Starmer said in October 2024 that the UK will boycott all diplomatic relations with Israel due to its long lasting war in Gaza."
    },

    28: {
        "Claim": "Ethiopia has expelled Israel's ambassador in April 2025."
    },

    29: {
        "Claim": "The World Jewish Congress (WJC) has amended the International Holocaust Remembrance Alliance (IHRA) working definition of antisemitism in March 2025 to include: 'Conflating the assassination of JFK to jews/israel'."
    }
}

len(gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_1)

10

In [24]:

## how many remaining claims left?
print(len(df_gaza_israel[(df_gaza_israel["Text_Only_Claim"] == True) & (df_gaza_israel["Website"] != "Snopes")][10:]))

df_gaza_israel[(df_gaza_israel["Text_Only_Claim"] == True) & (df_gaza_israel["Website"] != "Snopes")][18:20]

16


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
67,67,misbar.com,https://www.misbar.com/en/factcheck/2025/03/04/erdogan-did-not-urge-us-control-israel-over-syria-conflict,Erdogan Did Not Urge U.S. To Control Israel Over Syria Conflict,2025-03-04 00:00:00+00:00,2025-03-04 00:00:00+00:00,Israel,fake,NaN,NaN,"A statement attributed to Turkey's President Erdogan, allegedly warning Israel","No Record of Turkish President Making Such a Statement. Contrary to social media users’ viral claim, Misbar’s team found no evidence supporting the viral claim that Turkish President Recep Tayyip Erdogan threatened Israel or urged the United States to control its ally over its intervention in southern Syria. Misbar’s investigative team reviewed Erdogan’s official X account and found no such statement. In addition, Misbar’s team searched the Turkish presidency’s Directorate of Communications but found no official statements mentioning the U.S. or Israel in connection with the recent incursion into southern Syria.",True,False,False,False,API,False
70,70,checkyourfact.com,https://checkyourfact.com/2025/02/06/fact-check-iron-dome-intercept-ballistic-missiles/,FACT CHECK: Does The Iron Dome Intercept 90% Of Ballistic Missiles?,2025-01-28 00:00:00+00:00,2025-02-06 00:00:00+00:00,Israel,unsubstantiated,NaN,NaN,NewsNation and the New York Post reported that the Iron Dome intercepted 90% of ballistic missiles fired at Israel.,"President Donald Trump signed an executive order directing Secretary of Defense Pete Hegseth to develop a missile defense plan to defend the United States from ballistic missiles, hypersonic missiles and other aerial threats, according to DefenseScoop. NewsNation and the New York Post reported on Trump’s missile defense executive order. NewsNation reported that “Iron Dome was 90% effective against ballistic missiles fired at Israel by Iran in October, which has prompted Trump to order the planning of a ‘state-of-the-art’ missile defense system to be designed in the United States.” The New York Post reported that the Iron Dome “blasts ballistic missiles out of the sky with an accuracy rate of roughly 90%.” Both of these claims are misleading. The Iron Dome is designed to shoot down rockets and other short-range munitions, such as mortar shells, according to the BBC. It is true that it has been reported that the Iron Dome has shot down 90% of rockets fired, per the BBC. However, the Iron Dome is not used to intercept ballistic missiles, per the BBC. Israel uses David’s Sling and the Arrows 2 and 3 systems to shoot down cruise and ballistic missiles, per the outlet. Furthermore, Israel used the Arrows 2 and 3 missile defense systems to intercept Iranian ballistic missiles during Iran’s October 2024 attack, according to Axios. In addition, the USS Bulkeley and USS Cole fired a dozen anti-missile interceptors, per the outlet.",True,False,False,False,API,NEI


- claim with "id" 32:
    - There was no drone attack on November 18
    - Claim date: 28.11.2024
    - Use the month date
    - Claim could be reformulated:
        - "Commander of the Israeli Air Force, Major General Tomer Bar has been killed by Hеzbollаh in Tel Aviv in a drone attack in November 2024."


- claim with "id" 33:
    - Context: US charges Iranian man in plot to kill Donald Trump on November, 8 2024, The United States on Nov. 8 charged an Iranian man linked to an alleged plot to assassinate President-elect Trump ordered by Iran’s Revolutionary Guards Corps. (https://www.reuters.com/world/us/us-charges-iranian-man-plot-kill-donald-trump-justice-dept-says-2024-11-08/)
    - Claim date: 09.11.2024
    - Use the year of the assassination attempt on Donald Trump
    - Claim could be reformulated:
        "The assassination plot to kill Donald Trump in 2024 was orchestrated by two Jewish Americans. Israel wants war with Iran but wants American money to fund it and American servicemen to fight it.”


- claim with "id" 36:
    - Context: Iranian attack on Israel on October 1, 2024 (https://en.wikipedia.org/wiki/October_2024_Iranian_strikes_against_Israel)
    - Claim date: 01.10.2024
    - Use the exact context date
    - Claim could be reformulated in:
        "Gallant, a retired military general and a member of Prime Minister Benjamin Netanyahu’s war cabinet, was killed in the Iranian attack on Israel on October 1, 2024."


- claim with "id" 52:
    - no reformulation necessary

- claim with "id" 53:
    - Context: 
        - The social media post: https://truthsocial.com/@realDonaldTrump/posts/112848319395019188
        - The attempted assassination: https://en.wikipedia.org/wiki/Attempted_assassination_of_Donald_Trump_in_Pennsylvania
    - Post date: 25.07.2024
    - Use the posting date
    - Claim could be reformulated in:
        "President Donald Trump posted on Truth Social on July 25, 2024, as a reaction to the attempted assasination of him on July 13, 2024, 'I hope that America obliterates Israel.'"


- claim with "id" 54:
    - Context: As tensions between Israel and Hezbollah escalate, the U.S. State Department has advised Americans in Lebanon to leave the country. (Fact-Checking Article)
    - Claim date: 03.10.2024
    - Use the month date
    - Claim could be reformulated in:
        "As the tensions between Israel and Hezbollah escalate in October, 2024, the U.S. government is offering Americans loans to pay for flights out of Lebanon while they have been flying Israeli citizens out of Israel on chartered flights for a year 'on US taxpayers' dime."

- claim with "id" 55:
    - Only remove special characters, content stays the same
    - "Donald Trump stated on October 7, 2024 in an interview with Hugh Hewitt: Of Gaza, 'I’ve been there.'"

- claim with "id" 64:
    - Claim date: 03.04.2025
    - Since incident did not occur and specific claim date is not relevat, the month date will be used
    - Add that he supposedly fled Iran to Israel
    - Claim could be reformulated to:
        "Iran's IRGC Quds Force commander Gen. Esmail Qaani has fled Iran in April 2025 and is confirmed safe in Israel."


- claim with "id" 67:
    - Context: On February 25, the Israeli military announced strikes on sites in southern Syria, just hours after Syria’s new leadership demanded that Israel withdraw from territories it occupied since the fall of the Assad regime.
    - Claims have been posted in early March 2025
    - Adding the context and date to the claim
    - Claim could be reformulated to:
        "Turkey's President urged the U.S. in March 2025 to control its ally Israel over its intervention in southern Syria in February 2025."


- claim with "id" 70:
    - Context: Iranian attack on Israel on October 1, 2024 (https://en.wikipedia.org/wiki/October_2024_Iranian_strikes_against_Israel)
    - Claim date: 28.01.2025
    - Use the exact date of the context
    - Claim could be reformulated to:
        "The Iron Dome intercepted 90% of ballistic missiles fired at Israel in Iran's attack on October 1, 2024."


In [25]:
gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_2 = {

    32: {
        "Claim": "Commander of the Israeli Air Force, Major General Tomer Bar has been killed by Hеzbollаh in Tel Aviv in a drone attack in November 2024."
    },

    33: {
        "Claim": "The assassination plot to kill Donald Trump in 2024 was orchestrated by two Jewish Americans. Israel wants war with Iran but wants American money to fund it and American servicemen to fight it."
    },

    36: {
        "Claim": "Gallant, a retired military general and a member of Prime Minister Benjamin Netanyahu’s war cabinet, was killed in the Iranian attack on Israel on October 1, 2024."
    },

    52: {
        "Claim": "The population in Gaza has increased by 2.02% since Oct. 7, 2023."
    },

    53: {
        "Claim": "President Donald Trump posted on Truth Social on July 25, 2024, as a reaction to the attempted assasination of him on July 13, 2024, 'I hope that America obliterates Israel.'"
    },

    54: {
        "Claim":  "As the tensions between Israel and Hezbollah escalate in October, 2024, the U.S. government is offering Americans loans to pay for flights out of Lebanon while they have been flying Israeli citizens out of Israel on chartered flights for a year 'on US taxpayers' dime."
    },

    55: {
        "Claim": "Donald Trump stated on October 7, 2024 in an interview with Hugh Hewitt: Of Gaza, 'I’ve been there.'"
    },

    64: {
        "Claim": "Iran's IRGC Quds Force commander Gen. Esmail Qaani has fled Iran in April 2025 and is confirmed safe in Israel."
    },

    67: {
        "Claim": "Turkey's President urged the U.S. in March 2025 to control its ally Israel over its intervention in southern Syria in February 2025."
    },

    70: {
        "Claim": "The Iron Dome intercepted 90% of ballistic missiles fired at Israel in Iran's attack on October 1, 2024."
    }
}

len(gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_2)

10

In [26]:
df_gaza_israel[(df_gaza_israel["Text_Only_Claim"] == True) & (df_gaza_israel["Website"] != "Snopes")][20:]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
74,74,newsmeter.in,https://newsmeter.in/fact-check/no-yair-netanyahu-was-not-assassinated-in-drone-attack-on-israeli-pms-home-737767,"Fact Check: No, Yair Netanyahu was not assassinated in drone attack on Israeli PM’s home",2024-10-21 00:00:00+00:00,2024-10-21 00:00:00+00:00,Israel,false,NaN,NaN,"Netanyahu’s son Yair was killed in a drone attack that targeted the Israeli PM’s home in Caesarea, Israel.","NewsMeter found that the claim is false. A keyword search led us to a report by BBC titled, ‘Netanyahu says he is undeterred after reported drone attack on his home’, dated October 20, 2024. According to the report, a drone was used to target the Israeli PM’s private residence in the coastal town of Caesarea on Saturday morning. Neither Netanyahu nor his wife were present, and no injuries were reported. Further, we found another report by the Times of Israel titled: ‘Israel vows Iran will ‘pay’ for Hezbollah attack on Netanyahu home; Tehran distances itself’, dated October 20, 2024. No casualties were reported. According to the report, on Saturday morning, three drones were allegedly deployed from Lebanon in an assault aimed at the prime minister’s private home. Two of them were stopped near Rosh Hanikra and Nahariya, while the third detonated in Caesarea, a coastal town famous for its luxury homes and ancient Roman remains. At the time, the prime minister and his wife were away from the residence. We found similar updates in reports by India TV, Mint and India Today. None of them mentioned any casualties. Further, we checked Yair’s social media handles and analysed his online activity. We found that Yair Netanyahu's account is active on X. His latest activity was seen on October 21 with a repost of an interview of a Hezbollah leader. We also found Yair’s Instagram account. Although his last post was on October 7, he shared his latest status 10 hours ago. We checked Yair’s Facebook account; his last activity was on October 8. Hence, the claim that the Hezbollah drone attack on the Israeli PM’s home in Caesarea killed his son Yair Netanyahu is false.",True,False,False,False,API,False
77,77,checkyourfact.com,https://checkyourfact.com/2024/10/08/fact-check-did-israel-just-have-an-earthquake/,FACT CHECK: Did Israel Just Have An Earthquake?,2024-10-05 00:00:00+00:00,2024-10-08 00:00:00+00:00,Israel,false,NaN,NaN,A post made on X claims that Israel experienced an earthquake on October 5th.,"The claim is false. USGS data shows that no earthquake occurred in Israel on or shortly before October 5th. No credible sources have reported that an earthquake occurred near Tel Aviv around October 5th. The EMSC made a post on X that verifies there is no data that indicates an earthquake occurred in Israel around October 5th. The caption reads, “#Israel 19 min ago⚠ We have no data confirming this crowdsourced detection. 👉 It likely was a false detection. No more info will be displayed in the thread. ” When asked for comment, EMSC told Check Your Fact, “we stress the preliminary and unconfirmed nature of this detection. The second message in the thread confirms that this detection has not been confirmed.” Check Your Fact has also reached out to USGS for comment. Due to the tension between Israel and Iran, many social media users have attributed the false detection to an alleged Iranian nuclear test. No reputable sources have reported that Iran conducted a nuclear test on or close to October 5th.",True,False,False,False,API,False
78,78,misbar.com,https://www.misbar.com/en/factcheck/2024/10/06/netanyahu-did-not-flee-to-poland-amid-irans-attack,Netanyahu Did Not Flee to Poland Amid Iran's Attack,2024-10-06 00:00:00+00:00,2024-10-06 00:00:00+00:00,Israel,fake,NaN,NaN,Netanyahu lifted off out of Israel ab

- claim with "id" 74:
    - Context: Drone attack on Netanyahu's residence on October 19, 2024 (https://en.wikipedia.org/wiki/2024_drone_attack_on_Benjamin_Netanyahu%27s_residence)
    - Add the context date to claim
    - Claim could be reformulated in:
        "Netanyahu’s son Yair was killed in a drone attack on October 19, 2024 that targeted the Israeli PM’s home in Caesarea, Israel."

- claim with "id" 77:
    - Add the year to the claim
    - Claim could be reformulated to:
        "Israel experienced an earthquake on October 5, 2024."

- claim with "id" 78:
    - Context: Iranian attack on Israel on October 1, 2024 (https://en.wikipedia.org/wiki/October_2024_Iranian_strikes_against_Israel)
    - Add the context date
    - Claim could be reformulated to:
        "Netanyahu left Israel and fled to Poland amid Iran's missile attack on Israel on October 1, 2024."


- claim with "id" 89:
    - Context: Assassination of Ismail Haniyeh on July, 31 2024 (https://en.wikipedia.org/wiki/Assassination_of_Ismail_Haniyeh)
    - Could be reformulated to:
       "For the assassination on July 31, 2024, Israeli intelligence tracked Hamas leader Ismail Haniyeh through WhatsApp."


- claim with "id" 90:
    - Add the Olympics year to the claim
    - Could be reformulated to:
        "An Algerian judo athlete withdrew from his Olympics 2024 match against an Israeli opponent."


- claim with "id" 97:
    - Claim date: 09.07.2024
    - Use the year as a date
    - Could be reformulated to:
        "The US reinstituted draft in 2024 for the Israel-Hamas war."

In [27]:
gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_3 = {

    74: {
        "Claim": "Netanyahu’s son Yair was killed in a drone attack on October 19, 2024 that targeted the Israeli PM’s home in Caesarea, Israel."
    },

    77: {
        "Claim": "Israel experienced an earthquake on October 5, 2024."
    },

    78: {
        "Claim": "Netanyahu left Israel and fled to Poland amid Iran's missile attack on Israel on October 1, 2024."
    },

    89: {
        "Claim": "For the assassination on July 31, 2024, Israeli intelligence tracked Hamas leader Ismail Haniyeh through WhatsApp."
    },

    90: {
        "Claim": "An Algerian judo athlete withdrew from his Olympics 2024 match against an Israeli opponent."
    },

    97: {
        "Claim": "The US reinstituted draft in 2024 for the Israel-Hamas war."
    },

}

len(gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_3)

6

In [28]:
## Combine all dictionaries with reformulated claims

### combine all text-only claims first for a better overview

gaza_israel_dictionary_reformulated_claims_text_only_combined = gaza_israel_dictionary_reformulated_claims_text_only_snopes | gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_1 | gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_2 | gaza_israel_dictionary_reformulated_claims_text_only_remaining_claims_part_3
print(len(gaza_israel_dictionary_reformulated_claims_text_only_combined)) #check the length

gaza_israel_dictionary_all_reformulated_claims = gaza_israel_dictionary_reformulated_claims_ai_and_altered_images | gaza_israel_dictionary_reformulated_claims_normal_images_combined | gaza_israel_dictionary_reformulated_claims_text_only_combined
print(len(gaza_israel_dictionary_all_reformulated_claims)) #check the length to ensure all claims have been reformulated
gaza_israel_dictionary_all_reformulated_claims

35
100


{2: {'Claim': 'This is an authentic image of a mother holding the bones of her son in Gaza in March 2025. After more than a year and a half of searching for him under the rubble of their bombed-out house.'},
 6: {'Claim': 'This is an authentic image of Benjamin Netanyahu being hospitalized in Tel Aviv in November 2024 due to a serious health issue.'},
 10: {'Claim': "This is an authentic image of Israel bombing Beirut's International Airport in October 2024."},
 19: {'Claim': "This is an authentic image of Donald Trump, Benjamin Netanyahu and Elon Musk during Netanyahu's official visit to the U.S. in February 2025."},
 22: {'Claim': 'This is an authentic image of Mike Tyson posing with the Palestinian flag in the ring before his fight against Jake Paul on November 15, 2024.'},
 31: {'Claim': 'This is an authentic image of Bono and Bob Geldof holding the Israeli flag outside the Israeli embassy in Dublin.'},
 79: {'Claim': "This is an authentic image of Israeli Air Force's Ramat David a

In [29]:
## Adding the dictionary data to the df

import numpy as np

def update_df_with_reformulated_claims(df, data_dict, id_column = "id"):


    # Create a copy of the df to avoid modifying the original
    df_updated = df.copy()

    # Create the "Claim" column with the reformulated claims and fill it with nan values to track if a claim misses
    df_updated["Claim"] = np.nan

    # Track IDS not found in the df
    ids_not_found = []

    # Loop through each claim in the dicationary
    for index, data in data_dict.items():
        # Find rows where id column matches the dictionary key
        match = df_updated[id_column] == index

        # If a matching row is found, add the reformulated claims 
        if match.any():
            df_updated.loc[match, "Claim"] = data["Claim"]

        else:
            ids_not_found.append(index)

    
    # Pritn IDs not found (if any exist)
    if ids_not_found:
        print(f"The following IDs were not found in the df: {ids_not_found}")

    return df_updated

In [30]:
df_gaza_israel_updated = update_df_with_reformulated_claims(df = df_gaza_israel, data_dict=gaza_israel_dictionary_all_reformulated_claims)

# Check if any NA values still exist in new "Claim" column
print(df_gaza_israel_updated["Claim"].isna().sum())

# Check column types again
print(df_gaza_israel_updated.dtypes)

df_gaza_israel_updated.head()

0
id                            int64
Website                      object
Article_URL                  object
Headline                     object
Claim_Date                   object
Review_Date                  object
Query/Keyword                object
Label_Website                object
Image_URL                    object
Original_Claim_Website       object
Original_Claim_Only          object
Context/Label_Explanation    object
Text_Only_Claim                bool
Normal_Image                   bool
AI_Generated_Image             bool
Altered_Image                  bool
Data_Collection_Type         object
Label                        object
Claim                        object
dtype: object


/var/folders/yf/8jt3z6xx21z0rv6lrj7sdr800000gn/T/ipykernel_1777/2581872894.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'This is an authentic image of a mother holding the bones of her son in Gaza in March 2025. After more than a year and a half of searching for him under the rubble of their bombed-out house.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_updated.loc[match, "Claim"] = data["Claim"]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label,Claim
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in Gaza spread online,2025-04-06 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://pbs.twimg.com/media/Gn4wjgBXoAALax1?format=jpg&name=medium,"""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip,"" says an April 6, 2025 X post with more than 30,000 likes.","""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip.""","A reverse image search reveals the photograph was uploaded to Wikimedia Commons, a website that collects free-use images, with a description that says that it was taken in 2007 in Iraq's Babil province (archived here). Wikimedia Commons credits a blog post from the US National Guard as the source of the image. While the site has since been deleted, an archived version available via the Internet Archive says that the men in the image are two sergeants named Robbie Doman and Christopher W. Sullenberger. Doman, standing in the middle of the photograph, received media coverage in 2017 when he appeared on the NBC television show ""American Ninja Warrior"" after being injured in an explosion in 2008 (archived here, and here). Several US media outlets published images of Doman from his service in Iraq, which are similar to those used in the social media posts (archived here).",False,True,False,False,Manual,False,"This image shows General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, which has died in the Gaza Strip in April 2025."
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.39432WD,"Pictures show Gaza in ruins, not Myanmar quake damage",2025-03-29 00:00:00,2025-04-17 00:00:00,Israeli-Palestinian conflict,false,https://web.archive.org/web/20250402070342im_/https://scontent-sjc3-1.xx.fbcdn.net/v/t39.30808-6/487419008_122233817522067190_5197398173696178365_n.jpg?stp=dst-jpg_s600x600_tt6&_nc_cat=108&ccb=1-7&_nc_sid=833d8c&_nc_ohc=Vrh6myPuQ6QQ7kNvgEk5XXK&_nc_oc=AdlUD89_DZpLh7YT1h3kFtbXsa5hjVVf72r_jD7lK9JlOF-ZbIpsMP742_7qAbGCwXM&_nc_zt=23&_nc_ht=scontent-sjc3-1.xx&_nc_gid=fSR43b_Iy30vsY8iyeCKXQ&oh=00_AYHnGD8C7-NR7_yARBjobfW6AGmcjuJ4rs3dDhokDkqldw&oe=67F29A7A,"""A 7.9-magnitude earthquake in Myanmar, devastation everywhere, everything in ruins,"" read part of a simplified Chinese caption of a Facebook post published March 29.","""A 7.9-magnitude earthquake in Myanmar, devastation everywhere, everything in ruins.""","However, a reverse image search for the first photo in the post led to a TikTok video shared February 20 by a user based in Gaza. The clip shows the same destroyed building (archived link). A photo from the Turkish Anadolu news agency shows the same scene. ""A view of the destruction in Jabalia camp after Israeli forces withdrew from the area following the ceasefire in Gaza on February 05, 2025,"" the caption says.",False,True,False,False,Manual,False,"This image shows the ruins after a 7.9-magnitude earthquake in Myanmar on March 28, 2025."
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.37W26WD,Image of 'Gazan mother with son's bones' is made with AI,2025-03-24 00:00:00,2025-03-27 00:00:00,Israeli-Palestinian conflict,false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/486290815_1290888865331200_3774810229142652666_n.jpg?_nc_cat=108&ccb=1-7&_nc_sid=127cfc&_nc_ohc=dJB5KMYY-K8Q7kNvwG36ECt&_nc_oc=Adm1o8yPbYbSNkcgvalWokrqaqaaHITQDp4TW4yh1603BysZ0yTW6m3OzdfFRAcVyc0&_nc_zt=23&_nc_ht=scontent-hou1-1.xx&_nc_gid=7AoU9LU1o_p7jCObsgUWlQ&oh=00_AfH7GWH4PBqeBawSJ9_R87ZfrG1kYAoZjldaIHBiuECusg&oe=681E8A2A,"""A mother holds the bones of 

In [31]:
## Make the "manual"/"api" values in the "Data_Collection_Type" in capital letters to be consistent
### with the spelling of all other columns

print(df_gaza_israel_updated["Data_Collection_Type"].value_counts())

df_gaza_israel_updated["Data_Collection_Type"] = df_gaza_israel_updated["Data_Collection_Type"].replace({'manual': 'Manual', 'api': 'API'})

print(df_gaza_israel_updated["Data_Collection_Type"].value_counts())


## Save the updated df for the next step: image scraping

df_gaza_israel_updated.to_csv("../gaza_israel/Combined_dataset/gaza_israel_dataset_combined_010724_300425_after_claim_reformulation.csv")

Data_Collection_Type
Manual    57
API       43
Name: count, dtype: int64
Data_Collection_Type
Manual    57
API       43
Name: count, dtype: int64


## 2) Ukraine-Russia Dataset

In [1]:
### Loading relevant libraries and the gaza-israel dataset

import pandas as pd

df_ukraine_russia = pd.read_csv("../ukraine_russia/Combined_dataset/ukraine_russia_dataset_combined_010724_300425_after_methodology_check_after_label_aggregation.csv", index_col=0)
print(len(df_ukraine_russia))
print(df_ukraine_russia.dtypes)
df_ukraine_russia.head()

80
id                            int64
Website                      object
Article_URL                  object
Headline                     object
Claim_Date                   object
Review_Date                  object
Query/Keyword                object
Label_Website                object
Image_URL                    object
Original_Claim_Website       object
Original_Claim_Only          object
Context/Label_Explanation    object
Text_Only_Claim                bool
Normal_Image                   bool
AI_Generated_Image             bool
Altered_Image                  bool
Data_Collection_Type         object
Label                        object
dtype: object


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.372Y6CV,Fake newspaper cover on Ukrainian soldiers in ...,2025-03-15 00:00:00,2025-03-19 00:00:00,"""War in Ukraine""",altered,https://scontent-hou1-1.xx.fbcdn.net/v/t39.308...,"""The Kursk expedition was a disaster and a com...","""70,000 Ukrainian soldiers in the Kursk region...",But the supposed Hull Daily Mail headline blas...,False,False,False,True,Manual,False
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YR9KZ,"No, Zelensky hasn't bought Eagle's Nest, it is...",2025-02-18 00:00:00,2025-02-27 00:00:00,"""War in Ukraine""",false,NaN,"According to the latest claims, Zelensky alleg...","According to the latest claims, Zelensky alleg...",The Eagle's Nest is in the property of the sta...,True,False,False,False,Manual,False
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YC3DG,Claims that Ukraine banned Truth Social are false,2025-02-20 00:00:00,2025-02-21 00:00:00,"""War in Ukraine""",false,NaN,"""BREAKING: Zelensky blocks access to President...","""BREAKING: Zelensky blocks access to President...",A spokesperson for Trump Media and Technology ...,True,False,False,False,Manual,False
3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36P98ZW,Fake 'apocalypse' cover of The Economist circu...,2024-11-18 00:00:00,2024-12-03 00:00:00,"""War in Ukraine""",false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.308...,"""APOCALYPSE: Allowing missile strikes deep int...","""APOCALYPSE: Allowing missile strikes deep int...","However, The Economist does list Telegram amon...",False,False,False,True,Manual,False
4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36MM6QY,Old photo misrepresented as coffins of 'Britis...,2024-11-03 00:00:00,2024-11-27 00:00:00,"""War in Ukraine""",false,https://images.static.press.net/v2/image/previ...,"""Recently, 18 members of the British special f...","""18 British Special Forces were killed in Ukra...",A reverse image search and keyword searches on...,False,True,False,False,Manual,False


### 2.1) Claims with AI-/altered images

In [2]:
## Have a look at all claims with AI-images (None available)

pd.set_option("max_colwidth", None)

print(len(df_ukraine_russia[df_ukraine_russia["AI_Generated_Image"]==True]))

df_ukraine_russia[df_ukraine_russia["AI_Generated_Image"]==True]

0


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label


In [3]:
## Have a look at all claims with altered images

print(len(df_ukraine_russia[df_ukraine_russia["Altered_Image"] == True]))

df_ukraine_russia[df_ukraine_russia["Altered_Image"] == True]

9


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.372Y6CV,Fake newspaper cover on Ukrainian soldiers in Kursk spreads online,2025-03-15 00:00:00,2025-03-19 00:00:00,"""War in Ukraine""",altered,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/484304343_642451842044376_1862227891799489919_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=127cfc&_nc_ohc=vNbhgJpxh8gQ7kNvwGpkp-E&_nc_oc=AdmpyxpVjf7biRJsqu7qYnoOgqPFCs_D4iP6ar6puC1wqX6YNlcFafIdG-8nnXYzjxc&_nc_zt=23&_nc_ht=scontent-hou1-1.xx&_nc_gid=MAcjtJX9GTgQ2YUC-llHOw&oh=00_AfLo3snw99HxoCPu-SIDqkUTiqR4YFLhMwMCNXXWaZof5A&oe=681FB7CC,"""The Kursk expedition was a disaster and a complete waste of life. Britain egged it on,"" says a March 15, 2025 post on X from Ian Miles Cheong, a far-right blogger who has previously spread other misinformation -- and whose posts have been repeatedly amplified by billionaire X owner Elon Musk. The post claims to show the Hull Daily Mail, a regional newspaper serving the port city of Hull in northeastern England, saying on its front page: ""70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure.""","""70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure.""","But the supposed Hull Daily Mail headline blasting Zelensky over the Kursk incursion is a fabrication.\n""That story re Ukraine was a fake,"" said Lija Kresowaty, a spokesperson for Reach PLC, the Hull Daily Mail's parent company, in a March 18 email.\n",False,False,False,True,Manual,False
3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36P98ZW,Fake 'apocalypse' cover of The Economist circulates online\n\n,2024-11-18 00:00:00,2024-12-03 00:00:00,"""War in Ukraine""",false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/472950587_122151330002360930_7054174540542497030_n.jpg?_nc_cat=109&ccb=1-7&_nc_sid=833d8c&_nc_ohc=nJzv0r9UvFAQ7kNvwHdS2CB&_nc_oc=Adk5vViDiKunFNC7hgI8uM_QEkQ-aiwu8ZcaWUx6ghodO-kSI9bR52qqMUlvcsVztV0&_nc_zt=23&_nc_ht=scontent-hou1-1.xx&_nc_gid=5szBbbYjnhVjBWsAcCSNug&oh=00_AfKyUOV8Ma0o8FuEwfRkU4ytfCL_KOOsMCFLWm_7vpnt_A&oe=681FA57F,"""APOCALYPSE: Allowing missile strikes deep into Russia is the start of World War III,' warns the iconic prophetic magazine The Economist,"" says a November 18, 2024 https://perma.cc/K8MH-DP39.","""APOCALYPSE: Allowing missile strikes deep into Russia is the start of World War III,' warns the iconic prophetic magazine The Economist.""","However, The Economist does list Telegram among its social media channels -- and its website does not show any November 2024 covers similar to the one posted online (archived here).",False,False,False,True,Manual,False
12,12,Reuters,https://www.reuters.com/fact-check/photo-world-leaders-removing-suit-jackets-ukraine-is-altered-2025-03-25/,Fact Check: Photo of world leaders removing suit jackets for Ukraine is altered,2025-03-15 00:00:00,2025-03-25 00:00:00,"""Ukraine Fact Check""",altered,https://leadstories.com/screenshot_3495456.jpg,"""European leaders took off their suits to support Ukraine,"" said social media posts, opens new tab referring to a reporter's criticism, opens new tab (timestamp 1:20) of Zelenskiy for not wearing a suit during a White House visit three days earlier.","""European leaders took off their suits to support Ukraine.""","World leaders gathered for a group photo at a summit in London did not remove their jackets in solidarity with Ukraine, and an image shared online appearing to show them doing so has been manipulated.",False,False,False,True,Manual,False
14,14,Reuters,https://www.reuters.com/fact-check/fox-news-did-not-report-usaid-funded-information-warfare-ukraine-2025-02-27/,Fact Ch

**Deciding on using dates or not** (key question: Does the shared altered image relate to a specific event or has a specific date in the image?):

- claim ("id": 0) ->  USE DATE
    - The altered screenshot does contain a date: Thursday March 13, 2025
    - Thus, a date will be used
    - Claim will be reformulated to:
        "This image shows a screenshot of an authentic Hull Daily Mail's frontpage from March 13, 2025 saying '70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure'."

- claim ("id": 3) -> USE DATE
    - The altered screenshot of a supposedly The Economist cover does not show a date
    - The altered screenshot of a supposedly The Economist cover does not relate to a specific event
    - The fabricated image only shows Trump and Putin with the word "Apocalypse". There is no mentions of any strikes into Russia or WW III. Thus, the mentioning of the strikes of the original claim will be removed.
    - The claim and image started circulating on social media in December 2024
    - The AFP Factcheck checked the November 2024 covers and compared it to the altered one.
    - Thus, both is possible: not use a date or use November/December 2024 or the end of 2024 as a date.
    - Use a general date ("end of 2024") so both November and December 2024 are included. Using no date at all would have the risk that a similar past cover from the Economist with Trump and Putin could be available, although it is not meant: the context is right after the election of Donald Trump for his 2nd term as US president.
 
    - Claim will be reformulated to:
        "This image shows a screenshot of an authentic cover of the magazine The Economist from the end of 2024 illustrating Donald Trump and Vladimir Putin and warning of an apocalypse." 
        

- claim ("id": 12) -> USE DATE
    - Claim date: March 15, 2025
    - The altered image and claim refers to a specific event: Meeting/emergency summit of world leaders in London on March 2, 2025 to support Ukraine (e.g., https://www.express.co.uk/news/world/2021496/full-list-leaders-london-ukraine)
    - Use the place and date of the summit in the reformulated claim
    - Use "London summit on Ukraine" as meeting's title: https://en.wikipedia.org/wiki/2025_London_Summit_on_Ukraine
    - Since there were also non-European leaders in the summit/image (e.g., Canada), the wording will be changed from "European" to "world" leaders
    - Claim will be reformulated to:
        "This is an authentic image of world leaders taking off their suits to symbolically support Ukraine at the London summit on Ukraine on March 2, 2025."
    

- claim ("id": 14) -> DON'T USE DATE
    - Claim date is from February, 02, 2025
    - The altered screenshot of the supposed Fox News article does not have a date.
    - The altered screenshot of the supposed Fox News article does also not relate to a specific event and talks about USAID funding to Ukraine.
    - Don't use a date, since the image does not have one, it does not relate to a specific event with a date and it is not clear, since when the image has been circulating on the internet.
    - Claim will be reformulated to:
        - "This image shows a screenshot of an authentic Fox News article with the title 'Psychological and Information Warfare Center of Ukraine received $140,000,000 per month from USAID'."
       


- claim ("id": 47) -> USE DATE
    - Claim date: March 22, 2025
    - The screenshot of the altered/doctored Charlie Hebdo magazine cover has a date: March 20, 2025
    - This date will be used
    - Claim will be reformulated to:
        - "This image shows a screenshot of an authentic Charlie Hebdo magazine cover from March 20, 2025 illustrating Ukrainian President Volodoymr Zelenskyy lying in a coffin and saying 'Ukraine is an independet state' in French."
    
    
- claim ("id": 48) -> USE DATE
    - Claim date: March 8, 2025
    - The altered image relates to a specific event: the White House meeting between Trump, Vance and Zelenskyy on February 28, 2025
    - This date and place will be used in the reformulated claim
    - Claim will be reformulated to:
        - "This image shows an authentic transcript of the heated White House meeting between US president Donald Trump, vice-president JD Vance and Ukrainian president Volodymyr Zelenskyy on February 28, 2025." 


- claim ("id": 60) -> DON'T USE DATE
    - The altered image does not show a specific date.
    - The altered image also does not relate to a specific event with a date, but rather accuses US politicians of taking bribes from Ukraine.
    - Thus, no date will be used.
    - Claim will be reformulated to:
        - "This image shows an authentic list realeased by Ukraine showing US politicians taking bribes from Ukraine. The highest bribe was 92 Million US-$ for Joe Biden."


- claim ("id": 61) -> USE DATE
    - Claim date: November 8, 2024
    - The altered image/screenshot of the supposed X post has a date: November 6, 2024 
    - This date will be used.
    - Claim will be reformulated to:
        - "This image shows a screenshot of an authentic X post from Elon Musk on November 6, 2025 saying 'The last dollar for Zelenskiy'."

- claim ("id": 70) -> DON'T USE DATE
    - An image of an altered passport of Bashar al-Assad.
    - The image does not show a date or relate to a specific event. 
    - Thus, no date will be used.
    - Claim will be reformulated to:
        - "This image shows an authentic Russian passport of deposed Syrian dictator Bashar al-Assad."









In [4]:
### Using a dictionary to store the manually reformulated claims for AI- and altered images to later add to the dataframe
#### using the indices of the claim for mapping


""" 

General rules/format for manual reformulation of claims with AI-/altered images:

1) If the claim relates to an image (and not a screenshot of something), it will be reformulated into:
    - 'This is an authentic image of .....'
    - Since all of the claims with AI-/altered images have the label "False", this claim reformulation fits to the label
      and it tests the ability of the model to detect the fabricated nature of the image 

2) If the claim relates to a screenshot of a digitally altered content, e.g., newspaper articles, it will be reformulated into:
    - 'This image shows a screenshot of an authentic article/post/question......'
    - The claim reformulation thereby tests the model's ability to detect the altered nature of the content
    - An alternative formulation could have been: 
        - 'This image shows an authentic screenshot of .....'
        - Yet, it is not the screenshot that has been altered, but the content. 

3) If the claim reformulation contains a date, the American date format is used: Month Day, Year.

"""

ukraine_russia_dictionary_reformulated_claims_altered_images = {



0: {
    "Claim": "This image shows a screenshot of an authentic Hull Daily Mail's frontpage from March 13, 2025 saying '70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure'."
},


3: {
    "Claim": "This image shows a screenshot of an authentic cover of the magazine The Economist from the end of 2024 illustrating Donald Trump and Vladimir Putin and warning of an apocalypse." 
},

12: {
    "Claim": "This is an authentic image of world leaders taking off their suits to symbolically support Ukraine at the London summit on Ukraine on March 2, 2025."
},

14: {
    "Claim": "This image shows a screenshot of an authentic Fox News article with the title 'Psychological and Information Warfare Center of Ukraine received $140,000,000 per month from USAID'."
},

47: {
    "Claim": "This image shows a screenshot of an authentic Charlie Hebdo magazine cover from March 20, 2025 illustrating Ukrainian President Volodoymr Zelenskyy lying in a coffin and saying 'Ukraine is an independet state' in French."
},

48: {
    "Claim": "This image shows an authentic transcript of the heated White House meeting between US president Donald Trump, vice-president JD Vance and Ukrainian president Volodymyr Zelenskyy on February 28, 2025." 
},

60: {
    "Claim": "This image shows an authentic list realeased by Ukraine showing US politicians taking bribes from Ukraine. The highest bribe was 92 Million US-$ for Joe Biden."
},

61: {
    "Claim": "This image shows a screenshot of an authentic X post from Elon Musk on November 6, 2025 saying 'The last dollar for Zelenskiy'."
},

70: {
    "Claim": "This image shows an authentic Russian passport of deposed Syrian dictator Bashar al-Assad."
},




}

In [5]:
## check that the dictionary is complete 

print(len(df_ukraine_russia[df_ukraine_russia["Altered_Image"] == True]))

print(len(ukraine_russia_dictionary_reformulated_claims_altered_images))

9
9


### 2.2) Claims with normal images

In [6]:
print(len(df_ukraine_russia[df_ukraine_russia["Normal_Image"] == True]))

df_ukraine_russia[df_ukraine_russia["Normal_Image"] == True][:10] # first 10 samples

17


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36MM6QY,Old photo misrepresented as coffins of 'British soldiers killed in Ukraine war',2024-11-03 00:00:00,2024-11-27 00:00:00,"""War in Ukraine""",false,https://images.static.press.net/v2/image/preview/0503b84fd6676c7af91213f2a1725816/2.3962551.jpg?1,"""Recently, 18 members of the British special forces successfully infiltrated Russia from Kyiv, but they all returned to their country in wooden boxes,"" read a post on Chinese platform Weibo from November 3, 2024.\nText on the picture reads: ""18 British Special Forces were killed in Ukraine. They were liquidated in the Odessa Region.""","""18 British Special Forces were killed in Ukraine. They were liquidated in the Odessa Region.""","A reverse image search and keyword searches on Google and Russian search engine Yandex found the image published on news agency PA Media's website (archived link).\nAccording to the photo caption, it was taken by British Royal Air Force (RAF) photographer Ross Tilly on September 13, 2006 and shows the coffins of 14 British soldiers who were repatriated after they were killed in an air crash in Afghanistan.",False,True,False,False,Manual,False
5,5,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36LH9P7,Zelensky targeted with fake picture of 'Hitler's Mercedes at presidential palace',2024-10-08 00:00:00,2024-11-08 00:00:00,"""War in Ukraine""",false,https://pbs.twimg.com/media/DzTnadMX4AEYfc2?format=jpg&name=medium,"""Hitler's parade car was bought by Zelensky,"" says a post in the Chinese social media platform Weibo from October 8, 2024.\nIt showed an image of a Mercedes-Benz 770 parked outside a building, alongside a photo of Zelensky captioned: ""Zelensky purchased Hitler's car for $15 M"".\nIt said the vehicle's ""mystery"" owner was revealed when the Ukrainian leader ""was spotted getting out of the car in Kyiv"".","""Hitler's parade car was bought by Zelensky for $ 15 M.""",The Ukrainian Center for Combating Disinformation -- part of the country's National Security and Defense Council -- refuted the false claim that the photo showed Hitler's Mercedes at the presidential palace (archived link).,False,True,False,False,Manual,False
6,6,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36H63KK,"Decades-old image shows airshow collision, not 'US fighter jet shot down by Russia'\n\n",2024-09-23 00:00:00,2024-09-27 00:00:00,"""War in Ukraine""",false,https://d3trj3zqmkebtg.cloudfront.net/pics/48/48672_big.jpg,"""A US F16 fighter jet was shot down on the 28th in airspace over the Black Sea after refusing to obey Russian orders. The video was made public,"" read the simplified Chinese caption to a TikTok video shared on September 23, 2024.\nThe video appears to show an image of a fighter jet in flames, followed by photos of US defence chief Lloyd Austin and Russian foreign ministry spokeswoman Maria Zakharova.","""A US F16 fighter jet was shot down on the 28th in airspace over the Black Sea after refusing to obey Russian orders.""","While a US-made F-16 -- supplied to Ukraine by its Western allies -- crashed during a combat mission in August 2024, the image shows a Russian MiG-29 that collided with another aircraft during an airshow on July 24, 1993.",False,True,False,False,Manual,False
8,8,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36FD73Z,"Photos show Chinese medics flying to epicentre of coronavirus outbreak, not 'deployed to aid Russia'\n\n",2024-08-25 00:00:00,2024-09-05 00:00:00,"""War in Ukraine""",false,https://pbs.twimg.com/media/GVxOANvXIAAYreV?format=jpg&name=medium,"""The Chinese Communist Party's Red Cross troops in Russia have changed into winter clothes and set off for the Russian-Ukrainian front to aid Russia in rescu

Methodology: 
- Access fact-checking article again, if the context (date, location) or the alleged or real event can not be inferred from one of the columns of the data frame (e.g., Headline, Claim_Date, Original_Claim_Website, Original_Claim_Only, Label_Explanation/Context)


- claim with "id" 4 -> USE MONTH OF CLAIM DATE 
    - Claim date: 03.11.2024
    - Original_Claim_Only: "18 British Special Forces were killed in Ukraine. They were liquidated in the Odessa Region."
    - Since the image is from 2006 and a different context, the month of the claim will be used as a date, because the claim and image (wrongly) suggest that the British forces were killed in recent fightings in the Odessa Region in Ukraine in November 2024.
    - Use the month as a date
    - Claim will be reformulated in:
        "This image shows the coffins of 18 British Special Forces who were killed in November 2024 in the Odessa Region in Ukraine."

- claim with "id" 5 -> DON'T USE DATE
    - Claim date: 08.10.2024
    - Original_Claim_Only: "Hitler's parade car was bought by Zelensky for $ 15 M."	
    - Context/Label_Explanation: The Ukrainian Center for Combating Disinformation -- part of the country's National Security and Defense Council -- refuted the false claim that the photo showed Hitler's Mercedes at the presidential palace (archived link).
    - Correct context of the image: The image of the Mercedes was taken in California in 2014.
    - I will not use any date, because it is not relevant for verifying the claim. 
    - Claim will be reformulated in:
        "This image shows Hitler's parade car which was bought by the Ukrainian president Volodymyr Zelenskyy for 15 Million US-$."
        

- claim with "id" 6 -> USE CONTEXT DATE (1 month before CLAIM DATE)
    - Claim date: 23.09.2024
    - Original_Claim_Only: "A US F16 fighter jet was shot down on the 28th in airspace over the Black Sea after refusing to obey Russian orders."
    - Context/Label_Explanation: While a US-made F-16 -- supplied to Ukraine by its Western allies -- crashed during a combat mission in August 2024, the image shows a Russian MiG-29 that collided with another aircraft during an airshow on July 24, 1993.
    - The claim together with the image tries to suggest that the F-16 was shot down in a mission in August 2024. 
    - Therefore, August 2024 will be used as a date. 
    - Claim will be refomulated in:
        "This image shows a US F-16 fighter jet which was shot down in August 2024 in airspace over the Black Sea after refusing to obey Russian orders."



- claim with "id" 8: -> USE MONTH OF CLAIM DATE
    - Claim date: 25.08.2024
    - Original_Claim_Only: "The Chinese Communist Party's Red Cross troops in Russia have changed into winter clothes and set off for the Russian-Ukrainian front to aid Russia in rescuing its occupied territories."
    - Context/Label_Explanation: The image is from the coronavirus outbreak in China from January 2020.
    - Since the claim does not specify in which phase or the Ukraine war this image supposedly should be and the claim date is from August 2024, but the claim text mentions winter clothes, no date will be used. Also without a date, the model should be able to detect that the image is not from the Ukraine war at all - no matter in which phase of the war.
    - Claim will be reformulated to:
        "This image shows the Chinese Communist Party's Red Cross troops in Russia who have changed into winter clothes and set off for the Russian-Ukrainian front to aid Russia in rescuing its occupied territories."


- claim with "id" 9: -> USE MONTH OF CLAIM DATE
    - Claim date: 12.08.2024
    - Original_Claim_Only: "Ukrainian President Volodymyr Zelensky arrived in the Sudzha of Kursk region, Russia, which had just been occupied by Ukrainian troops, and met with representatives of the people there."
    - Although the image is from 2022, the claim tried to suggest that it is from the Kursk Offensive 
    - Context: Kursk Offensive August 6, 2024 - March 2025 (https://en.wikipedia.org/wiki/Kursk_offensive_(2024–2025))
    - Since the claim date is also from August 2024, this month will be used as the date
    - Claim will be reformulated in: 
        "This image shows Ukrainian President Volodymyr Zelensky arriving in August 2024 in the Sudzha of Kursk region, Russia, which had just been occupied by Ukrainian troops in the Kursk Offensive, and meeting with representatives of the people there."


- clam with "id" 10: -> USE MONTH OF CONTEXT
    - Claim date: 10.07.2024
    - Original_Claim_Only: "The Ukrainian Government PHOTOSHOPPED fake children’s toys into images of the hospital they destroyed in Kiev. This is worse than ZIONIST PROPAGANDA!"
    - Context: The image shows rubble at Kyiv Okhmatdyt pediatric hospital, which was hit July 8 by a Russian cruise missile that tore through its toxicology department and damaged large portions of the clinic's surrounding buildings.
    - Use the date of the Russian cruise missle attack (July 8)
    - Claim will be refomulated in:
        "This image shows fake children's toys that were photoshopped into the image by the Ukrainian Government after Kyiv's pediatric hospital was attacked by Russian cruise missiles on July 8, 2024."



- claim with "id" 13: -> USE RUSSIAN INVASION IN UKRAINE AS GENERAL DATE REFERENCE
    - Claim date: 02.03.2025
    - Original_Claim_Only: “Whenever you meet your real boss you always wear a suit.”
    - Context/Label_Explanation: The photo of a suited Zelenskiy shaking Klaus Schwab’s hand is from January 2020, two years before Russia invaded Ukraine.
    - The claim suggest that the image is during the Ukraine war and accusing Zelenskyy of supposedly double standards: usually wearing military uniform, but wearing a suit when meeting his real "boss" Klaus Schwab.
    - Hence, no specific date will be used: only mentioning that is is during the Ukraine war to test out whether the model can figure out that the image predates the Ukraine war.
    - Claim will be reformulated in:
        "This image shows Ukrainian President Volodymyr Zelensky wearing a suit after the large-scale Russian invasion in Ukraine: in a talk with Klaus Schwab at the World Economic Forum."


- claim with "id" 22 -> USE MONTH DATE OF CLAIM
    - Claim date: 11.08.2024-08-11
    - Original_Claim_Only: “First picture of a CHEMICAL attack by Ukrainian artillery in KURSK !!!!!!! The Armed Forces of Ukraine used chemical weapons in the Kursk region. According to the Aida group, the enemy used ammunition filled with chlorine.”
    - Context: Kursk Offensive August 6, 2024 - March 2025 (https://en.wikipedia.org/wiki/Kursk_offensive_(2024–2025))
    - The claim tries to suggest that the image shows a chemical attack by Ukrainian forces in the Kursk Offensive in August 2024
    - Therefore, the month of the claim will be used as a date
    - Claim will be reformulated in:
        "This image shows a chemical attack by the Armed Forces of Ukraine during the Kursk offensive in August 2024."


- claim with "id" 33 -> USE CLAIM DATE (EASTER 2025)
    - Claim date: 20.04.2025
    - Original_Claim: A Fox News broadcast labeled Kyiv (the capital of Ukraine) a Russian city on Easter 2025. (Snopes)\n\nOne X user posted a screenshot showing the alleged mislabeling, captioned: "Meanwhile, Fox News claims that Kyiv is Russia and broadcasts 'Patriarch Kirill'.” (X Post)
    - Use Easter 2025, which is also the claim date (20.04.2025) as the date
    - Claim will be reformulated in:
        "This image shows a Fox News broadcast from Easter 2025 where you can see the Patriarch Kirill in the center and Kyiv labeled as a Russian city in the top right corner of the image."


- claim with "id" 49 -> DON'T USE DATE
    - Claim date: 05.03.2025
    - Original_Claim_Only: Ukrainian President Volodymyr Zelenskyy signing bombs, claiming he was signing Israeli bombs intended for use against Palestinians in Gaza.
    - Label/Context_Explanation: A reverse image search led to a YouTube video published by AFP News Agency in September 2024. The video, which included frames similar to the viral image, showed Zelenskyy visiting an ammunition factory in Scranton, Pennsylvania.
    - Since the original image was taken in the US and the bombs were signed for the Ukraine-war and not for Israels bombardment of Gaza, no date will be used. The model needs to figure out that the image is from a different context.
    - Claim could be reformulated in:
        "This image shows Ukrainian President Volodymyr Zelenskyy signing Israeli bombs intended for use against Palestinians in Gaza."


In [7]:
### Using a dictionary to store the manually reformulated claims for normal images to later add to the dataframe
#### using the indices of the claim for mapping




ukraine_russia_dictionary_reformulated_claims_normal_images_part_1 = {

    4: {
        "Claim": "This image shows the coffins of 18 British Special Forces who were killed in November 2024 in the Odessa Region in Ukraine."
    },

    5: {
        "Claim": "This image shows Hitler's parade car which was bought by the Ukrainian president Volodymyr Zelenskyy for 15 Million US-$."
    },

    6: {
        "Claim": "This image shows a US F-16 fighter jet which was shot down in August 2024 in airspace over the Black Sea after refusing to obey Russian orders."
    },

    8: {
        "Claim": "This image shows the Chinese Communist Party's Red Cross troops in Russia who have changed into winter clothes and set off for the Russian-Ukrainian front to aid Russia in rescuing its occupied territories."
    },

    9: {
        "Claim": "This image shows Ukrainian President Volodymyr Zelensky arriving in August 2024 in the Sudzha of Kursk region, Russia, which had just been occupied by Ukrainian troops in the Kursk Offensive, and meeting with representatives of the people there."
    },

    10: {
        "Claim": "This image shows fake children's toys that were photoshopped into the image by the Ukrainian Government after Kyiv's pediatric hospital was attacked by Russian cruise missiles on July 8, 2024."
    },

    13: {
        "Claim": "This image shows Ukrainian President Volodymyr Zelensky wearing a suit after the large-scale Russian invasion in Ukraine: in a talk with Klaus Schwab at the World Economic Forum."
    },

    22: {
        "Claim": "This image shows a chemical attack by the Armed Forces of Ukraine during the Kursk offensive in August 2024."
    }, 

    33: {
        "Claim": "This image shows a Fox News broadcast from Easter 2025 where you can see the Patriarch Kirill in the center and Kyiv labeled as a Russian city in the top right corner of the image."
    },

    49: {
        "Claim": "This image shows Ukrainian President Volodymyr Zelenskyy signing Israeli bombs intended for use against Palestinians in Gaza."
    }

}

## verify the length
print(len(ukraine_russia_dictionary_reformulated_claims_normal_images_part_1))


10


In [8]:
## Last seven claims with normal images

df_ukraine_russia[df_ukraine_russia["Normal_Image"] == True][13:]



,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
75,75,checkyourfact.com,https://checkyourfact.com/2024/09/19/fact-check-small-nuclear-strike-russia/,FACT CHECK: Did A Small Nuclear Strike Hit Russia?,2024-09-18 00:00:00+00:00,2024-09-19 00:00:00+00:00,Russia,false,https://dims.apnews.com/dims4/default/810a90b/2147483647/strip/true/crop/19000x5594+0+0/resize/1440x424!/format/webp/quality/90/?url=https%3A%2F%2Fassets.apnews.com%2F26%2F32%2F8d546e1d085c24b0665a6f399dce%2Fce6c62f457764b4ea6b4ccd54e87a2e8,NaN,"Posts shared on X claims that a North Atlantic Treaty Organization (NATO) strike, possibly nuclear, hit a Russian ammunition depot.","This claim is false. The attack was not from NATO, but a Ukrainian drone strike, according to The Associated Press. A Kyiv security official told the outlet that the attack was carried out by Ukrainian Special Operation Forces, Security Service and Intelligence. At least 100 Ukrainian drones took part in the attack, the outlet reported. The regional governor of Tver told residents they would be able to go back to their homes.",False,True,False,False,API,False
76,76,misbar.com,https://www.misbar.com/en/factcheck/2024/08/26/an-old-image-of-ukrainians-circulated-as-american-mercenaries-invading-russiarsquos-kursk-along-with-ukraine,An Old Image of Ukrainians Circulated As American Mercenaries Invading Russia’s Kursk Along With Ukraine,2024-08-26 00:00:00+00:00,2024-08-26 00:00:00+00:00,Russia,misleading,https://pbs.twimg.com/media/GU1wcUMXAAAfRtb?format=png&name=900x900,NaN,American mercenaries exposed invading Russia's Kursk region These low-brainers provided evidence for their crimes before they were cremated.,"An Image Of A Ukrainian Solider Taken In Balakliia Through Google Reverse Image, Misbar’s team found that the image was originally posted in 2022 before Ukraine’s incursion into the Kursk region. In September 2022, Ukrainian journalist Denys Kazansky posted the image on his Telegram channel along with the caption, “Balakliia is Ukraine!” suggesting that it was originally captured in Balakelya, a city in eastern Ukraine. The image was posted in 2022 by the Ukrainian journalist Denys Kazanskyi before Ukraine’s incursion into the Kursk region.",False,True,False,False,API,Misleading
78,78,misbar.com,https://www.misbar.com/en/factcheck/2024/08/20/poland-marks-historic-victory-over-the-red-army-not-preparing-for-war-with-russia,"Poland Marks Historic Victory Over the Red Army, Not Preparing for War With Russia",2024-08-20 00:00:00+00:00,2024-08-20 00:00:00+00:00,Russia,misleading,https://assets.misbar.com/styles/large_870x490/public/ca394822814b13f54385dea4721da7c695c8af33.jpg.webp?itok=2yKkTtOz,NaN,"POLAND The Army is preparing for war with Russia because We live in times of war, says Prime Minister.","Poland Celebrates Historic Red Army Defeat On August 15, the same footage appeared on DW with the comment: “With tensions rising at its eastern borders, Poland put on a big show with a military parade to celebrate Armed Forces Day. Those tensions are also the reason why the country is aiming to build one of Europe's largest military forces.” Contrary to claims on social media, Poland paid tribute to its historic 1920 triumph over the Red Army on Thursday, August 15, by remembering fallen Poles and showcasing new military equipment. On Armed Forces Day, around 2,500 Polish soldiers, along with dozens of U.S. and other allied forces, marched in military vehicles or tanks, or flew in fighter jets and helicopters in Warsaw, Poland's capital.",False,True,False,False,API,Misleading
79,79,misbar.com,https://www.misbar.com/en/factcheck/2024/08/13/outdated-photo-of-destroyed-ukrainian-tanks-resurfaces-amid-ukraines-kursk-offensive,Outdated Photo of Destroyed Ukrainian Tanks Resurfaces Amid Ukraine's Kurs



- claim with "id" 56 -> USE CONTEXT DATE (EXACT DAY)
    - Claim date: 06.02.2025
    - Original_Claim_Only: "Ben Stiller received $4 million from USAID for his photographs with Ukrainian President Volodymyr Zelensky."
    - Label/Context_Explanation: Stiller met with Volodymyr Zelensky in Kyiv as part of his role as an ambassador for the United Nations Refugee Agency (UNHCR) in 2022.
    - Context: Stiller met with Zelenskyy as a goodwill ambassador of the United Nations High Commissioner for Refugees (UNHCR) on June 20,  2022 (https://www.nbcnews.com/news/world/ben-stiller-meets-zelenskyy-ukraine-refugees-rcna34472; https://abcnews.go.com/International/ben-stiller-meets-volodymyr-zelenskyy-ukraine-world-refugee/story?id=85517989)
    - Use the exact date of the meeting
    - Claim will be reformulated in:
        "This image shows Ben Stiller and Ukrainian President Volodymyr Zelensky at their meeting on June 20, 2022, for which Stiller received $4 million from USAID."



- claim with "id" 63 -> USE MONTH OF CLAIM
    - Claim date: 22.03.2025
    - Original_Claim_Only: "Photo of the US drone shot down by Russian missiles in the hands of the Houthis."
    - Context of the claim: US-Houthi conflict March-May 2025 (https://en.wikipedia.org/wiki/March–May_2025_United_States_attacks_in_Yemen)
    - The claim suggests that the US drone was shot down by Russian missiles during the US-Houthi conflict.
    - Since the claim is also from March 2025, this month will be used as a date.
    - Claim will be reformulated in:
        "This image shows a US drone which was shot down in March 2025 by the Houthis using Russian missiles."
             

- claim with "id" 74 -> USE RUSSIAN INVASION IN UKRAINE AS GENERAL DATE REFERENCE
    - Claim_date: 27.09.2024
    - Original_Claim_Only: "The Russian Embassy in South Africa claimed in a Sept. 26 X post to show a downed Ukrainian F-16."
    - Context/Label_Explanation: The image of the F-16 is from 2019. It is an American F-16 that crashed. This claim is false. Through a reverse image search, Check Your Fact found that the image is from 2019. The image shows an American F-16 that crashed into a California warehouse in May 2019, according to Air and Space Forces Magazine.
    - The claim suggests that the images showes a Ukrainian F-16 which was shot down by Russia during the war in Ukraine. The claim date is from September 2024. 
    - Since the image does not relate to a specific event, the Ukraine war will be used as a general date reference. 
    - The model needs to figure out that the image is actually from a different context (from 2019 in California).
    - Claim will be refomulated in:
        "This image shows a Ukrainian F-16 fighter jet which was shot down by Russia as part of the large-scale Russian invasion in Ukraine."



- claim with "id" 75: -> USE CONTEXT DATE (EXACT DAY)
    - Claim date: 18.09.2024
    - Original_Claim_Only: "Posts shared on X claims that a North Atlantic Treaty Organization (NATO) strike, possibly nuclear, hit a Russian ammunition depot."
    - Context/Label_Explanation: The attack was not from NATO, but a Ukrainian drone strike, according to The Associated Press. A Kyiv security official told the outlet that the attack was carried out by Ukrainian Special Operation Forces, Security Service and Intelligence.
    - The event/context: Toropets depot explosions (https://en.wikipedia.org/wiki/Toropets_depot_explosions)
        - On the night of 17–18 September 2024, during the Russo-Ukrainian War, Ukraine launched a drone attack on the 107th Arsenal, Main Missile and Artillery Directorate (GRAU), an ammunition depot of the Russian Armed Forces in Toropets, Tver Oblast. 
    - Use the date of the event/context.
    - Claim will be reformulated in:
        "This image shows a Russian ammunition depot in Toropets, Tver Oblast, before and after it was hit by a, possible nuclear, strike from the North Atlantic Treaty Organization (NATO)."


- claim with "id" 76: -> USE MONTH OF CLAIM
    - Claim date: 26.08.2024
    - Original_Claim_Only: "American mercenaries exposed invading Russia's Kursk region These low-brainers provided evidence for their crimes before they were cremated"
    - Context/Label_Explanation: Through Google Reverse Image, Misbar’s team found that the image was originally posted in 2022 before Ukraine’s incursion into the Kursk region.
    - Context: Kursk Offensive August 6, 2024 - March 2025 (https://en.wikipedia.org/wiki/Kursk_offensive_(2024–2025))
    - Since the claim is from August 2024 and suggests that the image is related to the Kursk Offensive, which started in August 2024, this month will be used as the date.
    - Claim will be reformulated in: 
        "This image shows American mercenaries participating in Ukraine's incursion into Russia's Kursk Oblast in August 2024."



- clam with "id" 78: -> USE CONTEXT DATE (EXACT DAY)
    - Claim date: 20.08.2024
    - Original_Claim_Only: "POLAND The Army is preparing for war with Russia because We live in times of war, says Prime Minister."
    - Context/Label_Explanation: Contrary to claims on social media, Poland paid tribute to its historic 1920 triumph over the Red Army on Thursday, August 15, by remembering fallen Poles and showcasing new military equipment. 
    - Poland's military parade on August 15, 2024: https://apnews.com/article/poland-armed-forces-day-russia-army-parade-db5c8663bf0b6e3c403b0c8482da4aed
    - The date of the parade will be used.
    - Claim could be refomulated in:
        "This image shows a military parade of Poland's army on August 14, 2024, preparing for war with Russia."



- claim with "id" 79: -> 
    - Claim date: 13.08.2024
    - Original_Claim_Only: "Russian Ministry of Defense: The total losses of the Ukrainian Armed Forces in the Kursk region were recorded as 1,350 soldiers, 29 tanks and 23 armored personnel carriers"
    - Context/Label_Explanation: Contrary to the circulating claim on social media, the photo was not captured in the context of Ukraine's recent advance in Russia's bordering Kursk region. Misbar’s team traced the photo to earlier publications and discovered it had been uploaded to a Vietnamese-language website. The photo, published on June 14, 2023, depicts Leopard 2 tanks and several Ukrainian armored vehicles reportedly destroyed by Russia in Zaporizhia.
    - Context: Kursk Offensive August 6, 2024 - March 2025 (https://en.wikipedia.org/wiki/Kursk_offensive_(2024–2025))
    - The claim wrongly links the image to the Kursk Offensive starting in August 2024. Since the claim is also from August 2024, this month will be used as the date.
    - Because the original claim is only talking about alleged losses of the Ukraninan Armed Forces and does not describe the image, which shows destroyed Ukrainian tanks, the claim will be reformulated substantially.
    - Claim will be reformulated in:
        "This image shows destroyed Ukrainian tanks during Ukraine's incursion into Russia's Kursk Oblast in August 2024."





In [9]:
ukraine_russia_dictionary_reformulated_claims_normal_images_part_2 = {

    56: {
        "Claim": "This image shows Ben Stiller and Ukrainian President Volodymyr Zelensky at their meeting on June 20, 2022, for which Stiller received $4 million from USAID."
    },

    63: {
        "Claim": "This image shows a US drone which was shot down in March 2025 by the Houthis using Russian missiles."
    },

    74: {
        "Claim": "This image shows a Ukrainian F-16 fighter jet which was shot down by Russia as part of the large-scale Russian invasion in Ukraine."
    },

    75: {
        "Claim":  "This image shows a Russian ammunition depot in Toropets, Tver Oblast, before and after it was hit by a, possible nuclear, strike from the North Atlantic Treaty Organization (NATO)."
    },

    76: {
        "Claim": "This image shows American mercenaries participating in Ukraine's incursion into Russia's Kursk Oblast in August 2024."
    },

    78: {
        "Claim": "This image shows a military parade of Poland's army on August 14, 2024, preparing for war with Russia."
    },

    79: {
        "Claim": "This image shows destroyed Ukrainian tanks during Ukraine's incursion into Russia's Kursk Oblast in August 2024."
    },

}

## verify the length
print(len(ukraine_russia_dictionary_reformulated_claims_normal_images_part_2))

7


In [10]:
## combine all dictionaries with normal images into one and check the length

ukraine_russia_dictionary_reformulated_claims_normal_images_combined = ukraine_russia_dictionary_reformulated_claims_normal_images_part_1  | ukraine_russia_dictionary_reformulated_claims_normal_images_part_2 
print(len(ukraine_russia_dictionary_reformulated_claims_normal_images_combined))

print(len(df_ukraine_russia[df_ukraine_russia["Normal_Image"] == True]))

17
17


### 2.3) Text-Only Claims

In [11]:
## Amount of all text-only claims

print(len(df_ukraine_russia[df_ukraine_russia["Text_Only_Claim"] == True]))

54


In [12]:
## Deciding which text claims need a reformulation and which do not 
## Remove unnecessary phrases like "BREAKING", "Breaking News", "WILD"
## If necessary: add date and context to the claim


### 1) Start with Claims from Snopes, because they need no or little reformulation

print(len(df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] == "Snopes")]))


# First 10 Snopes claims
df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] == "Snopes")][:10]

14


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
23,23,Snopes,https://www.snopes.com/fact-check/trump-military-aid-ukraine/,Trump falsely claimed US spent $350B on military aid to Ukraine\n,2025-02-19 00:00:00,2025-03-13 00:00:00,"""Ukraine""",false,NaN,"As of February 2025, the United States had given $350 billion in military aid to Ukraine for its war with Russia.","As of February 2025, the United States had given $350 billion in military aid to Ukraine for its war with Russia.","It's possible to arrive at varying totals depending on how military aid is defined or by considering how much money was allocated — as opposed to actually paid out — by the U.S. government, but none of these totals amounts to $350 billion.\nTrump's claim is incorrect. While it is possible to arrive at varying totals using different ways to categorize or count the financial figures, none amounts to $350 billion. As of this writing, the United States had allocated nearly $183 billion for the response to Russia's 2022 invasion of Ukraine, according to the U.S. government's inspector general overseeing the situation. That amount includes more than just what the U.S. government directly sent to Ukraine — and only $83 billion had actually been paid out as of December 2024. Thus, we rate this claim false.",True,False,False,False,Manual,False
24,24,Snopes,https://www.snopes.com/fact-check/warren-buffett-500-million-ukraine/,Warren Buffett did not send more than $500M to Ukraine — but his son did,2025-03-04 00:00:00,2025-03-12 00:00:00,"""Ukraine""",false,NaN,Warren Buffett has given more than $500 million to Ukraine.,Warren Buffett has given more than $500 million to Ukraine.,"Howard Buffett, Warren Buffett's son, donated more than $800 million to Ukraine since 2022, through his own foundation.",True,False,False,False,Manual,False
25,25,Snopes,https://www.snopes.com/fact-check/jd-vance-cousin-nate-fought-in-ukraine-3-years/,"Yes, Vice President JD Vance's cousin Nate fought in Ukraine for three years\n",2025-03-11 00:00:00,2025-03-12 00:00:00,"""Ukraine""",true,NaN,U.S. Vice President JD Vance's cousin Nate Vance fought in Ukraine for three years during its war with Russia.,U.S. Vice President JD Vance's cousin Nate Vance fought in Ukraine for three years during its war with Russia.,"Snopes readers searched our site and submitted emails asking if the claim was true. While we could not independently confirm that the claim was true, several reputable news organizations from various countries, including CNN and French newspaper Le Figaro (archived), all reported that Vance volunteered to fight in Ukraine and served alongside Ukrainian forces for three years, and provided sufficient photographic evidence.",True,False,False,False,Manual,True
26,26,Snopes,https://www.snopes.com/fact-check/zelenskyy-us-ukraine-war-quote/,Zelenskyy saying US will have to send 'sons and daughters to war' in Ukraine is from old video,2025-03-03 00:00:00,2025-03-05 00:00:00,"""Ukraine""",miscaptioned,NaN,"Ukrainian President Volodymyr Zelenskyy said the U.S. will have ""to send their sons and daughters"" to die for Ukraine after having a heated exchange with U.S. President Donald Trump at the White House in February 2025.","Ukrainian President Volodymyr Zelenskyy said the U.S. will have ""to send their sons and daughters"" to die for Ukraine after having a heated exchange with U.S. President Donald Trump at the White House in February 2025.","Zelenskyy did say, ""The U.S. will have to send their sons and daughters exactly the same way as we are sending [our] sons and daughters to war."" However, he was talking about a hypothetical situation in which Ukraine losing the war against Russia would lead to a broader conflict resulting in Americans being pulled into fighting. He made this sta


- claim with "id" 23:
    - No reformulation necessary

- claim with "id" 24:
    - Date needs to be added
    - Add: since 2022

- claim with "id" 25:
    - No reformulation necessary

- claim with "id" 26:
    - No reformulation necessary

- claim with "id" 27:
    - No reformulation necessary

- claim with "id" 28:
    - Date needs to be added
    - Add: In March 2025

- claim with "id" 29:
    - As decided in the label aggregation, the claim will be reformulated to fit the new label "True":
        "On Feb. 28, 2025, before a meeting with U.S. President Donald Trump, Ukraine President Volodymyr Zelenskyy met with Democrats."

- claim with "id" 30:
    - No reformulation necessary
    

- claim with "id" 31:
    - No reformulation necessary

- claim with "id" 32:
    - No reformulation necessary



In [13]:
ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_1 = {

    23: {
        "Claim": "As of February 2025, the United States had given $350 billion in military aid to Ukraine for its war with Russia."
    },

    24: {
        "Claim": "Warren Buffett has given more than $500 million to Ukraine since 2022."
    },

    25: {
        "Claim": "U.S. Vice President JD Vance's cousin Nate Vance fought in Ukraine for three years during its war with Russia."
    },

    26: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy said the U.S. will have 'to send their sons and daughters' to die for Ukraine after having a heated exchange with U.S. President Donald Trump at the White House in February 2025."
    },

    27: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy owns 15 homes including a $35 million home in Florida, has $1.2 billion in an overseas bank account, owns 15 homes, three private planes and has a monthly income of $11 million."
    },

    28: {
        "Claim": "In March 2025 former President of Poland Lech Wałęsa wrote a letter to Donald Trump criticizing the U.S. President's decision to suspend the delivery of all U.S. military aid to Ukraine."
    },

    29: {
        "Claim": "On Feb. 28, 2025, before a meeting with U.S. President Donald Trump, Ukraine President Volodymyr Zelenskyy met with Democrats."
    },

    30: {
        "Claim": "In early 2025, Ukraine and Russia had reached a peace deal in their years-long war."
    },

    31: {
        "Claim": "In August 2024, Ukraine passed legislation banning the Orthodox Church in Ukraine."
    },

    32: {
        "Claim": "U.S. Vice President Kamala Harris once said, 'Ukraine is a country in Europe. It exists next to another country called Russia. Russia is a bigger country. Russia is a powerful country. Russia decided to invade a smaller country called Ukraine. So basically that's wrong.'"
    }
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_1)

10

In [14]:
# Remaining 4 Snopes claims
df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] == "Snopes")][10:]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
34,34,Snopes,https://www.snopes.com/fact-check/joe-rogan-russia-canada/,Joe Rogan said he'd rather visit Russia than Canada. He also said 'Canadian people are awesome'\n,2025-03-24 00:00:00,2025-03-25 00:00:00,"""Russia""",correct attribution,NaN,"Podcaster and UFC commentator Joe Rogan said, ""I don't go to Canada anymore,"" then added, ""I'd rather go to Russia.""","Podcaster and UFC commentator Joe Rogan said, ""I don't go to Canada anymore,"" then added, ""I'd rather go to Russia.""","Rogan laughed after making the remark but provided no direct indication that he was joking. The comment occurred during a podcast episode of ""The Joe Rogan Experience"" on March 22. Earlier in the same episode, he said of Canada, ""They have a terrible government but Canadian people are awesome.""",True,False,False,False,Manual,True
35,35,Snopes,https://www.snopes.com/fact-check/putin-wants-fauci-extradited-to-russia/,No Evidence Vladimir Putin Wants Fauci Extradited To Russia\n,2025-01-22 00:00:00,2025-01-23 00:00:00,"""Russia""",false,NaN,"In January 2025, Russian President Vladimir Putin was seeking to have Dr. Anthony Fauci in the U.S. extradited to Russia in exchange for ending the war in Ukraine.","In January 2025, Russian President Vladimir Putin was seeking to have Dr. Anthony Fauci in the U.S. extradited to Russia in exchange for ending the war in Ukraine.","The claim was false, however. If Putin had made such a request, reliable news sources in either Russia or the United States would have reported on it. That had not happened. If Russia wanted to extradite Fauci, it would likely have to file a Red Flag Notice with Interpol. That, too, had not happened.\nEven if the claim came from a reliable source of information, there's a major hurdle standing in the way: The United States and Russia do not have an extradition treaty with each other. According to the U.S. Department of Justice, the U.S. does not generally extradite without a treaty.",True,False,False,False,Manual,False
36,36,Snopes,https://www.snopes.com/fact-check/russian-tv-melania-trump/,Russian State TV Aired Nude Photos of Melania Trump?,2024-11-10 00:00:00,2024-11-12 00:00:00,"""Russia""",true,NaN,Russian state TV aired nude photos of Melania Trump shortly after the 2024 U.S. presidential election.,Russian state TV aired nude photos of Melania Trump shortly after the 2024 U.S. presidential election.,"The photos of Melania Trump used in the broadcast were not new or leaked images but widely known and previously published photos from her modeling career. Melania Trump addressed the photos in September 2024, saying: ""Why do I stand proudly behind my nude modeling work? The more pressing question is, why has the media chosen to scrutinize my celebration of the human form in a fashion photo shoot? Are we no longer able to appreciate the beauty of the human body?""",True,False,False,False,Manual,True
37,37,Snopes,https://www.snopes.com/fact-check/biden-board-empty-plane/,"No, Biden Did Not Board 'Empty Plane' After Prisoner Exchange with Russia",2024-08-02 00:00:00,2024-08-02 00:00:00,"""Russia""",false,NaN,U.S. President Joe Biden boarded an empty plane after three freed Americans arrived back in the U.S. and exited the aircraft.,U.S. President Joe Biden boarded an empty plane after three freed Americans arrived back in the U.S. and exited the aircraft.,"In sum, Biden did not board an ""empty plane."" Video evidence and information from eyewitnesses show Biden walked onto the plane to visit at least one pilot and possibly other people who were still onboard.",True,False,False,False,Manual,False


- claim with "id" 34:
    - Date needs to be added
    - Add: In March 2025

- claim with "id" 35:
    - No reformulation necessary

- claim with "id" 36:
    - No reformulation necessary

- claim with "id" 37:
    - Date needs to be added
    - US-Russia prisoner exchange was in August 2024 (https://apnews.com/article/russia-gershkovich-whelan-d803e266cb4e60135ec5d668d684529f)
    - Add August 2024




In [15]:
ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_2 = {

    34: {
        "Claim": "In March 2025 podcaster and UFC commentator Joe Rogan said, 'I don't go to Canada anymore,' then added, 'I'd rather go to Russia.'"
    },

    35: {
        "Claim": "In January 2025, Russian President Vladimir Putin was seeking to have Dr. Anthony Fauci in the U.S. extradited to Russia in exchange for ending the war in Ukraine."
    },

    36: {
        "Claim": "Russian state TV aired nude photos of Melania Trump shortly after the 2024 U.S. presidential election."
    },

    37: {
        "Claim": "In August 2024 U.S. President Joe Biden boarded an empty plane after three freed Americans arrived back in the U.S. and exited the aircraft."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_2)

4

In [16]:
## Combine the ukraine-russia text only snopes dictionaries

ukraine_russia_dictionary_reformulated_claims_text_only_snopes_combined = ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_1 |ukraine_russia_dictionary_reformulated_claims_text_only_snopes_part_2
len(ukraine_russia_dictionary_reformulated_claims_text_only_snopes_combined)

14

In [17]:
### 2) Rest of the text-only claims (n=40)

pd.set_option("max_colwidth", 250)


print(len(df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")])) ## 40 claims


## First 5 claims
df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][:5]


40


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YR9KZ,"No, Zelensky hasn't bought Eagle's Nest, it is still in ownership of Bavarian state",2025-02-18 00:00:00,2025-02-27 00:00:00,"""War in Ukraine""",false,NaN,"According to the latest claims, Zelensky allegedly purchased another site steeped in history: the so-called Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros.","According to the latest claims, Zelensky allegedly purchased another site steeped in history: the so-called Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros.","The Eagle's Nest is in the property of the state of Bavaria. The administration of state-owned real estate is the responsibility of IMBY, which is listed in the alleged purchase agreement. On February 7, 2025, the company posted a press release o...",True,False,False,False,Manual,False
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YC3DG,Claims that Ukraine banned Truth Social are false,2025-02-20 00:00:00,2025-02-21 00:00:00,"""War in Ukraine""",false,NaN,"""BREAKING: Zelensky blocks access to President Trump's social media platform Truth Social across Ukraine,"" says a February 20, 2025 X post from conservative influencer Benny Johnson, who AFP has previously fact-checked.","""BREAKING: Zelensky blocks access to President Trump's social media platform Truth Social across Ukraine.""","A spokesperson for Trump Media and Technology Group, which owns Truth Social, said the company has not launched the micro-blogging site in Ukraine.\n""Reports that Truth Social was blocked in Ukraine are false -- Truth Social has never been availa...",True,False,False,False,Manual,False
7,7,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36FQ2MU,BlackRock did not ask Ukraine to 'stop burying dead soldiers on its land'\n\n,2024-08-03 00:00:00,2024-09-13 00:00:00,"""War in Ukraine""",false,NaN,"""BlackRock has asked that Ukrainian soldiers not be buried on the land it owns in Ukraine,"" read a simplified Chinese X post shared on August 3, 2024.\nIt said BlackRock representatives ""have already bought 47 percent of Ukraine's land"" and reque...","""BlackRock has asked that Ukrainian soldiers not be buried on the land it owns in Ukraine.""","However, a representative for BlackRock said the claim was baseless.\n""This is completely false,"" the spokesperson told AFP on August 22. ""BlackRock Inc. does not own any land in Ukraine.""\nA spokesperson for the land committee of the Ukrainian A...",True,False,False,False,Manual,False
11,11,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.363K9LL,Posts falsely claim Zelensky's wife purchased $5 million luxury car\n\n,2024-07-06 00:00:00,2024-07-11 00:00:00,"""War in Ukraine""",false,NaN,"""Zelinsky's wife just bought a $5,000,000.00 Bugatti while vacationing in Paris. Bravo! Let's send more money over,"" says a July 6, 2024 Facebook post.","""Zelinsky's wife just bought a $5,000,000.00 Bugatti while vacationing in Paris. Bravo! Let's send more money over.""","The main piece of evidence in the posts is a photo of an invoice with the Bugatti logo.\nThe document indicates Zelenska should receive the luxury car in 2026. The invoice also says the price is 4,462,400, but it does not specify any currency.\nC...",True,False,False,False,Manual,False
15,15,Reuters,https://www.reuters.com/fact-check/zelenskiys-latest-approval-rating-is-63-not-4-contrary-trumps-claim-2025-02-21/,"Fact Check: Zelenskiy's latest approval rating is 63%, not 4%, contrary to Trump's claim",2025-02-18 00:00:00,2025-02-21 00:00:00,"""Ukraine Fact Check""",false,NaN,"""We have a situation where we haven't had elections in

- claim with "id" 1:
    - Reformulate the claim to remove irrelevant information
    - Claim will be reformulated in:
        - "Ukrainian President Volodymyr Zelenskyy purchased the Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros."

- claim with "id" 2:
    - Remove: "BREAKING"
    - Add the date of the supposed event: February 2025
    - Claim will be reformulated in:
        - "In February 2025 Ukrainian President Volodymyr Zelenskyy blocked access to US President Donald Trump's social media platform Truth Social across Ukraine."

- claim with "id" 7:
    - No reformulation necessary

- claim with "id" 11:
    - Remove the second sentence of the claim (irrelevant)
    - Remove the "just" in first sentence
    - Claim will be reformulated in:
        - "Ukrainian President Volodymyr Zelenskyy's wife bought a $5,000,000.00 Bugatti while vacationing in Paris."

- claim with "id" 15:
    - Claim was made by Trump on February 18, 2025
    - Add February 2025 as date
    - Since the fact-check only concentrates on the approval rating of Zelenskyy, the first part of the claim will be removed (elections, martial law)
    - Claim will be reformulated in:
        - "In February 2025 Ukrainian President Volodymyr Zelenskyy's approval rating in Ukraine was down at 4%."




In [18]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_1 = {

    1: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy purchased the Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros."
    },

    2: {
        "Claim": "In February 2025 Ukrainian President Volodymyr Zelenskyy blocked access to US President Donald Trump's social media platform Truth Social across Ukraine."
        },

    7: {
        "Claim": "BlackRock has asked that Ukrainian soldiers not be buried on the land it owns in Ukraine."
    },

    11: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy's wife bought a $5,000,000.00 Bugatti while vacationing in Paris."
    },

    15: {
        "Claim": "In February 2025 Ukrainian President Volodymyr Zelenskyy's approval rating in Ukraine was down at 4%."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_1)

5

In [19]:
## Next 5 claims (claim 5-10)
df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][5:10]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
16,16,Reuters,https://www.reuters.com/fact-check/zelenskiys-quote-about-trump-nato-membership-lacks-context-2024-12-20/,Fact Check: Zelenskiy’s quote about Trump and NATO membership lacks context\n,2024-12-09 00:00:00,2024-12-20 00:00:00,"""Ukraine Fact Check""",missing context,NaN,"Posts on Facebook, and X in December, quoted the Ukrainian leader as saying: “ZELENSKY: ""I WILL ASK BIDEN TO INVITE ME TO NATO...THERE'S NO POINT IN TALKING TO TRUMP.""","“ZELENSKY: ""I WILL ASK BIDEN TO INVITE ME TO NATO...THERE'S NO POINT IN TALKING TO TRUMP.""",Zelenskiy told reporters at a joint press conference with German opposition leader Friedrich Merz on Dec. 9 that he hoped to call outgoing U.S. President Joe Biden in the coming days to discuss an invitation to the military alliance.\nHe did also...,True,False,False,False,Manual,Misleading
17,17,Reuters,https://www.reuters.com/fact-check/ukraines-president-zelenskiy-did-not-buy-palace-des-neiges-hotel-french-ski-2024-12-13/,Fact Check: Ukraine's President Zelenskiy did not buy 'Palace des Neiges' hotel in French ski resort,2024-11-29 00:00:00,2024-12-13 00:00:00,"""Ukraine Fact Check""",false,NaN,"Posts on Facebook, and X said Zelenskiy purchased an ""88 million euro hotel in the Courchevel ski resort in France.""","""Zelenskiy purchased an 88 million euro hotel in the Courchevel ski resort in France.""","While the hotel is real, and was in fact bought by SBM, last year, neither Zelenskiy nor Film Heritage was involved in that sale. A spokesperson for the office of the president of Ukraine said the report was ""fake"". The hotel was not resold in Oc...",True,False,False,False,Manual,False
18,18,Reuters,https://www.reuters.com/fact-check/russian-man-sentenced-14-years-jail-treason-not-burning-koran-2024-12-03/,"Fact Check: Russian man sentenced to 14 years in jail for treason, not for burning Koran",2024-11-26 00:00:00,2024-12-03 00:00:00,"""Ukraine Fact Check""",false,NaN,"Nov. 26 posts on X and Facebook, some sharing an image of Zhuravel and others of Russian President Vladimir Putin holding a copy of the Koran, falsely said: “BREAKING: RUSSIA has sentenced a man to 14 YEARS IN PRISON for BURNING the QURAN.”",“BREAKING: RUSSIA has sentenced a man to 14 YEARS IN PRISON for BURNING the QURAN.”,"A Russian man was jailed for 14 years in November for state treason, not for burning a copy of the Koran, contrary to online claims.\nNikita Zhuravel, 20, was already serving three and a half years in prison for burning a copy of the Koran when h...",True,False,False,False,Manual,False
19,19,Reuters,https://www.reuters.com/fact-check/italys-meloni-comment-misrepresented-condemnation-us-stance-ukraine-missiles-2024-11-29/,Fact Check: Italy's Meloni comment misrepresented as condemnation of US stance on Ukraine missiles\n,2024-11-20 00:00:00,2024-11-29 00:00:00,"""Ukraine Fact Check""",misleading,NaN,"A day later, accounts on X and Facebook said: “BREAKING: Italian Prime Minister Meloni CONDEMNED the US-Ukrainian BOMBING of Russia.”",“BREAKING: Italian Prime Minister Meloni CONDEMNED the US-Ukrainian BOMBING of Russia.”,"Italian Prime Minister Giorgia Meloni said at the G20 summit in Rio de Janeiro that she understood the U.S. decision to approve Ukraine using its weapons in Russia, but her comments have been misleadingly represented online as a condemnation.",True,False,False,False,Manual,Misleading
20,20,Reuters,https://www.reuters.com/fact-check/no-evidence-nato-chief-said-hell-expel-us-if-trump-surrenders-ukraine-2024-11-25/,Fact Check: No evidence NATO chief said he’ll expel US if Trump ‘surrenders Ukraine’\n,2024-11-05 00:00:00,2024-11-26 00:00:00,"""Ukraine Fact Check""",no evidence,NaN,"Posts on Facebook shared after Trump won the Nov. 5 U.S. pr

- claim with "id" 16:
    - The press conference was on December 9, 2024
    - Add this context information
    - Claim will be reformulated in:
        - "In a press conference on December 9, 2024 Ukrainian President Volodymyr Zelenskyy said: 'I will ask Biden to invite me to NATO ... there's no point in talking to Trump.'"

- claim with "id" 17:
    - Just add Ukrainian President and first name
    - Remain of claim stays the same
    - Claim will be reformulated in:
        - "Ukrainian President Volodymyr Zelenskyy purchased an 88 million euro hotel in the Courchevel ski resort in France."


- claim with "id" 18:
    - Remove the "BREAKING"
    - Add the month of the sentence: November 2024
    - Claim will be reformulated in:
        - “In November 2024 Russia sentenced a man to 14 years in prison for burning the Quran."

- claim with "id" 19:
    - Remove the "BREAKING"
    - Add the context: G20 summit in Rio de Janeiro November 2024
    - Claim will be reformulated in:
        - "Italian Prime Minister Giorgia Meloni condemned the US-Ukrainian bombing of Russia at the G20 summit in Rio de Janeiro in November 2024.”


- claim with "id" 20:
    - Remove the "BREAKING"
    - Remove the last part of the alleged quote 'asking if that “would be a bad thing”.' (not relevant)
    - The claim emerged in November 2024 after Trump won the presidential election
    - Add this context to the claim 
    - Claim will be reformulated in: 
        "After Donald Trump won the US presidential election in 2024, NATO Secretary General Mark Rutte said that ‘If Trump surrenders Ukraine to Putin, he will personally expel the United States from the alliance’.”

In [20]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_2 = {

    16: {
        "Claim": "In a press conference on December 9, 2024 Ukrainian President Volodymyr Zelenskyy said: 'I will ask Biden to invite me to NATO ... there's no point in talking to Trump.'"
    },

    17: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy purchased an 88 million euro hotel in the Courchevel ski resort in France."
        },

    18: {
        "Claim": "In November 2024 Russia sentenced a man to 14 years in prison for burning the Quran."
    },

    19: {
        "Claim": "Italian Prime Minister Giorgia Meloni condemned the US-Ukrainian bombing of Russia at the G20 summit in Rio de Janeiro in November 2024."
    },

    20: {
        "Claim": "After Donald Trump won the US presidential election in 2024, NATO Secretary General Mark Rutte said that ‘If Trump surrenders Ukraine to Putin, he will personally expel the United States from the alliance’."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_2)

5

In [21]:
# The next 5 claims (claim 10-15)

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][10:15]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
21,21,Reuters,https://www.reuters.com/fact-check/fabricated-quote-by-sheinbaum-calling-zelenskiy-drug-addict-2024-09-30/,Fact Check: Fabricated quote by Sheinbaum calling Zelenskiy a ‘drug addict’ \n,2024-09-18 00:00:00,2024-09-30 00:00:00,"""Ukraine Fact Check""",false,NaN,"Posts on social media falsely cite Sheinbaum as rejecting the invitation by saying, about Zelenskiy, “If I wanted to see a drug addict, I'd just take a walk around Tijuana.”","“If I wanted to see a drug addict, I'd just take a walk around Tijuana.”","Mexican President-elect Claudia Sheinbaum said on Sept. 17 that she did not immediately plan to visit Ukraine, citing her prioritization of domestic issues and Mexico’s principle of non-interference in foreign affairs. She did not call Ukrainian ...",True,False,False,False,Manual,False
38,38,Politifact,https://www.politifact.com/factchecks/2025/mar/20/facebook-posts/was-zelenskyys-pennsylvania-visit-election-interfe/,Ukraine President Volodymyr Zelenskyy “engaged in election interference by campaigning” in Pennsylvania for former Democratic presidential nominee Kamala Harris.,2025-03-01 00:00:00,2025-03-20 00:00:00,"""Ukraine""",false,NaN,"""Zelenskyy is an ungrateful foreigner who engaged in election interference by campaigning for Kamala (Harris) in (Pennsylvania),"" said text on the March 1 Facebook Reel posted by Scotty Moore, a Republican who lost a 2023 Florida House race and a...","Zelenskyy ""engaged in election interference by campaigning"" in Pennsylvania for Harris.","A Facebook post said Zelenskyy ""engaged in election interference by campaigning"" in Pennsylvania for Harris.\nIn September 2024, Zelenskyy visited a Scranton ammunition plant to thank its workers who were producing munitions for Ukraine. A video ...",True,False,False,False,Manual,False
39,39,Politifact,https://www.politifact.com/factchecks/2025/mar/14/alex-jones/infowars-reporter-jamie-white-who-was-fatally-shot/,Deceased InfoWars reporter Jamie White was on a “Ukrainian enemies hit list.”\n,2025-03-11 00:00:00,2025-03-14 00:00:00,"""Ukraine""",false,NaN,"Jones pointed to a June 12, 2024, X post from White that read, ""I’m on the Ukrainian ‘Enemies List’ due to my work at @infowars and with @RealAlexJones on the Ukraine proxy war.""",Deceased InfoWars reporter Jamie White was on a “Ukrainian enemies hit list.”,"After InfoWars reporter Jamie White was fatally shot March 10 in Austin, Texas, police said they believe the suspects were burglarizing White’s car when White interrupted them.\nWhite is not on a Ukrainian enemies list. In June 2024, Texty.org.ua...",True,False,False,False,Manual,False
40,40,Politifact,https://www.politifact.com/factchecks/2025/feb/24/threads-posts/elon-musk-didnt-post-that-he-would-suspend-volodym/,Elon Musk posted on X he is planning to suspend Ukraine President Volodymyr Zelenskyy’s X account.,2025-02-19 00:00:00,2025-02-24 00:00:00,"""Ukraine""",false,NaN,"""We are looking into suspending Ukraine Propagandist Zelenskyy’s account,"" a Feb. 19 Threads screenshot of a social media post sharing an X post that appeared to be from Musk’s X account said.",Elon Musk posted on X he is planning to suspend Ukraine President Volodymyr Zelenskyy’s X account.,The post is not on Elon Musk’s X account.\nA search for news reports about Musk saying he would ban Volodymyr Zelenskyy from X yielded no results.\nThe spelling of Zelenskyy’s name in the screenshot is not the same spelling Musk uses on X when cr...,True,False,False,False,Manual,False
41,41,Politifact,https://www.politifact.com/factchecks/2025/feb/19/donald-trump/did-ukraine-start-its-war-with-russia-as-president/,Volodymyr Zelenskyy “started” the war in Ukraine with Russia.,2025-02-18 00:00:00,2025-02-19 00:00:00,"""

- claim with "id" 21:
    - Add the date and the person of the supposed claim:
    Mexican President-elect Claudia Sheinbaum said on Sept. 17
    - Add the (wrong) information that she rejected the invitation by Zelenskyy 
    - Claim will be reformulated in:
        - "Mexican President-elect Claudia Sheinbaum rejected an invitation to Ukraine by Volodymr Zelenskyy and said towards him on September 17, 2024: 'If I wanted to see a drug addict, I'd just take a walk around Tijuana.'"

- claim with "id" 38:
    - Add the full name and title of Zelenskyy
    - Add the context: US presidential election in 2024
    - Claim will be reformulated in:
        - "Ukrainian President Volodymyr Zelenskyy engaged in election interference by campaigning in Pennsylvania for Kamala Harris ahead of the US presidential election in 2024."


- claim with "id" 39:
    - Add the context: Jamie White died shortly after midnight on March 10, 2025 after being shot outside his apartment complex in Austin, Texas. 
    - Claim will be reformulated in:
        - “After InfoWars reporter Jamie White was shot dead on March 10, 2025 in Austin, Texas, it was revelaed that he was on a Ukranian enemies hit list."
        
    

- claim with "id" 40:
    - The claim is from February 2025
    - In case Zelenskyy's account could have suspended after February (although unlikely), the month of the claim will be added
    - Claim will be reformulated in:
        - "In February 2025 Elon Musk posted on X that he is planning to suspend Ukraine President Volodymyr Zelenskyy’s X account.”


- claim with "id" 41:
    - Add February 2022 to make clear that the invasion is meant and not the war since 2014
    - Claim will be reformulated in: 
        "Ukrainian President Volodymyr Zelenskyy started the 2022 war in Ukraine with Russia.”

In [22]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_3 = {

    21: {
        "Claim": "Mexican President-elect Claudia Sheinbaum rejected an invitation to Ukraine by Volodymr Zelenskyy and said towards him on September 17, 2024: 'If I wanted to see a drug addict, I'd just take a walk around Tijuana.'"
    },

    38: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy engaged in election interference by campaigning in Pennsylvania for Kamala Harris ahead of the US presidential election in 2024."
        },

    39: {
        "Claim": "After InfoWars reporter Jamie White was shot dead on March 10, 2025 in Austin, Texas, it was revelaed that he was on a Ukranian enemies hit list."
    },

    40: {
        "Claim": "In February 2025 Elon Musk posted on X that he is planning to suspend Ukraine President Volodymyr Zelenskyy’s X account."
    },

    41: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy started the 2022 war in Ukraine with Russia."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_3)

5

In [23]:
# The next 5 claims (claim 15-20)

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][15:20]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
42,42,Politifact,https://www.politifact.com/factchecks/2025/feb/05/facebook-posts/zelenskyys-statement-about-ukraine-aid-didnt-revea/,Ukrainian President Volodymyr Zelenskyy “just threw the whole Ukraine money laundering operation under the bus … by stating he’s received less than half of the over $177 billion in aid sent by the United States.”,2025-02-02 00:00:00,2025-02-05 00:00:00,"""Ukraine""",false,NaN,"""(Zelenskyy) just threw the whole Ukraine money laundering operation under the bus … by stating he’s received less than half of the over $177 billion in aid sent by the United States,"" one Facebook post said.",Ukrainian President Volodymyr Zelenskyy “just threw the whole Ukraine money laundering operation under the bus … by stating he’s received less than half of the over $177 billion in aid sent by the United States.”,"Ukrainian President Volodymyr Zelenskyy said his country’s military has received only a portion of the more than $175 billion in U.S. aid earmarked for Ukraine’s war against Russia. Now, critics of that U.S. support are floating an unsupported cl...",True,False,False,False,Manual,False
43,43,Politifact,https://www.politifact.com/factchecks/2024/nov/26/instagram-posts/theres-no-evidence-half-of-all-ukrainian-aid-was-l/,"""Half of all Ukrainian aid was laundered and sent back to Democrat candidates for re-election.""",2024-11-25 00:00:00,2024-11-26 00:00:00,"""Ukraine""",false,NaN,"In an Instagram video, conservative pundit Jeremy Herrell said a former Polish minister said ""nearly half of all the money that we sent to Ukraine was siphoned off and sent back to the Democrat Party and their candidates.""","""Half of all Ukrainian aid was laundered and sent back to Democrat candidates for re-election.""",There's no evidence half of all Ukrainian aid was laundered to fund Democratic political campaigns.\nThere’s no evidence that half of the U.S. aid to Ukraine was diverted to fund Democratic political campaigns. \nThe U.S. government oversees equi...,True,False,False,False,Manual,False
44,44,Politifact,https://www.politifact.com/factchecks/2024/sep/30/social-media/no-link-between-ukraine-and-helene-aid-and-joe-bid/,“Biden Announces: ‘2.4 billion more to Ukraine’ but ‘no more aid for Hurricane Helene.’”,2024-09-30 00:00:00,2024-09-30 00:00:00,"""Ukraine""",false,NaN,"""Biden announces ‘$2.4 billion more to Ukraine"" but ""No more aid for Hurricane Helene,"" a Sept. 30 Facebook post said.",“Biden Announces: ‘2.4 billion more to Ukraine’ but ‘no more aid for Hurricane Helene.’”,"No link between Ukraine and Helene aid, and Joe Biden didn’t rule out future hurricane aid\nIn Sept. 29 remarks to reporters, President Joe Biden did not say no more aid would be provided for areas affected by Hurricane Helene. He also did not ti...",True,False,False,False,Manual,False
45,45,Politifact,https://www.politifact.com/factchecks/2024/sep/20/facebook-posts/colin-kaepernick-isnt-russia-bound-this-claim-orig/,Colin Kaepernick said he “will go to Russia if not properly respected in the United States.”,2024-09-13 00:00:00,2024-09-20 00:00:00,"""Russia""",false,NaN,"""Colin Kaepernick: ‘Will go to Russia if not respected properly in the United States,’"" a Sept. 13 post said.",Colin Kaepernick said he “will go to Russia if not properly respected in the United States.”,Colin Kaepernick isn’t Russia-bound; this claim originated on a self-described satire site.,True,False,False,False,Manual,False
46,46,misbar.com,https://www.misbar.com/en/factcheck/2025/04/14/netanyahu-did-not-announce-plans-relocate-gazans-ukraine-and-syria,Netanyahu Did Not Announce Plans to Relocate Gazans to Ukraine and Syria,2025-04-14 00:00:00+00:00,2025-04-14 00:00:00+00:00,Ukraine,misleading,NaN,NaN,Netanyahu recent

- claim with "id" 42:
    - Reformulate the claim in a way that it covers the intent/meaning of it: Zelenskyy's statement about US Ukraine aid would reveal it as a money laundering operation
    - Zelenskyy's statement was made in a AP news interview from February 1, 2025
    - Claim will be reformulated in:

        - "Ukrainian President Volodymyr Zelenskyy's statement in an AP news interview from February 1, 2025 that his country's military has received only a portion of the more than 175 billion in U.S. aid confirms that the US aid is a money laundering operation."



- claim with "id" 43:
    - Clarify that US aid to Ukraine is meant
        - "Half of all US aid to Ukraine was laundered and sent back to Democrat candidates for re-election."


- claim with "id" 44:
    - Context: President Joe Biden made remarks to reporters on Sept. 29 
    - Add the context
    “Biden Announces: ‘2.4 billion more to Ukraine’ but ‘no more aid for Hurricane Helene.’”
    - Claim will be reformulated in:
        - “In remarks to reporters on September 29, 2024 US President Joe Biden announced that no more aid would be provided for the areas affected by Hurrican Helene, but 2.4 billion US-$ more for Ukraine."
        
    

- claim with "id" 45:
    - Remove the direct quote from the claim (not necessary)
    - Add that he is an American football player
    - Claim will be reformulated in:
        - "American football player Colin Kaepernick said he will go to Russia if not properly respected in the United States.”


- claim with "id" 46:
    - Context: Israeli Prime Minister Benjamin Netanyahu's visit to the White House in April 2025
    - Add this context to the claim and remove the "recently" to make it more precise
    - Claim will be reformulated in: 
        "During his visit to the White House in April 2025 Israeli Prime Minister Benjamin Netanyahu announced plans to resettle Palestinians in Gaza to Ukraine and Syria.”

In [24]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_4 = {

    42: {
        "Claim": "Ukrainian President Volodymyr Zelenskyy's statement in an AP news interview from February 1, 2025 that his country's military has received only a portion of the more than 175 billion in U.S. aid confirms that the US aid is a money laundering operation."
    },

    43: {
        "Claim": "Half of all US aid to Ukraine was laundered and sent back to Democrat candidates for re-election."
        },

    44: {
        "Claim": "In remarks to reporters on September 29, 2024 US President Joe Biden announced that no more aid would be provided for the areas affected by Hurrican Helene, but 2.4 billion US-$ more for Ukraine."
    },


    45: {
        "Claim": "American football player Colin Kaepernick said he will go to Russia if not properly respected in the United States."
    },

    46: {
        "Claim": "During his visit to the White House in April 2025 Israeli Prime Minister Benjamin Netanyahu announced plans to resettle Palestinians in Gaza to Ukraine and Syria."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_4)

5

In [25]:
# The next 5 claims (claim 20-25)

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][20:25]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
50,50,misbar.com,https://www.misbar.com/en/factcheck/2025/02/23/germany-has-not-threatened-cut-ties-us-after-talks-russia,Germany Has Not Threatened To Cut Ties With the U.S. After Talks With Russia,2025-02-23 00:00:00+00:00,2025-02-23 00:00:00+00:00,Ukraine,fake,NaN,NaN,German Foreign Minister Baerbock threatens US with severing ties amid Trump's comments on Zelensky and Ukraine.,"Germany’s Foreign Minister Criticized Europe’s Exclusion From U.S.-Russia Talks. The claim emerged after Germany’s Foreign Minister, Annalena Baerbock, criticized the exclusion of European countries from U.S.-Russia talks on the Ukrainian war, wh...",True,False,False,False,API,False
51,51,misbar.com,https://www.misbar.com/en/factcheck/2025/02/20/germany%E2%80%99s-olaf-scholz-has-not-declared-state-emergency-over-trump%E2%80%99s-ukraine-plan,Germany’s Olaf Scholz Has Not Declared a State of Emergency Over Trump’s Ukraine Plan,2025-02-20 00:00:00+00:00,2025-02-20 00:00:00+00:00,Ukraine,misleading,NaN,NaN,German Chancellor Olaf Scholz declared a state of emergency in Germany due to Trump's peace plan for Ukraine.,"Contrary to the viral claim, German Chancellor Olaf Scholz did not call for a state of emergency in Germany after Trump’s recent peace talks. A closer review of Scholz’s speech reveals that he referred to “Notlage,” not “Notstand” as claimed. In...",True,False,False,False,API,Misleading
52,52,misbar.com,https://www.misbar.com/en/factcheck/2025/02/16/trump-and-putin-have-not-reached-agreement-end-russia-ukraine-war,Trump and Putin Have Not Reached an Agreement to End Russia-Ukraine War,2025-02-16 00:00:00+00:00,2025-02-16 00:00:00+00:00,Ukraine,misleading,NaN,NaN,President Trump: I have reached an agreement with Putin to end the war in Ukraine.,"Misbar’s team searched for the claimed statement from Trump but did not find any reports from credible media outlets. However, Trump said in a post on his Truth Social platform that he had a call with Putin on February 12, 2025, during which the...",True,False,False,False,API,Misleading
53,53,newsweek.com,https://www.newsweek.com/fact-check-trump-russia-ukraine-war-losses-troops-2030104,Fact Check: Trump Says Russia Has Lost 1.5 Million Troops In Ukraine War,2025-02-14 00:00:00+00:00,2025-02-14 00:00:00+00:00,Ukraine,false,NaN,NaN,Trump says Russia has lost 1.5 million troops In Ukraine war,"For the benefit of the doubt, and as it wasn't explicit, Newsweek has treated the term ""lost"" to mean losses from those killed or injured. Trump said last month that 1.7 million soldiers across both sides had been killed, as reported by Politico...",True,False,False,False,API,False
54,54,newsweek.com,https://www.newsweek.com/fact-check-tucker-carlson-ukraine-weapons-black-market-cartels-2030011,Fact Check: Are Ukrainian Weapons Being Sold on Black Market?,2025-02-12 00:00:00+00:00,2025-02-12 00:00:00+00:00,Ukraine,false,NaN,NaN,Ukraine is selling half of the armaments it receives from the United States,"Claims that NATO weapons and ammunition are being put on sale on the dark web have re-surfaced in recent months, but as other outlets have reported, there is no evidence that any such products have actually made it into the hands of a prospectiv...",True,False,False,False,API,False


- claim with "id" 50:
    - Context: The statement was made in February 2025. Add this as a date
    - Claim will be reformulated in:
        - "In February 2025 German Foreign Minister Annalena Baerbock threatened the US with severing ties amid US President Donald Trump's comments on Ukrainian President Volodymyr Zelenskyy and Ukraine."



- claim with "id" 51:  
    - Context: Olaf Scholz's statment was made in February 2025. Add this as a date
    - Claim will be reformulated in:
        - "In February 2025 German Chancellor Olaf Scholz declared a state of emergency in Germany due to US President Donald Trump's peace plan for Ukraine."


- claim with "id" 52:
    - Context: Trump's call with Putin on February 12, 2025
    - Add this context to claim
    - Claim will be reformulated in:
        - “After a call with Russia's President Vladimir Putin on February 12, 2025, US President Donald Trump announced that he had reached an agreement to end the war in Ukraine."
        
    

- claim with "id" 53:
    - Context: Trump made this claim at a press conference in the White House, on February 11, 2025
    - It is true that Trump made this claim, but the claim is labeled "False", because the factual content of the claim is wrong.
    - Therefore, the claim will be reformulated to focus on the facts.
    - February 2025 will be added since a date reference is necessary when talking about (false) amounts of Russian troops lost in Ukraine
    - Claim will be reformulated in:
        - "As of February 2025, Russia has lost 1.5 million troops in the Ukraine war."
        


- claim with "id" 54:
    - No reformulation is neccesary. 

In [26]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_5 = {

    50: {   
        "Claim": "In February 2025 German Foreign Minister Annalena Baerbock threatened the US with severing ties amid US President Donald Trump's comments on Ukrainian President Volodymyr Zelenskyy and Ukraine."
    },

    51: {
        "Claim": "In February 2025 German Chancellor Olaf Scholz declared a state of emergency in Germany due to US President Donald Trump's peace plan for Ukraine."
        },

    52: {
        "Claim": "After a call with Russia's President Vladimir Putin on February 12, 2025, US President Donald Trump announced that he had reached an agreement to end the war in Ukraine."
    },


    53: {
        "Claim": "As of February 2025, Russia has lost 1.5 million troops in the Ukraine war."
    },

    54: {
        "Claim": "Ukraine is selling half of the armaments it receives from the United States."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_5)

5

In [27]:
# The next 5 claims (claim 25-30)

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][25:30]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
55,55,checkyourfact.com,https://checkyourfact.com/2025/02/12/fact-check-e-news-usaid-celebrities-millions-ukraine/,"FACT CHECK: No, E! News Did Not Report USAID Paid Celebrities Millions To Visit Ukraine",2025-02-05 00:00:00+00:00,2025-02-12 00:00:00+00:00,Ukraine,false,NaN,NaN,A post shared on X claims E! News reported that United States Agency Aid and Development (USAID) paid millions for celebrities to visit Ukraine.,There is no evidence for this claim. E! News does not have this alleged reporting on its website. No media outlets have referenced this alleged reporting nor have reported on this allegation that USAID paid celebrities to visit Ukraine.,True,False,False,False,API,False
57,57,checkyourfact.com,https://checkyourfact.com/2025/02/05/fact-check-trump-criticize-starmer-deal-uk-ukraine/,"FACT CHECK: No, Donald Trump Did Not Criticize Keir Starmer Over A 100-Year Deal Between The UK And Ukraine",2025-01-21 00:00:00+00:00,2025-02-05 00:00:00+00:00,Ukraine,false,NaN,NaN,claims Trump purportedly criticized British PM Keir Starmer over a 100-year deal between the U.K. and Ukraine,"The claim is false. While Trump did sit down with Von for an interview back in August 2024, he did not mention Starmer or the U.K.’s 100-year deal with Zelenskyy. Likewise, Trump does not appear to have publicly referenced the claim via his perso...",True,False,False,False,API,False
58,58,newsweek.com,https://www.newsweek.com/fact-check-does-zelensky-have-four-percent-approval-rating-2033424,Fact Check: Does Zelensky Have a 4% Approval Rating?,2025-02-19 00:00:00+00:00,2025-02-19 00:00:00+00:00,Ukraine,false,NaN,NaN,Donald Trump alleged Volodymyr Zelensky has low approval ratings,"According to a recent survey by the Kyiv International Institute of Sociology (KIIS) published in early February, 57 percent of Ukrainians said they trust Zelensky. The rating marks a slight increase from the KIIS's December 2024 polls, conducte...",True,False,False,False,API,False
59,59,checkyourfact.com,https://checkyourfact.com/2025/01/16/fact-check-no-eight-mansions-belonging-to-ukrainian-military-officials-did-not-burn-down-in-la/,"FACT CHECK: No, Eight Mansions Belonging To Ukrainian Military Officials Did Not Burn Down In LA",2025-01-11 00:00:00+00:00,2025-01-16 00:00:00+00:00,Ukraine,false,NaN,NaN,"A post shared on X claims eight mansions belonging to Ukrainian military officials burned down in Los Angeles, California.","There is no evidence of this claim. If eight mansions belonging to Ukrainian military officials burned down, media outlets would have covered it, yet none have. A wider internet search also did not yield any results for this claim, either. ”",True,False,False,False,API,False
62,62,rappler.com,https://www.rappler.com/newsbreak/fact-check/vladimir-putin-alive-not-dead-moscow-limousine-explosion/,"FACT CHECK: Putin is alive, not killed in Moscow limo explosion",2025-04-17 00:00:00+00:00,2025-04-17 00:00:00+00:00,Russia,false,NaN,NaN,Russian President Vladimir Putin has died in an explosion in one of his limousines.,"Putin is alive. On April 11, Putin met US envoy Steve Winkoff in Saint Petersburg to discuss “various aspects of the Ukrainian settlement.” The Kremlin leader also recently met Russian nationals who were freed following their captivity in the Ga...",True,False,False,False,API,False


- claim with "id" 55:
    - Just remove the first part of the claim (a post shared on X) -> unnecessary
    - Claim will be reformulated in:
        - "E! News reported that the United States Agency Aid and Development (USAID) paid millions for celebrities to visit Ukraine."



- claim with "id" 57:  
    - Context: Trump sit down with Pocaster Theo Von for an interview back in August 2024
    - Add the context to the claim
    - Claim will be reformulated in:
        - "In an interview with podcaster Theo Von in August 2024, Donald Trump criticized British Prime Minister Keir Starmer over a 100-year deal between the U.K. and Ukraine."


- claim with "id" 58:
    - The claim is a duplicate. It has been reformulated before.
    - Thus, this sample will be removed from the dataset.
        
    

- claim with "id" 59:
    - Context: California wildfires January 2025: (https://en.wikipedia.org/wiki/January_2025_Southern_California_wildfires)
    - Add this context information to claim
    - Remove the first, unnecessary part of the claim (A post shared on X claims....)
    - Claim will be reformulated in:
        - "During the January 2025 wildfires, eight mansions belonging to Ukrainian military officials burned down in Los Angeles, California."
        


- claim with "id" 62:
    - No reformulation is neccesary. 

In [28]:
# Remove the duplicate claim with "id" 58

## 1) Check the length of the ukraine-russia df before the removal

print(len(df_ukraine_russia))

## 2) Peform removal

sample = df_ukraine_russia[df_ukraine_russia["id"]== 58].index

df_ukraine_russia.drop(sample, inplace=True)

## 3) Check the length of the ukraine-russia df after the removal

print(len(df_ukraine_russia))

80
79


In [29]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_6 = {

    55: {   
        "Claim": "E! News reported that the United States Agency Aid and Development (USAID) paid millions for celebrities to visit Ukraine."
    },

    57: {
        "Claim": "In an interview with podcaster Theo Von in August 2024, Donald Trump criticized British Prime Minister Keir Starmer over a 100-year deal between the U.K. and Ukraine."
        },


    59: {
        "Claim": "During the January 2025 wildfires, eight mansions belonging to Ukrainian military officials burned down in Los Angeles, California."
    },

    62: {
        "Claim": "Russian President Vladimir Putin has died in an explosion in one of his limousines."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_6)

4

In [30]:
# The next 6 claims (claim 29-35) -> Need to make 6, because of the drop of one observation in the previous sample above. 
# Otherwise the observation with id "64" would get lost.

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][29:35]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
64,64,checkyourfact.com,https://checkyourfact.com/2025/03/19/fact-check-tulsi-trump-putin-good-friends/,FACT CHECK: Did Tulsi Gabbard Say Trump And Putin Were Good Friends?,2025-03-17 00:00:00+00:00,2025-03-19 00:00:00+00:00,Russia,false,NaN,NaN,A post shared on X claims Director of National Intelligence (DNI) Tulsi Gabbard said that President Donald Trump and Russian President Vladimir Putin were good friends.,"This claim is false. Gabbard was talking about Modi being friends with Trump, not Putin. She made these remarks during an interview with NDTV. While some outlets reported that she was talking about Trump and Putin, others corrected it. For exampl...",True,False,False,False,API,False
65,65,misbar.com,https://www.misbar.com/en/factcheck/2025/02/27/macron-did-not-call-putin-butcher-his-press-conference-trump,Macron Did Not Call Putin a Butcher in His Press Conference With Trump,2025-02-27 00:00:00+00:00,2025-02-27 00:00:00+00:00,Russia,misleading,NaN,NaN,"French President Emmanuel Macron called Russian President Vladimir Putin a “butcher” during his press conference with Donald Trump on Monday, Feb. 24, 2025.","Users Misunderstood Macron’s Reference to the Bucha Massacre Misbar’s team reviewed the viral video some users shared supporting their claim and found that Macron was referring to the Bucha massacre During the press conference, a journalist aske...",True,False,False,False,API,Misleading
66,66,newsweek.com,https://www.newsweek.com/fact-check-did-jd-vance-say-us-military-action-available-against-russia-2032348,Fact Check: Did JD Vance Say US 'Military Action' Available Against Russia?,2025-02-18 00:00:00+00:00,2025-02-18 00:00:00+00:00,Russia,true,NaN,NaN,"Did JD Vance say US ""military action"" available against Russia?","In an interview with The Wall Street Journal published on February 14, Vance was asked about security guarantees the U.S. could provide Ukraine as part of a peace settlement with Russia, and what ""instruments of pressure"" could ensure Moscow's c...",True,False,False,False,API,True
67,67,rappler.com,https://www.rappler.com/newsbreak/fact-check/no-russia-invasion-china-japan-february-2025/,FACT CHECK: No invasion of Russia by China and Japan,2025-02-10 00:00:00+00:00,2025-02-10 00:00:00+00:00,Russia,false,NaN,NaN,China and Japan plan to invade the Russian Federation,There are no reports from the foreign affairs and defense ministries of China and the foreign affairs and defense ministries of Japan about a supposed joint invasion of Russia. The video is a mere discussion of the geopolitical situation between...,True,False,False,False,API,False
68,68,newsweek.com,https://www.newsweek.com/fact-check-has-russia-offered-assistance-california-wildfire-disaster-2014213,Fact Check: Has Russia Offered Assistance to California Wildfire Disaster?,2025-01-21 00:00:00+00:00,2025-01-21 00:00:00+00:00,Russia,unverified,NaN,NaN,Has Russia offered assistance to California wildfire disaster?,"Newsweek has not found any statement from Russian officials offering support to California. There are no official public statements from Sergey Shchetinin, named on Russian government websites as the Chairman of the Executive Committee of the Ru...",True,False,False,False,API,NEI
69,69,usatoday.com,https://www.usatoday.com/story/news/factcheck/2025/01/16/adam-schiff-senate-committees-fact-check/77718124007/,"No, Adam Schiff wasn’t blocked from Senate committees | Fact check",2025-01-11 00:00:00+00:00,2025-01-16 00:00:00+00:00,Russia,false,NaN,NaN,John Thune blocked Adam Schiff's assignments over Russia investigation,"There is no evidence Senate Majority Leader John Thune blocked Sen. Adam Schiff from any committee assignments. Schiff is listed as a member of four Senate committees

- claim with "id" 64:
    - Context: Tulsi Gabbard gave an interview to NDTV on March 17, 2025 (https://www.ndtv.com/world-news/trump-focused-on-peace-tulsi-gabbard-on-russia-ukraine-war-7941806)
    - Add this context to the claim
    - Remove the first, unnecessary part of the claim (A post shared on X claims)
    - Claim will be reformulated in:
        "In an interview with NDTV on March 17, 2025 Director of US National Intelligence Tulsi Gabbard said that President Donald Trump and Russian President Vladimir Putin were good friends."


- claim with "id" 65:
    - No reformulation necessary.



- claim with "id" 66:  
    - Context: JD Vance gave an interview to the The Wall Street Journal published on February 14, 2025
    - Add the context to the claim 
    - Reformulate the claim from a question to a statement
    - Claim will be reformulated in:
        - "In an interview with the Wall Street Journal, published on February 14, 2025, US Vice President J.D. Vance said the US military action is available against Russia."


- claim with "id" 67:
    - No reformulation necessary.
        
    

- claim with "id" 68:
    - Context: California wildfire January 2025
    - Add the context
    - Reformulate the claim from a question to a statement
    - Claim will be reformulated in:
        - "Russia offered assistance for the California wildfires from January 2025."
        


- claim with "id" 69:
    - Context: A Jan. 11 Threads post claims a newly elected senator has been singled out because of his past work investigating President-elect Donald Trump (Russia investigation)
    - Add this context
    - Also add that both discussed people are Senators.
    - Claim will be reformulated in: 
        - "Majority Leader John Thune blocked Senator Adam Schiff's committee assignments because of his past work in the Russia investigation over Donald Trump."

In [31]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_7 = {

    64: {
        "Claim": "In an interview with NDTV on March 17, 2025 Director of US National Intelligence Tulsi Gabbard said that President Donald Trump and Russian President Vladimir Putin were good friends."
    },

    65: {   
        "Claim": "French President Emmanuel Macron called Russian President Vladimir Putin a 'butcher' during his press conference with Donald Trump on Monday, Feb. 24, 2025."
    },

    66: {
        "Claim": "In an interview with the Wall Street Journal, published on February 14, 2025, US Vice President J.D. Vance said the US military action is available against Russia."
        },

    67: {
        "Claim": "China and Japan plan to invade the Russian Federation."
    },


    68: {
        "Claim": "Russia offered assistance for the California wildfires from January 2025."
    },

    69: {
        "Claim": "Majority Leader John Thune blocked Senator Adam Schiff's committee assignments because of his past work in the Russia investigation over Donald Trump."
    },
}

len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_7)

6

In [32]:
# The last 4 claims (claim 36-39)

df_ukraine_russia[(df_ukraine_russia["Text_Only_Claim"] == True) & (df_ukraine_russia["Website"] != "Snopes")][35:]

,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label
71,71,rappler.com,https://www.rappler.com/newsbreak/fact-check/russia-did-not-declare-war-united-states/,FACT CHECK: Russia did not declare war on US,2024-12-02 00:00:00+00:00,2024-12-02 00:00:00+00:00,Russia,false,NaN,NaN,Russian President Vladimir Putin has declared war on the United States.,Russia has not declared war on the US. There are no reports from credible media outlets or government statements confirming the claim. The video merely narrates recent developments in the ongoing conflict between Russia and Ukraine.,True,False,False,False,API,False
72,72,checkyourfact.com,https://checkyourfact.com/2024/10/28/fact-check-did-russia-become-the-fourth-largest-economy-in-the-world/,FACT CHECK: Did Russia Become The Fourth Largest Economy in the World?,2024-10-23 00:00:00+00:00,2024-10-28 00:00:00+00:00,Russia,false,NaN,NaN,A post on X claims that the IMF has declared Russia the fourth-largest economy in the world.,"The claim is lacking context. When adjusted for adjusted for PPP, Russia ranks fourth. The metric, according to Investopedia, is a “macroeconomic analysis metric” that compares economic productivity and standards of living to other countries usi...",True,False,False,False,API,False
73,73,newsweek.com,https://www.newsweek.com/pete-hegseth-vladimir-putin-war-criminal-1985251,Fact Check: Did Pete Hegseth Call Putin a 'War Criminal?',2024-11-13 00:00:00+00:00,2024-11-13 00:00:00+00:00,Russia,needs context,NaN,NaN,Did Pete Hegseth call Putin a 'war criminal'?,"While Hegseth aligned with the claim that Putin was a ""war criminal,"" the clip shared online was from March 2022 not long after the outbreak of the conflict. Since then Hegseth has been more cautious, and critical of the U.S. intervention in Ukr...",True,False,False,False,API,Misleading
77,77,misbar.com,https://www.misbar.com/en/factcheck/2024/08/22/joe-biden-updates-nuclear-strategy-no-imminent-nuclear-war-orders-issued,"Joe Biden Updates Nuclear Strategy, No Imminent Nuclear War Orders Issued",2024-08-22 00:00:00+00:00,2024-08-22 00:00:00+00:00,Russia,excitement,NaN,NaN,"Biden ordered U.S. forces to prepare for possible nuclear confrontations with Russia, China, and North Korea.","New York Times' Article Misinterpreted The article mentioned that the United States is considering increasing its nuclear arsenal, reversing decades of cutbacks due to concerns about China and Russia. This transition, driven by fears of China's ...",True,False,False,False,API,True


- claim with "id" 71:
    - No reformulation necessary.



- claim with "id" 72:  
    - The claim date is from October 2024
    - Use this month as a date, because a reference is neccesary as economic figures of states change over time.
    - Remove the first, unnecessary part of the claim (A post on X claims that )
    - Claim will be reformulated in:
        - "In October 2024 the International Monetary Fund (IMF) declared Russia the fourth-largest economy in the world."


- claim with "id" 73:
    - Context: The claim has the label "Misleading", because the claim suggests that Pete Hugseth called Putin a "war criminal" recently, as part of being a US minister. However, he did make this statement, but in March 2022 - where we was not a minister.
    - Hence, a reference that he (allegedly) said it as a minister is necessary
    - Reformulate the claim from a question to a statement
    - Claim will be reformulated in:
        - "US Secretary of Defense Pete Hegseth called Putin a 'war criminal'."
        
    

- claim with "id" 77:
    - Context: The NYT article is from August 20, 2024
    - The classified document was approved in March 2024 according to the article
    - Make clear that Biden ordered this decision in 2024, while he was still being US President
    - Claim will be reformulated in:
        - "While serving as US President in 2024, Joe Biden ordered the US forces to prepare for possible nuclear confrontations with Russia, China, and North Korea."
        


In [33]:
ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_8 = {

    71: {   
        "Claim": "Russian President Vladimir Putin has declared war on the United States."
    },

    72: {
        "Claim": "In October 2024 the International Monetary Fund (IMF) declared Russia the fourth-largest economy in the world."
        },

    73: {
        "Claim": "US Secretary of Defense Pete Hegseth called Putin a 'war criminal'."
    },

    77: {
        "Claim": "While serving as US President in 2024, Joe Biden ordered the US forces to prepare for possible nuclear confrontations with Russia, China, and North Korea."
}
}


len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_8)

4

In [34]:
## Combine all the dictionaries of the text only remaining claims


ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_combined = ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_1 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_2 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_3 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_4 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_5 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_6 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_7 | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_part_8
print(len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_combined))


39


In [35]:
## Combine all dictionaries with reformulated claims (altered images, normal images, text only claims (Snopes, remaining))

## Checking the length of the per category combined dictionaries first
print(len(ukraine_russia_dictionary_reformulated_claims_altered_images))
print(len(ukraine_russia_dictionary_reformulated_claims_normal_images_combined))
print(len(ukraine_russia_dictionary_reformulated_claims_text_only_snopes_combined))
print(len(ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_combined))

## Combine all together 

ukraine_russia_dictionary_all_reformulated_claims = ukraine_russia_dictionary_reformulated_claims_altered_images | ukraine_russia_dictionary_reformulated_claims_normal_images_combined | ukraine_russia_dictionary_reformulated_claims_text_only_snopes_combined | ukraine_russia_dictionary_reformulated_claims_text_only_remaining_claims_combined
print(len(ukraine_russia_dictionary_all_reformulated_claims)) ## length is correct (one duplicate claim was dropped from df)
ukraine_russia_dictionary_all_reformulated_claims

9
17
14
39
79


{0: {'Claim': "This image shows a screenshot of an authentic Hull Daily Mail's frontpage from March 13, 2025 saying '70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure'."},
 3: {'Claim': 'This image shows a screenshot of an authentic cover of the magazine The Economist from the end of 2024 illustrating Donald Trump and Vladimir Putin and warning of an apocalypse.'},
 12: {'Claim': 'This is an authentic image of world leaders taking off their suits to symbolically support Ukraine at the London summit on Ukraine on March 2, 2025.'},
 14: {'Claim': "This image shows a screenshot of an authentic Fox News article with the title 'Psychological and Information Warfare Center of Ukraine received $140,000,000 per month from USAID'."},
 47: {'Claim': "This image shows a screenshot of an authentic Charlie Hebdo magazine cover from March 20, 2025 illustrating Ukrainian President Volodoymr Zelenskyy lying in a co

In [36]:
## Adding the dictionary data to the df (Use the function from gaza-israel again)

import numpy as np

def update_df_with_reformulated_claims(df, data_dict, id_column = "id"):


    # Create a copy of the df to avoid modifying the original
    df_updated = df.copy()

    # Create the "Claim" column with the reformulated claims and fill it with nan values to track if a claim misses
    df_updated["Claim"] = np.nan

    # Track IDS not found in the df
    ids_not_found = []

    # Loop through each claim in the dicationary
    for index, data in data_dict.items():
        # Find rows where id column matches the dictionary key
        match = df_updated[id_column] == index

        # If a matching row is found, add the reformulated claims 
        if match.any():
            df_updated.loc[match, "Claim"] = data["Claim"]

        else:
            ids_not_found.append(index)

    
    # Pritn IDs not found (if any exist)
    if ids_not_found:
        print(f"The following IDs were not found in the df: {ids_not_found}")

    return df_updated

In [37]:
df_ukraine_russia_updated = update_df_with_reformulated_claims(df = df_ukraine_russia, data_dict=ukraine_russia_dictionary_all_reformulated_claims)

# Check if any NA values still exist in new "Claim" column
print(df_ukraine_russia_updated["Claim"].isna().sum())

# Check column types again
print(df_ukraine_russia_updated.dtypes)

df_ukraine_russia_updated.head()

0
id                            int64
Website                      object
Article_URL                  object
Headline                     object
Claim_Date                   object
Review_Date                  object
Query/Keyword                object
Label_Website                object
Image_URL                    object
Original_Claim_Website       object
Original_Claim_Only          object
Context/Label_Explanation    object
Text_Only_Claim                bool
Normal_Image                   bool
AI_Generated_Image             bool
Altered_Image                  bool
Data_Collection_Type         object
Label                        object
Claim                        object
dtype: object


/var/folders/yf/8jt3z6xx21z0rv6lrj7sdr800000gn/T/ipykernel_9929/952152285.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'This image shows a screenshot of an authentic Hull Daily Mail's frontpage from March 13, 2025 saying '70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure'.' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_updated.loc[match, "Claim"] = data["Claim"]


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label,Claim
0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.372Y6CV,Fake newspaper cover on Ukrainian soldiers in Kursk spreads online,2025-03-15 00:00:00,2025-03-19 00:00:00,"""War in Ukraine""",altered,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/484304343_642451842044376_1862227891799489919_n.jpg?_nc_cat=102&ccb=1-7&_nc_sid=127cfc&_nc_ohc=vNbhgJpxh8gQ7kNvwGpkp-E&_nc_oc=AdmpyxpVjf7biRJsqu7qYnoOgqPFCs_D4iP6ar6puC1wqX6YNlcFafIdG-8nnXYzjxc&_...,"""The Kursk expedition was a disaster and a complete waste of life. Britain egged it on,"" says a March 15, 2025 post on X from Ian Miles Cheong, a far-right blogger who has previously spread other misinformation -- and whose posts have been repeat...","""70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure.""","But the supposed Hull Daily Mail headline blasting Zelensky over the Kursk incursion is a fabrication.\n""That story re Ukraine was a fake,"" said Lija Kresowaty, a spokesperson for Reach PLC, the Hull Daily Mail's parent company, in a March 18 ema...",False,False,False,True,Manual,False,"This image shows a screenshot of an authentic Hull Daily Mail's frontpage from March 13, 2025 saying '70,000 Ukrainian soldiers in the Kursk region died in vain. The UK poured hundreds of millions of pounds into Zelensky's crushing failure'."
1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YR9KZ,"No, Zelensky hasn't bought Eagle's Nest, it is still in ownership of Bavarian state",2025-02-18 00:00:00,2025-02-27 00:00:00,"""War in Ukraine""",false,NaN,"According to the latest claims, Zelensky allegedly purchased another site steeped in history: the so-called Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros.","According to the latest claims, Zelensky allegedly purchased another site steeped in history: the so-called Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros.","The Eagle's Nest is in the property of the state of Bavaria. The administration of state-owned real estate is the responsibility of IMBY, which is listed in the alleged purchase agreement. On February 7, 2025, the company posted a press release o...",True,False,False,False,Manual,False,"Ukrainian President Volodymyr Zelenskyy purchased the Eagle's Nest, or Kehlsteinhaus, in Berchtesgaden in the southern German state of Bavaria, for 14.2 million euros."
2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36YC3DG,Claims that Ukraine banned Truth Social are false,2025-02-20 00:00:00,2025-02-21 00:00:00,"""War in Ukraine""",false,NaN,"""BREAKING: Zelensky blocks access to President Trump's social media platform Truth Social across Ukraine,"" says a February 20, 2025 X post from conservative influencer Benny Johnson, who AFP has previously fact-checked.","""BREAKING: Zelensky blocks access to President Trump's social media platform Truth Social across Ukraine.""","A spokesperson for Trump Media and Technology Group, which owns Truth Social, said the company has not launched the micro-blogging site in Ukraine.\n""Reports that Truth Social was blocked in Ukraine are false -- Truth Social has never been availa...",True,False,False,False,Manual,False,In February 2025 Ukrainian President Volodymyr Zelenskyy blocked access to US President Donald Trump's social media platform Truth Social across Ukraine.
3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36P98ZW,Fake 'apocalypse' cover of The Economist circulates online\n\n,2024-11-18 00:00:00,2024-12-03 00:00:00,"""War in Ukraine""",false,https://scontent-hou1-1.xx.fbcdn.net/v/t39.30808-6/472950587_12215133000236

In [38]:
## Need to reindex the claims, because one observation/claim (id: 58) was dropped from df

def reindex_df(df):
    # Reset the index to ensure it reflects the correct range (0 to 78)
    df = df.reset_index(drop=True)
    
    #update the 'id' column to match the new index
    df['id'] = df.index  
    
    return df

In [39]:
df_ukraine_russia_updated_reindexed = reindex_df(df_ukraine_russia_updated)
print(len(df_ukraine_russia_updated_reindexed)) #checked the length
df_ukraine_russia_updated_reindexed.tail(5) #check that the last indices are correct (last one should be 78 and not 79)

79


,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Label_Website,Image_URL,Original_Claim_Website,Original_Claim_Only,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,Label,Claim
74,74,checkyourfact.com,https://checkyourfact.com/2024/09/19/fact-check-small-nuclear-strike-russia/,FACT CHECK: Did A Small Nuclear Strike Hit Russia?,2024-09-18 00:00:00+00:00,2024-09-19 00:00:00+00:00,Russia,false,https://dims.apnews.com/dims4/default/810a90b/2147483647/strip/true/crop/19000x5594+0+0/resize/1440x424!/format/webp/quality/90/?url=https%3A%2F%2Fassets.apnews.com%2F26%2F32%2F8d546e1d085c24b0665a6f399dce%2Fce6c62f457764b4ea6b4ccd54e87a2e8,NaN,"Posts shared on X claims that a North Atlantic Treaty Organization (NATO) strike, possibly nuclear, hit a Russian ammunition depot.","This claim is false. The attack was not from NATO, but a Ukrainian drone strike, according to The Associated Press. A Kyiv security official told the outlet that the attack was carried out by Ukrainian Special Operation Forces, Security Service ...",False,True,False,False,API,False,"This image shows a Russian ammunition depot in Toropets, Tver Oblast, before and after it was hit by a, possible nuclear, strike from the North Atlantic Treaty Organization (NATO)."
75,75,misbar.com,https://www.misbar.com/en/factcheck/2024/08/26/an-old-image-of-ukrainians-circulated-as-american-mercenaries-invading-russiarsquos-kursk-along-with-ukraine,An Old Image of Ukrainians Circulated As American Mercenaries Invading Russia’s Kursk Along With Ukraine,2024-08-26 00:00:00+00:00,2024-08-26 00:00:00+00:00,Russia,misleading,https://pbs.twimg.com/media/GU1wcUMXAAAfRtb?format=png&name=900x900,NaN,American mercenaries exposed invading Russia's Kursk region These low-brainers provided evidence for their crimes before they were cremated.,"An Image Of A Ukrainian Solider Taken In Balakliia Through Google Reverse Image, Misbar’s team found that the image was originally posted in 2022 before Ukraine’s incursion into the Kursk region. In September 2022, Ukrainian journalist Denys Kaz...",False,True,False,False,API,Misleading,This image shows American mercenaries participating in Ukraine's incursion into Russia's Kursk Oblast in August 2024.
76,76,misbar.com,https://www.misbar.com/en/factcheck/2024/08/22/joe-biden-updates-nuclear-strategy-no-imminent-nuclear-war-orders-issued,"Joe Biden Updates Nuclear Strategy, No Imminent Nuclear War Orders Issued",2024-08-22 00:00:00+00:00,2024-08-22 00:00:00+00:00,Russia,excitement,NaN,NaN,"Biden ordered U.S. forces to prepare for possible nuclear confrontations with Russia, China, and North Korea.","New York Times' Article Misinterpreted The article mentioned that the United States is considering increasing its nuclear arsenal, reversing decades of cutbacks due to concerns about China and Russia. This transition, driven by fears of China's ...",True,False,False,False,API,True,"While serving as US President in 2024, Joe Biden ordered the US forces to prepare for possible nuclear confrontations with Russia, China, and North Korea."
77,77,misbar.com,https://www.misbar.com/en/factcheck/2024/08/20/poland-marks-historic-victory-over-the-red-army-not-preparing-for-war-with-russia,"Poland Marks Historic Victory Over the Red Army, Not Preparing for War With Russia",2024-08-20 00:00:00+00:00,2024-08-20 00:00:00+00:00,Russia,misleading,https://assets.misbar.com/styles/large_870x490/public/ca394822814b13f54385dea4721da7c695c8af33.jpg.webp?itok=2yKkTtOz,NaN,"POLAND The Army is preparing for war with Russia because We live in times of war, says Prime Minister.","Poland Celebrates Historic Red Army Defeat On August 15, the same footage appeared on DW with the comment: “With tensions rising at its eastern borders, Poland put on a big show with a military parade to celebrate Armed Forces Day. Those tension...",False,True,False,False,API,Misleading,"This image shows a military parade of Poland's arm

In [40]:
## Checking the final label distribution and claim type again

print(df_ukraine_russia_updated_reindexed["Label"].value_counts())

print(df_ukraine_russia_updated_reindexed["Text_Only_Claim"].value_counts())

print(df_ukraine_russia_updated_reindexed["Normal_Image"].value_counts())

print(df_ukraine_russia_updated_reindexed["AI_Generated_Image"].value_counts())

print(df_ukraine_russia_updated_reindexed["Altered_Image"].value_counts())



## Save the updated df for the next step: image scraping

df_ukraine_russia_updated_reindexed.to_csv("../ukraine_russia/Combined_dataset/ukraine_russia_dataset_combined_010724_300425_after_claim_reformulation.csv")

Label
False         53
Misleading    15
True           9
NEI            2
Name: count, dtype: int64
Text_Only_Claim
True     53
False    26
Name: count, dtype: int64
Normal_Image
False    62
True     17
Name: count, dtype: int64
AI_Generated_Image
False    79
Name: count, dtype: int64
Altered_Image
False    70
True      9
Name: count, dtype: int64
